# edge-connect with differentiable augmentation
edge-connect: [knazeri/edge-connect](https://github.com/knazeri/edge-connect)

Yukariins fork: [Yukariin/edge-connect](https://github.com/Yukariin/edge-connect)

Differentiable Augmentation: [mit-han-lab/data-efficient-gans](https://github.com/mit-han-lab/data-efficient-gans)

My fork: [styler00dollar/Colab-edge-connect](https://github.com/styler00dollar/Colab-edge-connect)

In [ ]:
# check gpu
!nvidia-smi

In [ ]:
#@title Install
%cd /content/
!git clone https://github.com/styler00dollar/Colab-edge-connect
!pip install numpy
!pip install scipy==1.1
!pip install future
!pip install matplotlib
!pip install pillow
!pip install opencv-python
!pip install scikit-image
!pip install pyaml
!pip install tensorboardX

# Test with pre-trained models

In [ ]:
#@title Download models
%cd /content/Colab-edge-connect
!pip install gdown
!mkdir checkpoint_places
%cd checkpoint_places
# /checkpoints broken
# places
!gdown --id 1gesVuuYMtlWSQRR2JE5eO0QZHskYRfqv
!gdown --id 1_oYnmK7kppXqka9UUsHrZB4gWE4ouSgT
!gdown --id 1M-r_ds4VZJnUqViDMofd4-Fy8-q2aeKJ
!gdown --id 1G8lXquU3eREfs8KorFpFC8N4YmTQRksF
%cd ..
!mkdir checkpoint_celeba
%cd checkpoint_celeba
!gdown --id 1wy0pEaXTqmya2yeLwWFmTBf4ICexCdce
!gdown --id 1hqZRjnqZBGnSTtGJRHXEvvdGVICUGa7u
!gdown --id 17FemN4FAKpS5-8Dos582IrOiSCZNDOAO
!gdown --id 15mH1ZHMf83q3woBHFELr_TptSRGc5g5j
%cd ..
!mkdir checkpoint_street
%cd checkpoint_street
!gdown --id 1ORF2uN4lB3F6YndPm1ny8VIDrsWQBwUS
!gdown --id 1EwHK8YjcpO-X3xhmeo2dtqGvtY5vOMMj
!gdown --id 1AWxB8AwTOrlOmAUho3IQQlmawtp3y8gZ
!gdown --id 12Ua8oQwk0iLdYgrb08bqBhfyiBIumQEK

In [ ]:
#@title dummy config
%%writefile /content/Colab-edge-connect/config.yml
MODE: 1             # 1: train, 2: test, 3: eval
MODEL: 1            # 1: edge model, 2: inpaint model, 3: edge-inpaint model, 4: joint model
MASK: 3             # 1: random block, 2: half, 3: external, 4: (external, random block), 5: (external, random block, half)
EDGE: 1             # 1: canny, 2: external
NMS: 1              # 0: no non-max-suppression, 1: applies non-max-suppression on the external edges by multiplying by Canny
SEED: 10            # random seed
GPU: [0]            # list of gpu ids
DEBUG: 0            # turns on debugging mode
VERBOSE: 0          # turns on verbose mode in the output console

TRAIN_FLIST: ./datasets/places2_train.flist
VAL_FLIST: ./datasets/places2_val.flist
TEST_FLIST: ./datasets/places2_test.flist

TRAIN_EDGE_FLIST: ./datasets/places2_edges_train.flist
VAL_EDGE_FLIST: ./datasets/places2_edges_val.flist
TEST_EDGE_FLIST: ./datasets/places2_edges_test.flist

TRAIN_MASK_FLIST: ./datasets/masks_train.flist
VAL_MASK_FLIST: ./datasets/masks_val.flist
TEST_MASK_FLIST: ./datasets/masks_test.flist

LR: 0.0001                    # learning rate
D2G_LR: 0.1                   # discriminator/generator learning rate ratio
BETA1: 0.0                    # adam optimizer beta1
BETA2: 0.9                    # adam optimizer beta2
BATCH_SIZE: 8                 # input batch size for training
INPUT_SIZE: 256               # input image size for training 0 for original size
SIGMA: 2                      # standard deviation of the Gaussian filter used in Canny edge detector (0: random, -1: no edge)
MAX_ITERS: 2e6                # maximum number of iterations to train the model

EDGE_THRESHOLD: 0.5           # edge detection threshold
L1_LOSS_WEIGHT: 1             # l1 loss weight
FM_LOSS_WEIGHT: 10            # feature-matching loss weight
STYLE_LOSS_WEIGHT: 250        # style loss weight
CONTENT_LOSS_WEIGHT: 0.1      # perceptual loss weight
INPAINT_ADV_LOSS_WEIGHT: 0.1  # adversarial loss weight

GAN_LOSS: nsgan               # nsgan | lsgan | hinge
GAN_POOL_SIZE: 0              # fake images pool size

SAVE_INTERVAL: 1000           # how many iterations to wait before saving model (0: never)
SAMPLE_INTERVAL: 1000         # how many iterations to wait before sampling (0: never)
SAMPLE_SIZE: 12               # number of images to sample
EVAL_INTERVAL: 0              # how many iterations to wait before model evaluation (0: never)
LOG_INTERVAL: 10              # how many iterations to wait before logging training status (0: never)

Currently default paths are ```/content/image.png``` and ```/content/mask.png```. Currently it's not supported that you change paths.

In [ ]:
#@title Image and mask need to be dividable by 4, this code does fix wrong images 
import cv2
import numpy
path_inpainting = '/content/image.png' #@param {type:"string"}
path_mask = '/content/mask.png' #@param {type:"string"}
image=cv2.imread(path_mask)
image_size0 = numpy.floor(image.shape[0]/4)
image_size1 = numpy.floor(image.shape[1]/4)
image=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
ret,image=cv2.threshold(image,254,255,cv2.THRESH_BINARY)
image = cv2.resize(image, (int(image_size1*4), int(image_size0*4)), cv2.INTER_NEAREST)
cv2.imwrite(path_mask, image)

image=cv2.imread(path_inpainting)
image = cv2.resize(image, (int(image_size1*4), int(image_size0*4)), cv2.INTER_NEAREST)
cv2.imwrite(path_inpainting, image)

In [ ]:
#@title print shape
import cv2
image = cv2.imread(path_inpainting)
print(image.shape)
image = cv2.imread(path_mask)
print(image.shape)

In [ ]:
#@title Test Inpainting (result will be ```image.png```, the same filename you used as input)
%cd /content/Colab-edge-connect
!python test.py \
  --model 3 \
  --checkpoints /content/Colab-edge-connect/checkpoint_places \
  --input /content/image.png \
  --mask /content/mask.png \
  --output /content/

# Training

Interesting stuff:
- New pytorch versions won't work with the original code. This fork fixes it.
- The ```.tflist``` simply lists filepaths for images.
- It supports blocks as inpainting method, but random/custom masks need to be manually downloaded and input with a ```.tflist``` as well. Two example datasets are linked in the original github.
- [Model 4 is not recommended](https://github.com/knazeri/edge-connect/issues/144). You should probably use model 3.
- [Resuming and using a model as pretrained is being done by simply starting training while the models are in the specified checkpoint path.](https://github.com/knazeri/edge-connect/issues/54). Just make sure to use the default names. Further info in my fork README.
- Do not use Style loss + AMP. Will result in Nan errors.

In [ ]:
#@title Create empty folders
!mkdir /content/training-checkpoints/
!mkdir /content/train/
!mkdir /content/val/
!mkdir /content/mask_train/
!mkdir /content/mask_val/

Input all your data..

In [ ]:
#@title modify paths inside prepare.py to create file list
%%writefile /content/Colab-edge-connect/prepare.py
#!/usr/bin/python

import argparse
import os
from random import shuffle

parser = argparse.ArgumentParser()
parser.add_argument('--is_shuffled', default='1', type=int,
                    help='Needed to shuffle')

if __name__ == "__main__":

    args = parser.parse_args()

    train_filename = 'train.flist'
    validation_filename = 'val.flist'

    train_path = '/home/path/'
    val_path = '/home/path/'

    training_file_names = []
    validation_file_names = []

    training_folder = os.listdir(train_path)

    for training_item in training_folder:
        training_item = train_path + "/" + training_item
        training_file_names.append(training_item)

    validation_folder = os.listdir(val_path)

    for validation_item in validation_folder:
        validation_item = val_path + "/" + validation_item
        validation_file_names.append(validation_item)

    # print all file paths
    for i in training_file_names:
        print(i)
    for i in validation_file_names:
        print(i)

    # This would print all the files and directories

    # shuffle file names if set
    if args.is_shuffled == 1:
        shuffle(training_file_names)
        shuffle(validation_file_names)

    # make output file if not existed
    if not os.path.exists(train_filename):
        os.mknod(train_filename)

    if not os.path.exists(validation_filename):
        os.mknod(validation_filename)

    # write to file
    fo = open(train_filename, "w")
    fo.write("\n".join(training_file_names))
    fo.close()

    fo = open(validation_filename, "w")
    fo.write("\n".join(validation_file_names))
    fo.close()

    # print process
    print("Written file is: ", train_filename, ", is_shuffle: ", args.is_shuffled)


In [ ]:
#@title create file list
%cd /content/Colab-edge-connect
!python prepare.py

In [ ]:
#@title Training config
%%writefile /content/training-checkpoints/config.yml
MODE: 1             # 1: train, 2: test, 3: eval
MODEL: 3            # 1: edge model, 2: inpaint model, 3: edge-inpaint model, 4: joint model
MASK: 5             # 1: random block, 2: half, 3: external, 4: (external, random block), 5: (external, random block, half)
EDGE: 1             # 1: canny, 2: external
NMS: 1              # 0: no non-max-suppression, 1: applies non-max-suppression on the external edges by multiplying by Canny
SEED: 10            # random seed
GPU: [0]            # list of gpu ids
DEBUG: 0            # turns on debugging mode
VERBOSE: 0          # turns on verbose mode in the output console

# input
# edge-connect default: 'input_size', 'center_crop'
RESIZE_MODE: 'random_resize'     # 'input_size', 'random_resize', 'input_size_and_random_downscale' # keeps aspect ratio
CROP_MODE: 'random_crop'      # 'random_crop', 'center_crop' # center_crop assumes that the smaller axis is the INPUT_SIZE. 
FLIP_MODE: ['horizontal_flip', 'vertical_flip'] # ['horizontal_flip', 'vertical_flip']
# ratio
INPUT_SIZE_AND_RANDOM_DOWNSCALE_RATIO: 0.5
HORIZONTAL_FLIP_RATIO: 0.5
VERTICAL_FLIP_RATIO: 0.5

MOSAIC_TEST: 0                # Currently experimental. Inputs randomly resized image, which is overlayed with mask to the generator. Preview images during training won't show the real inputs. Currently 256x256 hardcoded.
USE_AMP: 1                    # Mixed precision training. Currently experimental. Will show a lot of Nan/Inf errors, but it seems to train fine.

BATCH_SIZE: 1                 # input batch size for training
INPUT_SIZE: 256               # input image size for training 0 for original size

# Original Default: ['Adversarial', 'Perceptual', 'Style', 'DEFAULT_L1', 'DEFAULT_GAN']
DISCRIMINATOR: 'pixel'       # default, pixel, patch
DISCRIMINATOR_CALC: 'MSELoss'  # None, BCEWithLogitsLoss, MSELoss
GENERATOR_CALC: 'MSELoss'      # None, BCEWithLogitsLoss, MSELoss

# Options: ['Adversarial', 'Perceptual', 'Style', 'DEFAULT_L1', 'NEW_L1', 'DEFAULT_GAN', 'NEW_GAN', 'HFEN', 'TV', 'ElasticLoss', 'RelativeL1', 'L1CosineSim', 'ClipL1', 'FFT', 'OF', 'GP', 'CP', 'Contextual']
# Warning: Currently, AMP + Style will result in Nan errors
GENERATOR_LOSS: ['Adversarial', 'Perceptual', 'DEFAULT_L1', 'NEW_L1', 'DEFAULT_GAN', 'HFEN', 'TV', 'ElasticLoss', 'RelativeL1', 'L1CosineSim', 'ClipL1', 'FFT', 'OF', 'GP', 'CP', 'Contextual'] 

# default loss
INPAINT_ADV_LOSS_WEIGHT: 0.1  # adversarial loss weight # maybe depricated
CONTENT_LOSS_WEIGHT: 0.1      # perceptual loss weight
STYLE_LOSS_WEIGHT: 250        # style loss weight
L1_LOSS_WEIGHT: 1             # l1 loss weight

DEFAULT_GAN_LOSS: nsgan       # nsgan | lsgan | hinge
GAN_POOL_SIZE: 0              # fake images pool size

# new loss functions (values not tested)
DISCRIMINATOR_FAKE_LOSS_WEIGHT: 0.1
DISCRIMINATOR_REAL_LOSS_WEIGHT: 0.1
GENERATOR_CALC_WEIGHT: 0.1

NEW_GAN_WEIGHT: 5e-3
NEW_GAN_LOSS: 'hinge'         #vanilla, lsgan, srpgan, nsgan, BCE, hinge, wgan-gp (Only hinge seems not to crash, added GAN does not seem to work properly. This option is not really recommended.)

L1_WEIGHT: 0.01               # using new L1
HFEN_WEIGHT: 0.1              # high frequency error norm (HFEN) weight
TV_WEIGHT: 0.000001           # total variation loss weight
ElasticLoss_WEIGHT: 0.01
RelativeL1_WEIGHT: 0.01 
L1CosineSim_WEIGHT: 0.01 
ClipL1_WEIGHT: 0.01 
FFT_WEIGHT: 0.01 
OF_WEIGHT: 0.01               # Overflow loss weight
GP_WEIGHT: 0.01               # Gradient Profile (GP) loss weight
CP_WEIGHT: 0.01               # Color Profile (CP) loss weight
Contextual_WEIGHT: 10.01 
HFEN_TYPE: 'Charbonnier' #L1, MSE, Charbonnier, Elastic, Relative, L1CosineSim


TRAIN_FLIST: /content/train/train.tflist
VAL_FLIST: /content/val/val.tflist
TEST_FLIST: /content/val/val.tflist

TRAIN_EDGE_FLIST: NULL
VAL_EDGE_FLIST: NULL
TEST_EDGE_FLIST: NULL

TRAIN_MASK_FLIST: /content/mask_train/mask_train.tflist
VAL_MASK_FLIST: /content/mask_val/mask_val.tflist
TEST_MASK_FLIST: /content/mask_val/mask_val.tflist

LR: 0.0001                    # learning rate
D2G_LR: 0.1                   # discriminator/generator learning rate ratio
BETA1: 0.0                    # adam optimizer beta1
BETA2: 0.9                    # adam optimizer beta2
SIGMA: 2                      # standard deviation of the Gaussian filter used in Canny edge detector (0: random, -1: no edge)
MAX_ITERS: 1000000            # maximum number of iterations to train the model

EDGE_THRESHOLD: 0.5           # edge detection threshold
FM_LOSS_WEIGHT: 10            # feature-matching loss weight (used in EdgeModel)


# saving
SAVE_INTERVAL: 5000           # how many iterations to wait before saving model (0: never)
SAMPLE_INTERVAL: 500          # how many iterations to wait before sampling (0: never)
SAMPLE_SIZE: 1                # number of images to sample
EVAL_INTERVAL: 0              # how many iterations to wait before model evaluation (0: never)
LOG_INTERVAL: 1              # how many iterations to wait before logging training status (0: never)

In [ ]:
# Train model
%cd /content/Colab-edge-connect
!python train.py --model 3 --checkpoints /content/training-checkpoints/

- This will overwrite your original ```image.png```
- Make sure the dimension is dividable by 4.
- The default filenames for models are ```InpaintingModel_dis.pth``` and ```InpaintingModel_gen.pth```. Other names will be ignored.
- You may or may not need to flip the image. The needed code is below.
- ```image.png``` will be overwritten.

In [ ]:
#@title Image and mask need to be dividable by 4, this code does fix wrong images 
import cv2
import numpy
path_inpainting = '/content/image.png' #@param {type:"string"}
path_mask = '/content/mask.png' #@param {type:"string"}
image=cv2.imread(path_mask)
image_size0 = numpy.floor(image.shape[0]/4)
image_size1 = numpy.floor(image.shape[1]/4)
image=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
ret,image=cv2.threshold(image,254,255,cv2.THRESH_BINARY)
image = cv2.resize(image, (int(image_size1*4), int(image_size0*4)), cv2.INTER_NEAREST)
cv2.imwrite(path_mask, image)

image=cv2.imread(path_inpainting)
image = cv2.resize(image, (int(image_size1*4), int(image_size0*4)), cv2.INTER_NEAREST)
cv2.imwrite(path_inpainting, image)

In [ ]:
#@title Flip image
import cv2
filename = '/content/image.png' #@param {type:"string"}
image = cv2.imread(filename)
image = cv2.flip(image, 1)
cv2.imwrite(filename, image)

In [ ]:
#@title Test model
%cd /content/Colab-edge-connect
checkpoints = '/content/training-checkpoints/' #@param {type:"string"}
input = '/content/image.png' #@param {type:"string"}
mask = '/content/mask.png' #@param {type:"string"}
output = '/content/' #@param {type:"string"}
!python test.py \
  --model 3 \
  --checkpoints {checkpoints} \
  --input {input} \
  --mask {mask} \
  --output {output}

-----------------------------------------------------
Splitted files (Depricated)

In [ ]:
!mkdir /content/Colab-edge-connect/src/vic/

In [ ]:
#@title vic/loss.py
%%writefile /content/Colab-edge-connect/src/vic/loss.py
"""
BasicSR/codes/models/modules/loss.py (8-Nov-20)
https://github.com/victorca25/BasicSR/blob/dev2/codes/models/modules/loss.py
"""


#TODO: change this file to loss_fns.py?

import torch
import torch.nn as nn
import math
import numbers
import torch.nn.functional as F
import numpy as np

#import pdb

#from models.modules.architectures.perceptual import VGG_Model
from .perceptual import VGG_Model

#from models.modules.architectures.video import optical_flow_warp

from .filters import *
from .colors import *
from .common import norm, denorm


class CharbonnierLoss(nn.Module):
    """Charbonnier Loss (L1)"""
    def __init__(self, eps=1e-6):
        super(CharbonnierLoss, self).__init__()
        self.eps = eps

    def forward(self, x, y):
        b, c, h, w = y.size()
        loss = torch.sum(torch.sqrt((x - y).pow(2) + self.eps**2))
        return loss/(c*b*h*w)
    

# Define GAN loss: [vanilla | lsgan | wgan-gp | srpgan/nsgan | hinge]
# https://tuatini.me/creating-and-shipping-deep-learning-models-into-production/
class GANLoss(nn.Module):
    r"""
    Adversarial loss
    https://arxiv.org/abs/1711.10337
    """
    def __init__(self, gan_type, real_label_val=1.0, fake_label_val=0.0):
        super(GANLoss, self).__init__()
        self.gan_type = gan_type.lower()
        self.real_label_val = real_label_val
        self.fake_label_val = fake_label_val

        if self.gan_type == 'vanilla':
            self.loss = nn.BCEWithLogitsLoss()
        elif self.gan_type == 'lsgan':
            self.loss = nn.MSELoss()
        elif self.gan_type == 'srpgan' or self.gan_type == 'nsgan':
            self.loss = nn.BCELoss()
        elif self.gan_type == 'hinge':
            self.loss = nn.ReLU()
        elif self.gan_type == 'wgan-gp':

            def wgan_loss(input, target):
                # target is boolean
                return -1 * input.mean() if target else input.mean()

            self.loss = wgan_loss
        else:
            raise NotImplementedError('GAN type [{:s}] is not found'.format(self.gan_type))

    def get_target_label(self, input, target_is_real):
        if self.gan_type == 'wgan-gp':
            return target_is_real
        if target_is_real:
            return torch.empty_like(input).fill_(self.real_label_val) #torch.ones_like(d_sr_out)
        else:
            return torch.empty_like(input).fill_(self.fake_label_val) #torch.zeros_like(d_sr_out)

    def forward(self, input, target_is_real, is_disc = None):
        if self.gan_type == 'hinge': #TODO: test
            if is_disc:
                input = -input if target_is_real else input
                return self.loss(1 + input).mean()
            else:
                return (-input).mean()
        else:
            target_label = self.get_target_label(input, target_is_real)
            loss = self.loss(input, target_label)
            return loss


class GradientPenaltyLoss(nn.Module):
    def __init__(self, device=torch.device('cpu')):
        super(GradientPenaltyLoss, self).__init__()
        self.register_buffer('grad_outputs', torch.Tensor())
        self.grad_outputs = self.grad_outputs.to(device)

    def get_grad_outputs(self, input):
        if self.grad_outputs.size() != input.size():
            self.grad_outputs.resize_(input.size()).fill_(1.0)
        return self.grad_outputs

    def forward(self, interp, interp_crit):
        grad_outputs = self.get_grad_outputs(interp_crit)
        grad_interp = torch.autograd.grad(outputs=interp_crit, inputs=interp, \
            grad_outputs=grad_outputs, create_graph=True, retain_graph=True, only_inputs=True)[0]
        grad_interp = grad_interp.view(grad_interp.size(0), -1)
        grad_interp_norm = grad_interp.norm(2, dim=1)

        loss = ((grad_interp_norm - 1)**2).mean()
        return loss


class HFENLoss(nn.Module): # Edge loss with pre_smooth
    """Calculates high frequency error norm (HFEN) between target and 
     prediction used to quantify the quality of reconstruction of edges 
     and fine features. 
     
     Uses a rotationally symmetric LoG (Laplacian of Gaussian) filter to 
     capture edges. The original filter kernel is of size 15×15 pixels, 
     and has a standard deviation of 1.5 pixels.
     ks = 2 * int(truncate * sigma + 0.5) + 1, so use truncate=4.5
     
     HFEN is computed as the norm of the result obtained by LoG filtering the 
     difference between the reconstructed and reference images.

    [1]: Ravishankar and Bresler: MR Image Reconstruction From Highly
    Undersampled k-Space Data by Dictionary Learning, 2011
        https://ieeexplore.ieee.org/document/5617283
    [2]: Han et al: Image Reconstruction Using Analysis Model Prior, 2016
        https://www.hindawi.com/journals/cmmm/2016/7571934/
    
    Parameters
    ----------
    img1 : torch.Tensor or torch.autograd.Variable
        Predicted image
    img2 : torch.Tensor or torch.autograd.Variable
        Target image
    norm: if true, follows [2], who define a normalized version of HFEN.
        If using RelativeL1 criterion, it's already normalized. 
    """
    def __init__(self, loss_f=None, kernel='log', kernel_size=15, sigma = 2.5, norm = False): #1.4 ~ 1.5
        super(HFENLoss, self).__init__()
        # can use different criteria
        self.criterion = loss_f
        self.norm = norm
        #can use different kernels like DoG instead:
        if kernel == 'dog':
            kernel = get_dog_kernel(kernel_size, sigma)
        else:
            kernel = get_log_kernel(kernel_size, sigma)
        self.filter = load_filter(kernel=kernel, kernel_size=kernel_size)

    def forward(self, img1, img2):
        self.filter.to(img1.device)
        # HFEN loss
        log1 = self.filter(img1)
        log2 = self.filter(img2)
        hfen_loss = self.criterion(log1, log2)
        if self.norm:
            hfen_loss /= img2.norm()
        return hfen_loss


class TVLoss(nn.Module):
    def __init__(self, tv_type='tv', p = 1):
        super(TVLoss, self).__init__()
        assert p in [1, 2]
        self.p = p
        self.tv_type = tv_type

    def forward(self, x):
        img_shape = x.shape
        if len(img_shape) == 3 or len(img_shape) == 4:
            if self.tv_type == 'dtv':
                dy, dx, dp, dn  = get_4dim_image_gradients(x)

                if len(dy.shape) == 3:
                    # Sum for all axis. (None is an alias for all axis.)
                    reduce_axes = None
                    batch_size = 1
                elif len(dy.shape) == 4:
                    # Only sum for the last 3 axis.
                    # This results in a 1-D tensor with the total variation for each image.
                    reduce_axes = (-3, -2, -1)
                    batch_size = x.size()[0]
                #Compute the element-wise magnitude of a vector array
                # Calculates the TV for each image in the batch
                # Calculate the total variation by taking the absolute value of the
                # pixel-differences and summing over the appropriate axis.
                if self.p == 1:
                    loss = (dy.abs().sum(dim=reduce_axes) + dx.abs().sum(dim=reduce_axes) + dp.abs().sum(dim=reduce_axes) + dn.abs().sum(dim=reduce_axes)) # Calculates the TV loss for each image in the batch
                elif self.p == 2:
                    loss = torch.pow(dy,2).sum(dim=reduce_axes) + torch.pow(dx,2).sum(dim=reduce_axes) + torch.pow(dp,2).sum(dim=reduce_axes) + torch.pow(dn,2).sum(dim=reduce_axes)
                # calculate the scalar loss-value for tv loss
                loss = loss.sum()/(2.0*batch_size) # averages the TV loss all the images in the batch (note: the division is not in TF version, only the sum reduction)
                return loss
            else: #'tv'
                dy, dx  = get_image_gradients(x)

                if len(dy.shape) == 3:
                    # Sum for all axis. (None is an alias for all axis.)
                    reduce_axes = None
                    batch_size = 1
                elif len(dy.shape) == 4:
                    # Only sum for the last 3 axis.
                    # This results in a 1-D tensor with the total variation for each image.
                    reduce_axes = (-3, -2, -1)
                    batch_size = x.size()[0]
                #Compute the element-wise magnitude of a vector array
                # Calculates the TV for each image in the batch
                # Calculate the total variation by taking the absolute value of the
                # pixel-differences and summing over the appropriate axis.
                if self.p == 1:
                    loss = dy.abs().sum(dim=reduce_axes) + dx.abs().sum(dim=reduce_axes)
                elif self.p == 2:
                    loss = torch.pow(dy,2).sum(dim=reduce_axes) + torch.pow(dx,2).sum(dim=reduce_axes)
                # calculate the scalar loss-value for tv loss
                loss = loss.sum()/batch_size # averages the TV loss all the images in the batch (note: the division is not in TF version, only the sum reduction)
                return loss
        else:
            raise ValueError("Expected input tensor to be of ndim 3 or 4, but got " + str(len(img_shape)))
    

class GradientLoss(nn.Module):
    def __init__(self, loss_f = None, reduction='mean', gradientdir='2d'): #2d or 4d
        super(GradientLoss, self).__init__()
        self.criterion = loss_f
        self.gradientdir = gradientdir
    
    def forward(self, input, target):
        if self.gradientdir == '4d':
            inputdy, inputdx, inputdp, inputdn = get_4dim_image_gradients(input)
            targetdy, targetdx, targetdp, targetdn = get_4dim_image_gradients(target) 
            return (self.criterion(inputdx, targetdx) + self.criterion(inputdy, targetdy) + \
                    self.criterion(inputdp, targetdp) + self.criterion(inputdn, targetdn))/4
        else: #'2d'
            inputdy, inputdx = get_image_gradients(input)
            targetdy, targetdx = get_image_gradients(target) 
            return (self.criterion(inputdx, targetdx) + self.criterion(inputdy, targetdy))/2


class ElasticLoss(nn.Module):
    def __init__(self, a=0.2, reduction='mean'): #a=0.5 default
        super(ElasticLoss, self).__init__()
        self.alpha = torch.FloatTensor([a, 1 - a]) #.to('cuda:0')
        self.reduction = reduction

    def forward(self, input, target):
        if not isinstance(input, tuple):
            input = (input,)

        for i in range(len(input)):
            l2 = F.mse_loss(input[i].squeeze(), target.squeeze(), reduction=self.reduction).mul(self.alpha[0])
            l1 = F.l1_loss(input[i].squeeze(), target.squeeze(), reduction=self.reduction).mul(self.alpha[1])
            loss = l1 + l2

        return loss


#TODO: change to RelativeNorm and set criterion as an input argument, could be any basic criterion
class RelativeL1(nn.Module):
    '''
    Comparing to the regular L1, introducing the division by |c|+epsilon 
    better models the human vision system’s sensitivity to variations
    in the dark areas. (where epsilon = 0.01, to prevent values of 0 in the
    denominator)
    '''
    def __init__(self, eps=.01, reduction='mean'):
        super().__init__()
        self.criterion = torch.nn.L1Loss(reduction=reduction)
        self.eps = eps

    def forward(self, input, target):
        base = target + self.eps
        return self.criterion(input/base, target/base)


class L1CosineSim(nn.Module):
    '''
    https://github.com/dmarnerides/hdr-expandnet/blob/master/train.py
    Can be used to replace L1 pixel loss, but includes a cosine similarity term 
    to ensure color correctness of the RGB vectors of each pixel.
    lambda is a constant factor that adjusts the contribution of the cosine similarity term
    It provides improved color stability, especially for low luminance values, which
    are frequent in HDR images, since slight variations in any of the RGB components of these 
    low values do not contribute much totheL1loss, but they may however cause noticeable 
    color shifts. More in the paper: https://arxiv.org/pdf/1803.02266.pdf
    '''
    def __init__(self, loss_lambda=5, reduction='mean'):
        super(L1CosineSim, self).__init__()
        self.similarity = torch.nn.CosineSimilarity(dim=1, eps=1e-20)
        self.l1_loss = nn.L1Loss(reduction=reduction)
        self.loss_lambda = loss_lambda

    def forward(self, x, y):
        cosine_term = (1 - self.similarity(x, y)).mean()
        return self.l1_loss(x, y) + self.loss_lambda * cosine_term


class ClipL1(nn.Module):
    '''
    Clip L1 loss
    From: https://github.com/HolmesShuan/AIM2020-Real-Super-Resolution/
    ClipL1 Loss combines Clip function and L1 loss. self.clip_min sets the 
    gradients of well-trained pixels to zeros and clip_max works as a noise filter.
    data range [0, 255]: (clip_min=0.0, clip_max=10.0), 
    for [0,1] set clip_min to 1/255=0.003921.
    '''
    def __init__(self, clip_min=0.0, clip_max=10.0):
        super(ClipL1, self).__init__()
        self.clip_max = clip_max
        self.clip_min = clip_min

    def forward(self, sr, hr):
        loss = torch.mean(torch.clamp(torch.abs(sr-hr), self.clip_min, self.clip_max))
        return loss


class MaskedL1Loss(nn.Module):
    r"""Masked L1 loss constructor."""
    def __init__(self):
        super(MaskedL1Loss, self, normalize_over_valid=False).__init__()
        self.criterion = nn.L1Loss()
        self.normalize_over_valid = normalize_over_valid

    def forward(self, input, target, mask):
        r"""Masked L1 loss computation.
        Args:
            input (tensor): Input tensor.
            target (tensor): Target tensor.
            mask (tensor): Mask to be applied to the output loss.
        Returns:
            (tensor): Loss value.
        """
        mask = mask.expand_as(input)
        loss = self.criterion(input * mask, target * mask)
        if self.normalize_over_valid:
            # The loss has been averaged over all pixels.
            # Only average over regions which are valid.
            loss = loss * torch.numel(mask) / (torch.sum(mask) + 1e-6)
        return loss


class MultiscalePixelLoss(nn.Module):
    def __init__(self, loss_f = torch.nn.L1Loss(), scale = 5):
        super(MultiscalePixelLoss, self).__init__()
        self.criterion = loss_f
        self.downsample = nn.AvgPool2d(2, stride=2, count_include_pad=False)
        self.weights = [1, 0.5, 0.25, 0.125, 0.125]
        self.weights = self.weights[:scale]

    def forward(self, input, target, mask=None):
        loss = 0
        if mask is not None:
            mask = mask.expand(-1, input.size()[1], -1, -1)
        for i in range(len(self.weights)):
            if mask is not None:
                loss += self.weights[i] * self.criterion(input * mask, target * mask)
            else:
                loss += self.weights[i] * self.criterion(input, target)
            if i != len(self.weights) - 1:
                input = self.downsample(input)
                target = self.downsample(target)
                if mask is not None:
                    mask = self.downsample(mask)
        return loss


# Frequency loss 
# https://github.com/lj1995-computer-vision/Trident-Dehazing-Network/blob/master/loss/fft.py
class FFTloss(torch.nn.Module):
    def __init__(self, loss_f = torch.nn.L1Loss, reduction='mean'):
        super(FFTloss, self).__init__()
        self.criterion = loss_f(reduction=reduction)

    def forward(self, img1, img2):
        zeros=torch.zeros(img1.size()).to(img1.device)
        return self.criterion(torch.fft(torch.stack((img1,zeros),-1),2),torch.fft(torch.stack((img2,zeros),-1),2))


class OFLoss(torch.nn.Module):
    '''
    Overflow loss
    Only use if the image range is in [0,1]. (This solves the SPL brightness problem
    and can be useful in other cases as well)
    https://github.com/lj1995-computer-vision/Trident-Dehazing-Network/blob/master/loss/brelu.py
    '''
    def __init__(self):
        super(OFLoss, self).__init__()

    def forward(self, img1):
        img_clamp = img1.clamp(0,1)
        b,c,h,w = img1.shape
        return torch.log((img1 - img_clamp).abs() + 1).sum()/b/c/h/w

"""
class OFR_loss(torch.nn.Module):
    '''
    Optical flow reconstruction loss (for video)
    https://github.com/LongguangWang/SOF-VSR/blob/master/TIP/data_utils.py
    '''
    def __init__(self, reg_weight=0.1):
        super(OFR_loss, self).__init__()
        self.regularization = L1_regularization()
        self.reg_weight = reg_weight #lambda3

    def forward(self, x0, x1, optical_flow):
        warped = optical_flow_warp(x0, optical_flow)
        loss = torch.mean(torch.abs(x1 - warped)) + self.reg_weight * self.regularization(optical_flow)
        return loss
"""
class L1_regularization(torch.nn.Module):
    # TODO: This is TVLoss/regularization, modify to reuse existing loss. Used by OFR_loss()
    def __init__(self):
        super(L1_regularization, self).__init__()

    def forward(self, image):
        b, _, h, w = image.size()
        reg_x_1 = image[:, :, 0:h-1, 0:w-1] - image[:, :, 1:, 0:w-1]
        reg_y_1 = image[:, :, 0:h-1, 0:w-1] - image[:, :, 0:h-1, 1:]
        reg_L1 = torch.abs(reg_x_1) + torch.abs(reg_y_1)
        return torch.sum(reg_L1) / (b*(h-1)*(w-1))



#TODO: testing
# Color loss 
class ColorLoss(torch.nn.Module):
    def __init__(self, loss_f = torch.nn.L1Loss, reduction='mean', ds_f=None):
        super(ColorLoss, self).__init__()
        self.ds_f = ds_f
        self.criterion = loss_f

    def forward(self, input, target):
        input_uv = rgb_to_yuv(self.ds_f(input), consts='uv')
        target_uv = rgb_to_yuv(self.ds_f(target), consts='uv')
        return self.criterion(input_uv, target_uv)

#TODO: testing
# Averaging Downscale loss 
class AverageLoss(torch.nn.Module):
    def __init__(self, loss_f = torch.nn.L1Loss, reduction='mean', ds_f=None):
        super(AverageLoss, self).__init__()
        self.ds_f = ds_f
        self.criterion = loss_f

    def forward(self, input, target):
        input_uv = rgb_to_yuv(self.ds_f(input), consts='uv')
        target_uv = rgb_to_yuv(self.ds_f(target), consts='uv')
        return self.criterion(input_uv, target_uv)




########################
# Spatial Profile Loss
########################

class GPLoss(nn.Module):
    '''
    https://github.com/ssarfraz/SPL/blob/master/SPL_Loss/
    Gradient Profile (GP) loss
    The image gradients in each channel can easily be computed 
    by simple 1-pixel shifted image differences from itself. 
    '''
    def __init__(self, trace=False, spl_denorm=False):
        super(GPLoss, self).__init__()
        self.spl_denorm = spl_denorm
        if trace == True: # Alternate behavior: use the complete calculation with SPL_ComputeWithTrace()
            self.trace = SPL_ComputeWithTrace()
        else: # Default behavior: use the more efficient SPLoss()
            self.trace = SPLoss()

    def __call__(self, input, reference):
        ## Use "spl_denorm" when reading a [-1,1] input, but you want to compute the loss over a [0,1] range
        # Note: only rgb_to_yuv() requires image in the [0,1], so this denorm is optional, depending on the net
        if self.spl_denorm == True:
            input = denorm(input)
            reference = denorm(reference)
        input_h, input_v = get_image_gradients(input)
        ref_h, ref_v = get_image_gradients(reference)

        trace_v = self.trace(input_v,ref_v)
        trace_h = self.trace(input_h,ref_h)
        return trace_v + trace_h

class CPLoss(nn.Module):
    '''
    Color Profile (CP) loss
    '''
    def __init__(self, rgb=True, yuv=True, yuvgrad=True, trace=False, spl_denorm=False, yuv_denorm=False):
        super(CPLoss, self).__init__()
        self.rgb = rgb
        self.yuv = yuv
        self.yuvgrad = yuvgrad
        self.spl_denorm = spl_denorm
        self.yuv_denorm = yuv_denorm
        
        if trace == True: # Alternate behavior: use the complete calculation with SPL_ComputeWithTrace()
            self.trace = SPL_ComputeWithTrace()
            self.trace_YUV = SPL_ComputeWithTrace()
        else: # Default behavior: use the more efficient SPLoss()
            self.trace = SPLoss()
            self.trace_YUV = SPLoss()

    def __call__(self, input, reference):
        ## Use "spl_denorm" when reading a [-1,1] input, but you want to compute the loss over a [0,1] range
        # self.spl_denorm=False when your inputs and outputs are in [0,1] range already
        # Note: only rgb_to_yuv() requires image in the [0,1], so this denorm is optional, depending on the net
        if self.spl_denorm:
            input = denorm(input)
            reference = denorm(reference)
        total_loss= 0
        if self.rgb:
            total_loss += self.trace(input,reference)
        if self.yuv:
            # rgb_to_yuv() needs images in [0,1] range to work
            if not self.spl_denorm and self.yuv_denorm:
                input = denorm(input)
                reference = denorm(reference)
            input_yuv = rgb_to_yuv(input)
            reference_yuv = rgb_to_yuv(reference)
            total_loss += self.trace(input_yuv,reference_yuv)
        if self.yuvgrad:
            input_h, input_v = get_image_gradients(input_yuv)
            ref_h, ref_v = get_image_gradients(reference_yuv)

            total_loss +=  self.trace(input_v,ref_v)
            total_loss +=  self.trace(input_h,ref_h)

        return total_loss

## Spatial Profile Loss (SPL) with trace
class SPL_ComputeWithTrace(nn.Module):
    """
    Spatial Profile Loss (SPL)
    Both loss versions equate to the cosine similarity of rows/columns. 
    'SPL_ComputeWithTrace()' uses the trace (sum over the diagonal) of matrix multiplication 
    of L2-normalized input/target rows/columns.
    Slow implementation of the trace loss using the same formula as stated in the paper. 
    In principle, we compute the loss between a source and target image by considering such 
    pattern differences along the image x and y-directions. Considering a row or a column 
    spatial profile of an image as a vector, we can compute the similarity between them in 
    this induced vector space. Formally, this similarity is measured over each image channel ’c’.
    The first term computes similarity among row profiles and the second among column profiles 
    of an image pair (x, y) of size H ×W. These image pixels profiles are L2-normalized to 
    have a normalized cosine similarity loss.
    """
    def __init__(self,weight = [1.,1.,1.]): # The variable 'weight' was originally intended to weigh color channels differently. In our experiments, we found that an equal weight between all channels gives the best results. As such, this variable is a leftover from that time and can be removed.
        super(SPL_ComputeWithTrace, self).__init__()
        self.weight = weight

    def __call__(self, input, reference):
        a = 0
        b = 0
        for i in range(input.shape[0]):
            for j in range(input.shape[1]):
                a += torch.trace(torch.matmul(F.normalize(input[i,j,:,:],p=2,dim=1),torch.t(F.normalize(reference[i,j,:,:],p=2,dim=1))))/input.shape[2]*self.weight[j]
                b += torch.trace(torch.matmul(torch.t(F.normalize(input[i,j,:,:],p=2,dim=0)),F.normalize(reference[i,j,:,:],p=2,dim=0)))/input.shape[3]*self.weight[j]
        a = -torch.sum(a)/input.shape[0]
        b = -torch.sum(b)/input.shape[0]
        return a+b

## Spatial Profile Loss (SPL) without trace, prefered
class SPLoss(nn.Module):
    ''' 
    Spatial Profile Loss (SPL)
    'SPLoss()' L2-normalizes the rows/columns, performs piece-wise multiplication 
    of the two tensors and then sums along the corresponding axes. This variant 
    needs less operations since it can be performed batchwise.
    Note: SPLoss() makes image results too bright, when using images in the [0,1] 
    range and no activation as output of the Generator.
    SPL_ComputeWithTrace() does not have this problem, but results are very blurry. 
    Adding the Overflow Loss fixes this problem.
    '''
    def __init__(self):
        super(SPLoss, self).__init__()
        #self.weight = weight

    def __call__(self, input, reference):
        a = torch.sum(torch.sum(F.normalize(input, p=2, dim=2) * F.normalize(reference, p=2, dim=2),dim=2, keepdim=True))
        b = torch.sum(torch.sum(F.normalize(input, p=2, dim=3) * F.normalize(reference, p=2, dim=3),dim=3, keepdim=True))
        return -(a + b) / (input.size(2) * input.size(0))





########################
# Contextual Loss
########################

DIS_TYPES = ['cosine', 'l1', 'l2']

class Contextual_Loss(nn.Module):
    '''
    Contextual loss for unaligned images (https://arxiv.org/abs/1803.02077)

    https://github.com/roimehrez/contextualLoss
    https://github.com/S-aiueo32/contextual_loss_pytorch
    https://github.com/z-bingo/Contextual-Loss-PyTorch

    layers_weights: is a dict, e.g., {'conv_1_1': 1.0, 'conv_3_2': 1.0}
    crop_quarter: boolean
    '''
    def __init__(self, layers_weights, crop_quarter=False, max_1d_size=100, 
            distance_type: str = 'cosine', b=1.0, band_width=0.5, 
            use_vgg: bool = True, net: str = 'vgg19', calc_type: str =  'regular'):
        super(Contextual_Loss, self).__init__()

        assert band_width > 0, 'band_width parameter must be positive.'
        assert distance_type in DIS_TYPES,\
            f'select a distance type from {DIS_TYPES}.'

        listen_list = []
        self.layers_weights = {}
        try:
            listen_list = layers_weights.keys()
            self.layers_weights = layers_weights
        except:
            pass
        
        self.crop_quarter = crop_quarter
        self.distanceType = distance_type
        self.max_1d_size = max_1d_size
        self.b = b
        self.band_width = band_width #self.h = h, #sigma
        
        if use_vgg:
            self.vgg_model = VGG_Model(listen_list=listen_list, net=net)

        if calc_type == 'bilateral':
            self.calculate_loss = self.bilateral_CX_Loss
        elif calc_type == 'symetric':
            self.calculate_loss = self.symetric_CX_Loss
        else: #if calc_type == 'regular':
            self.calculate_loss = self.calculate_CX_Loss

    def forward(self, images, gt):
        device = images.device
        
        if hasattr(self, 'vgg_model'):
            assert images.shape[1] == 3 and gt.shape[1] == 3,\
                'VGG model takes 3 channel images.'
            
            loss = 0
            vgg_images = self.vgg_model(images)
            vgg_images = {k: v.clone().to(device) for k, v in vgg_images.items()}
            vgg_gt = self.vgg_model(gt)
            vgg_gt = {k: v.to(device) for k, v in vgg_gt.items()}

            for key in self.layers_weights.keys():
                if self.crop_quarter:
                    vgg_images[key] = self._crop_quarters(vgg_images[key])
                    vgg_gt[key] = self._crop_quarters(vgg_gt[key])

                N, C, H, W = vgg_images[key].size()
                if H*W > self.max_1d_size**2:
                    vgg_images[key] = self._random_pooling(vgg_images[key], output_1d_size=self.max_1d_size)
                    vgg_gt[key] = self._random_pooling(vgg_gt[key], output_1d_size=self.max_1d_size)

                loss_t = self.calculate_loss(vgg_images[key], vgg_gt[key])
                loss += loss_t * self.layers_weights[key]
                # del vgg_images[key], vgg_gt[key]
        #TODO: without VGG it runs, but results are not looking right
        else:
            if self.crop_quarter:
                images = self._crop_quarters(images)
                gt = self._crop_quarters(gt)

            N, C, H, W = images.size()
            if H*W > self.max_1d_size**2:
                images = self._random_pooling(images, output_1d_size=self.max_1d_size)
                gt = self._random_pooling(gt, output_1d_size=self.max_1d_size)

            loss = self.calculate_loss(images, gt)
        return loss

    @staticmethod
    def _random_sampling(tensor, n, indices):
        N, C, H, W = tensor.size()
        S = H * W
        tensor = tensor.view(N, C, S)
        device=tensor.device
        if indices is None:
            indices = torch.randperm(S)[:n].contiguous().type_as(tensor).long()
            indices = indices.clamp(indices.min(), tensor.shape[-1]-1) #max = indices.max()-1
            indices = indices.view(1, 1, -1).expand(N, C, -1)
        indices = indices.to(device)

        res = torch.gather(tensor, index=indices, dim=-1)
        return res, indices

    @staticmethod
    def _random_pooling(feats, output_1d_size=100):
        single_input = type(feats) is torch.Tensor

        if single_input:
            feats = [feats]

        N, C, H, W = feats[0].size()
        feats_sample, indices = Contextual_Loss._random_sampling(feats[0], output_1d_size**2, None)
        res = [feats_sample]

        for i in range(1, len(feats)):
            feats_sample, _ = Contextual_Loss._random_sampling(feats[i], -1, indices)
            res.append(feats_sample)

        res = [feats_sample.view(N, C, output_1d_size, output_1d_size) for feats_sample in res]

        if single_input:
            return res[0]
        return res

    @staticmethod
    def _crop_quarters(feature_tensor):
        N, fC, fH, fW = feature_tensor.size()
        quarters_list = []
        quarters_list.append(feature_tensor[..., 0:round(fH / 2), 0:round(fW / 2)])
        quarters_list.append(feature_tensor[..., 0:round(fH / 2), round(fW / 2):])
        quarters_list.append(feature_tensor[..., round(fH / 2):, 0:round(fW / 2)])
        quarters_list.append(feature_tensor[..., round(fH / 2):, round(fW / 2):])

        feature_tensor = torch.cat(quarters_list, dim=0)
        return feature_tensor

    @staticmethod
    def _create_using_L2(I_features, T_features):
        """
        Calculating the distance between each feature of I and T
        :param I_features:
        :param T_features:
        :return: raw_distance: [N, C, H, W, H*W], each element of which is the distance between I and T at each position
        """
        assert I_features.size() == T_features.size()
        N, C, H, W = I_features.size()

        Ivecs = I_features.view(N, C, -1)
        Tvecs = T_features.view(N, C, -1)
        #
        square_I = torch.sum(Ivecs*Ivecs, dim=1, keepdim=False)
        square_T = torch.sum(Tvecs*Tvecs, dim=1, keepdim=False)
        # raw_distance
        raw_distance = []
        for i in range(N):
            Ivec, Tvec, s_I, s_T = Ivecs[i, ...], Tvecs[i, ...], square_I[i, ...], square_T[i, ...]
            # matrix multiplication
            AB = Ivec.permute(1, 0) @ Tvec
            dist = s_I.view(-1, 1) + s_T.view(1, -1) - 2*AB
            raw_distance.append(dist.view(1, H, W, H*W))
        raw_distance = torch.cat(raw_distance, dim=0)
        raw_distance = torch.clamp(raw_distance, 0.0)
        return raw_distance

    @staticmethod
    def _create_using_L1(I_features, T_features):
        assert I_features.size() == T_features.size()
        N, C, H, W = I_features.size()

        Ivecs = I_features.view(N, C, -1)
        Tvecs = T_features.view(N, C, -1)

        raw_distance = []
        for i in range(N):
            Ivec, Tvec = Ivecs[i, ...], Tvecs[i, ...]
            dist = torch.sum(
                torch.abs(Ivec.view(C, -1, 1) - Tvec.view(C, 1, -1)), dim=0, keepdim=False
            )
            raw_distance.append(dist.view(1, H, W, H*W))
        raw_distance = torch.cat(raw_distance, dim=0)
        return raw_distance

    @staticmethod
    def _create_using_dotP(I_features, T_features):
        assert I_features.size() == T_features.size()
        # prepare feature before calculating cosine distance
        # mean shifting by channel-wise mean of `y`.
        mean_T = T_features.mean(dim=(0, 2, 3), keepdim=True)        
        I_features = I_features - mean_T
        T_features = T_features - mean_T

        # L2 channelwise normalization
        I_features = F.normalize(I_features, p=2, dim=1)
        T_features = F.normalize(T_features, p=2, dim=1)
        
        N, C, H, W = I_features.size()
        cosine_dist = []
        # work seperatly for each example in dim 1
        for i in range(N):
            # channel-wise vectorization
            T_features_i = T_features[i].view(1, 1, C, H*W).permute(3, 2, 0, 1).contiguous() # 1CHW --> 11CP, with P=H*W
            I_features_i = I_features[i].unsqueeze(0)
            dist = F.conv2d(I_features_i, T_features_i).permute(0, 2, 3, 1).contiguous()
            #cosine_dist.append(dist) # back to 1CHW
            #TODO: temporary hack to workaround AMP bug:
            cosine_dist.append(dist.to(torch.float32)) # back to 1CHW
        cosine_dist = torch.cat(cosine_dist, dim=0)
        cosine_dist = (1 - cosine_dist) / 2
        cosine_dist = cosine_dist.clamp(min=0.0)

        return cosine_dist

    #compute_relative_distance
    @staticmethod
    def _calculate_relative_distance(raw_distance, epsilon=1e-5):
        """
        Normalizing the distances first as Eq. (2) in paper
        :param raw_distance:
        :param epsilon:
        :return:
        """
        div = torch.min(raw_distance, dim=-1, keepdim=True)[0]
        relative_dist = raw_distance / (div + epsilon) # Eq 2
        return relative_dist

    def symetric_CX_Loss(self, I_features, T_features):
        loss = (self.calculate_CX_Loss(T_features, I_features) + self.calculate_CX_Loss(I_features, T_features)) / 2
        return loss #score

    def bilateral_CX_Loss(self, I_features, T_features, weight_sp: float = 0.1):
        def compute_meshgrid(shape):
            N, C, H, W = shape
            rows = torch.arange(0, H, dtype=torch.float32) / (H + 1)
            cols = torch.arange(0, W, dtype=torch.float32) / (W + 1)

            feature_grid = torch.meshgrid(rows, cols)
            feature_grid = torch.stack(feature_grid).unsqueeze(0)
            feature_grid = torch.cat([feature_grid for _ in range(N)], dim=0)

            return feature_grid

        # spatial loss
        grid = compute_meshgrid(I_features.shape).to(T_features.device)
        raw_distance = Contextual_Loss._create_using_L2(grid, grid) # calculate raw distance
        dist_tilde = Contextual_Loss._calculate_relative_distance(raw_distance)
        exp_distance = torch.exp((self.b - dist_tilde) / self.band_width) # Eq(3)
        cx_sp = exp_distance / torch.sum(exp_distance, dim=-1, keepdim=True) # Eq(4)

        # feature loss
        # calculate raw distances
        if self.distanceType == 'l1':
            raw_distance = Contextual_Loss._create_using_L1(I_features, T_features)
        elif self.distanceType == 'l2':
            raw_distance = Contextual_Loss._create_using_L2(I_features, T_features)
        else: # self.distanceType == 'cosine':
            raw_distance = Contextual_Loss._create_using_dotP(I_features, T_features)
        dist_tilde = Contextual_Loss._calculate_relative_distance(raw_distance)
        exp_distance = torch.exp((self.b - dist_tilde) / self.band_width) # Eq(3)
        cx_feat = exp_distance / torch.sum(exp_distance, dim=-1, keepdim=True) # Eq(4)

        # combined loss
        cx_combine = (1. - weight_sp) * cx_feat + weight_sp * cx_sp
        k_max_NC, _ = torch.max(cx_combine, dim=2, keepdim=True)
        cx = k_max_NC.mean(dim=1)
        cx_loss = torch.mean(-torch.log(cx + 1e-5))
        return cx_loss

    def calculate_CX_Loss(self, I_features, T_features):
        device = I_features.device
        T_features = T_features.to(device)

        if torch.sum(torch.isnan(I_features)) == torch.numel(I_features) or torch.sum(torch.isinf(I_features)) == torch.numel(I_features):
            print(I_features)
            raise ValueError('NaN or Inf in I_features')
        if torch.sum(torch.isnan(T_features)) == torch.numel(T_features) or torch.sum(
                torch.isinf(T_features)) == torch.numel(T_features):
            print(T_features)
            raise ValueError('NaN or Inf in T_features')

        # calculate raw distances
        if self.distanceType == 'l1':
            raw_distance = Contextual_Loss._create_using_L1(I_features, T_features)
        elif self.distanceType == 'l2':
            raw_distance = Contextual_Loss._create_using_L2(I_features, T_features)
        else: # self.distanceType == 'cosine':
            raw_distance = Contextual_Loss._create_using_dotP(I_features, T_features)
        if torch.sum(torch.isnan(raw_distance)) == torch.numel(raw_distance) or torch.sum(
                torch.isinf(raw_distance)) == torch.numel(raw_distance):
            print(raw_distance)
            raise ValueError('NaN or Inf in raw_distance')

        # normalizing the distances
        relative_distance = Contextual_Loss._calculate_relative_distance(raw_distance)
        if torch.sum(torch.isnan(relative_distance)) == torch.numel(relative_distance) or torch.sum(
                torch.isinf(relative_distance)) == torch.numel(relative_distance):
            print(relative_distance)
            raise ValueError('NaN or Inf in relative_distance')
        del raw_distance

        #compute_sim()
        # where h>0 is a band-width parameter
        exp_distance = torch.exp((self.b - relative_distance) / self.band_width) # Eq(3)
        if torch.sum(torch.isnan(exp_distance)) == torch.numel(exp_distance) or torch.sum(
                torch.isinf(exp_distance)) == torch.numel(exp_distance):
            print(exp_distance)
            raise ValueError('NaN or Inf in exp_distance')
        del relative_distance
        
        # Similarity
        contextual_sim = exp_distance / torch.sum(exp_distance, dim=-1, keepdim=True) # Eq(4)
        if torch.sum(torch.isnan(contextual_sim)) == torch.numel(contextual_sim) or torch.sum(
                torch.isinf(contextual_sim)) == torch.numel(contextual_sim):
            print(contextual_sim)
            raise ValueError('NaN or Inf in contextual_sim')
        del exp_distance
        
        #contextual_loss()
        max_gt_sim = torch.max(torch.max(contextual_sim, dim=1)[0], dim=1)[0] # Eq(1)
        del contextual_sim
        CS = torch.mean(max_gt_sim, dim=1)
        CX_loss = torch.mean(-torch.log(CS)) # Eq(5)
        if torch.isnan(CX_loss):
            raise ValueError('NaN in computing CX_loss')
        return CX_loss

In [ ]:
#@title vic/common.py
%%writefile /content/Colab-edge-connect/src/vic/common.py
"""
BasicSR/codes/dataops/common.py (8-Nov-20)
https://github.com/victorca25/BasicSR/blob/dev2/codes/dataops/common.py
"""

import os
import math
import pickle
import random
import numpy as np
import torch
import cv2
import logging

import copy
from torchvision.utils import make_grid

#from dataops.colors import *
from .colors import *
#from dataops.debug import tmp_vis, describe_numpy, describe_tensor

####################
# Files & IO
####################

###################### get image path list ######################
IMG_EXTENSIONS = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP', '.dng', '.DNG', '.webp','.npy', '.NPY']

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)


def _get_paths_from_images(path):
    '''get image path list from image folder'''
    assert os.path.isdir(path), '{:s} is not a valid directory'.format(path)
    images = []
    for dirpath, _, fnames in sorted(os.walk(path)):
        for fname in sorted(fnames):
            if is_image_file(fname):
                img_path = os.path.join(dirpath, fname)
                images.append(img_path)
    assert images, '{:s} has no valid image file'.format(path)
    return images


def _get_paths_from_lmdb(dataroot):
    '''get image path list from lmdb'''
    import lmdb
    env = lmdb.open(dataroot, readonly=True, lock=False, readahead=False, meminit=False)
    keys_cache_file = os.path.join(dataroot, '_keys_cache.p')
    logger = logging.getLogger('base')
    if os.path.isfile(keys_cache_file):
        logger.info('Read lmdb keys from cache: {}'.format(keys_cache_file))
        keys = pickle.load(open(keys_cache_file, "rb"))
    else:
        with env.begin(write=False) as txn:
            logger.info('Creating lmdb keys cache: {}'.format(keys_cache_file))
            keys = [key.decode('ascii') for key, _ in txn.cursor()]
        pickle.dump(keys, open(keys_cache_file, 'wb'))
    paths = sorted([key for key in keys if not key.endswith('.meta')])
    return env, paths


def get_image_paths(data_type, dataroot):
    '''get image path list
    support lmdb or image files'''
    env, paths = None, None
    if dataroot is not None:
        if data_type == 'lmdb':
            env, paths = _get_paths_from_lmdb(dataroot)
        elif data_type == 'img':
            paths = sorted(_get_paths_from_images(dataroot))
        else:
            raise NotImplementedError('data_type [{:s}] is not recognized.'.format(data_type))
    return env, paths


###################### read images ######################
def _read_lmdb_img(env, path):
    with env.begin(write=False) as txn:
        buf = txn.get(path.encode('ascii'))
        buf_meta = txn.get((path + '.meta').encode('ascii')).decode('ascii')
    img_flat = np.frombuffer(buf, dtype=np.uint8)
    H, W, C = [int(s) for s in buf_meta.split(',')]
    img = img_flat.reshape(H, W, C)
    return img


def read_img(env, path, out_nc=3, fix_channels=True):
    '''
        Reads image using cv2 (rawpy if dng) or from lmdb by default
        (can also use using PIL instead of cv2)
    Arguments:
        out_nc: Desired number of channels
        fix_channels: changes the images to the desired number of channels
    Output:
        Numpy uint8, HWC, BGR, [0,255] by default 
    '''

    img = None
    if env is None:  # img
        if(path[-3:].lower() == 'dng'): # if image is a DNG
            import rawpy
            with rawpy.imread(path) as raw:
                img = raw.postprocess()
        if(path[-3:].lower() == 'npy'): # if image is a NPY numpy array
            with open(path, 'rb') as f:
                img = np.load(f)
        else: # else, if image can be read by cv2 
            img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        #TODO: add variable detecting if cv2 is not available and try PIL instead
        # elif: # using PIL instead of OpenCV
            # img = Image.open(path).convert('RGB')
        # else: # For other images unrecognized by cv2
            # import matplotlib.pyplot as plt
            # img = (255*plt.imread(path)[:,:,:3]).astype('uint8')
    else:
        img = _read_lmdb_img(env, path)

    # if not img:
    #     raise ValueError(f"Failed to read image: {path}")

    if fix_channels:
        img = fix_img_channels(img, out_nc)

    return img

def fix_img_channels(img, out_nc):
    '''
        fix image channels to the expected number
    '''

    # if image has only 2 dimensions, add "channel" dimension (1)
    if img.ndim == 2:
        #img = img[..., np.newaxis] #alt
        #img = np.expand_dims(img, axis=2)
        img = np.tile(np.expand_dims(img, axis=2), (1, 1, 3))
    # special case: properly remove alpha channel 
    if out_nc == 3 and img.shape[2] == 4: 
        img = bgra2rgb(img)
    # remove all extra channels 
    elif img.shape[2] > out_nc: 
        img = img[:, :, :out_nc] 
    # if alpha is expected, add solid alpha channel
    elif img.shape[2] == 3 and out_nc == 4:
        img = np.dstack((img, np.full(img.shape[:-1], 255, dtype=np.uint8)))
    return img


####################
# image processing
# process on numpy image
####################

def bgra2rgb(img):
    '''
        cv2.cvtColor(img, cv2.COLOR_BGRA2BGR) has an issue removing the alpha channel,
        this gets rid of wrong transparent colors that can harm training
    '''
    if img.shape[2] == 4:
        #b, g, r, a = cv2.split((img*255).astype(np.uint8))
        b, g, r, a = cv2.split((img.astype(np.uint8)))
        b = cv2.bitwise_and(b, b, mask=a)
        g = cv2.bitwise_and(g, g, mask=a)
        r = cv2.bitwise_and(r, r, mask=a)
        #return cv2.merge([b, g, r]).astype(np.float32)/255.
        return cv2.merge([b, g, r])
    return img

def channel_convert(in_c, tar_type, img_list):
    # conversion among BGR, gray and y
    # Note: OpenCV uses inverted channels BGR, instead of RGB.
    #  If images are loaded with something other than OpenCV,
    #  check that the channels are in the correct order and use
    #  the alternative conversion functions.
    #if in_c == 4 and tar_type == 'RGB-A':  # BGRA to BGR, remove alpha channel
        #return [cv2.cvtColor(img, cv2.COLOR_BGRA2BGR) for img in img_list]
        #return [bgra2rgb(img) for img in img_list]
    if in_c == 3 and tar_type == 'gray':  # BGR to gray
        gray_list = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in img_list]
        return [np.expand_dims(img, axis=2) for img in gray_list]
    elif in_c == 3 and tar_type == 'RGB-LAB': #RGB to LAB
        return [cv2.cvtColor(img, cv2.COLOR_BGR2LAB) for img in img_list]
    elif in_c == 3 and tar_type == 'LAB-RGB': #RGB to LAB
        return [cv2.cvtColor(img, cv2.COLOR_LAB2BGR) for img in img_list]
    elif in_c == 3 and tar_type == 'y':  # BGR to y
        y_list = [bgr2ycbcr(img, only_y=True) for img in img_list]
        return [np.expand_dims(img, axis=2) for img in y_list]
    elif in_c == 1 and tar_type == 'RGB':  # gray/y to BGR
        return [cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) for img in img_list]
    else:
        return img_list

def rgb2ycbcr(img, only_y=True):
    '''same as matlab rgb2ycbcr
    only_y: only return Y channel
    Input:
        uint8, [0, 255]
        float, [0, 1]
    '''
    in_img_type = img.dtype
    img_ = img.astype(np.float32)
    if in_img_type != np.uint8:
        img_  *= 255.
    # convert
    if only_y:
        rlt = np.dot(img_ , [65.481, 128.553, 24.966]) / 255.0 + 16.0
    else:
        rlt = np.matmul(img_ , [[65.481, -37.797, 112.0], [128.553, -74.203, -93.786],
                              [24.966, 112.0, -18.214]]) / 255.0 + [16, 128, 128]
    if in_img_type == np.uint8:
        rlt = rlt.round()
    else:
        rlt /= 255.
    return rlt.astype(in_img_type)

def bgr2ycbcr(img, only_y=True, separate=False):
    '''bgr version of matlab rgb2ycbcr
    Python opencv library (cv2) cv2.COLOR_BGR2YCrCb has 
    different parameters with MATLAB color convertion.
    only_y: only return Y channel
    separate: if true, will returng the channels as 
        separate images
    Input:
        uint8, [0, 255]
        float, [0, 1]
    '''
    in_img_type = img.dtype
    img_ = img.astype(np.float32)
    if in_img_type != np.uint8:
        img_  *= 255.
    # convert
    if only_y:
        rlt = np.dot(img_ , [24.966, 128.553, 65.481]) / 255.0 + 16.0
    else:
        rlt = np.matmul(img_ , [[24.966, 112.0, -18.214], [128.553, -74.203, -93.786],
                              [65.481, -37.797, 112.0]]) / 255.0 + [16, 128, 128]
        # to make ycrcb like cv2
        # rlt = rlt[:, :, (0, 2, 1)]
    
    if in_img_type == np.uint8:
        rlt = rlt.round()
    else:
        rlt /= 255.
    
    if separate:
        rlt = rlt.astype(in_img_type)
        # y, cb, cr
        return rlt[:, :, 0], rlt[:, :, 1], rlt[:, :, 2]
    else:
        return rlt.astype(in_img_type)

'''
def ycbcr2rgb_(img, only_y=True):
    """same as matlab ycbcr2rgb
    (Note: this implementation is the original from BasicSR, but 
    appears to be for ycrcb, like cv2)
    Input:
        uint8, [0, 255]
        float, [0, 1]
    """
    in_img_type = img.dtype
    img_ = img.astype(np.float32)
    if in_img_type != np.uint8:
        img_  *= 255.
    
    # to make ycrcb like cv2
    # rlt = rlt[:, :, (0, 2, 1)]

    # convert
    # original (for ycrcb):
    rlt = np.matmul(img_ , [[0.00456621, 0.00456621, 0.00456621], [0, -0.00153632, 0.00791071],
                          [0.00625893, -0.00318811, 0]]) * 255.0 + [-222.921, 135.576, -276.836]

    #alternative conversion:
    # xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    # img_[:, :, [1, 2]] -= 128
    # rlt = img_.dot(xform.T)
    np.putmask(rlt, rlt > 255, 255)
    np.putmask(rlt, rlt < 0, 0)
    
    if in_img_type == np.uint8:
        rlt = rlt.round()
    else:
        rlt /= 255.
    return rlt.astype(in_img_type)
'''

def ycbcr2rgb(img, only_y=True):
    '''
    bgr version of matlab ycbcr2rgb
    Python opencv library (cv2) cv2.COLOR_YCrCb2BGR has 
    different parameters to MATLAB color convertion.

    Input:
        uint8, [0, 255]
        float, [0, 1]
    '''
    in_img_type = img.dtype
    img_ = img.astype(np.float32)
    if in_img_type != np.uint8:
        img_  *= 255.
    
    # to make ycrcb like cv2
    # rlt = rlt[:, :, (0, 2, 1)]

    # convert
    mat = np.array([[24.966, 128.553, 65.481],[112, -74.203, -37.797], [-18.214, -93.786, 112.0]])
    mat = np.linalg.inv(mat.T) * 255
    offset = np.array([[[16, 128, 128]]])

    rlt = np.dot((img_ - offset), mat)
    rlt = np.clip(rlt, 0, 255)
    ## rlt = np.rint(rlt).astype('uint8')
    
    if in_img_type == np.uint8:
        rlt = rlt.round()
    else:
        rlt /= 255.
    return rlt.astype(in_img_type)

'''
#TODO: TMP RGB version, to check (PIL)
def rgb2ycbcr(img_rgb):
    ## the range of img_rgb should be (0, 1)
    img_y = 0.257 * img_rgb[:, :, 0] + 0.504 * img_rgb[:, :, 1] + 0.098 * img_rgb[:, :, 2] + 16 / 255.0
    img_cb = -0.148 * img_rgb[:, :, 0] - 0.291 * img_rgb[:, :, 1] + 0.439 * img_rgb[:, :, 2] + 128 / 255.0
    img_cr = 0.439 * img_rgb[:, :, 0] - 0.368 * img_rgb[:, :, 1] - 0.071 * img_rgb[:, :, 2] + 128 / 255.0
    return img_y, img_cb, img_cr

#TODO: TMP RGB version, to check (PIL)
def ycbcr2rgb(img_ycbcr):
    ## the range of img_ycbcr should be (0, 1)
    img_r = 1.164 * (img_ycbcr[:, :, 0] - 16 / 255.0) + 1.596 * (img_ycbcr[:, :, 2] - 128 / 255.0)
    img_g = 1.164 * (img_ycbcr[:, :, 0] - 16 / 255.0) - 0.392 * (img_ycbcr[:, :, 1] - 128 / 255.0) - 0.813 * (img_ycbcr[:, :, 2] - 128 / 255.0)
    img_b = 1.164 * (img_ycbcr[:, :, 0] - 16 / 255.0) + 2.017 * (img_ycbcr[:, :, 1] - 128 / 255.0)
    img_r = img_r[:, :, np.newaxis]
    img_g = img_g[:, :, np.newaxis]
    img_b = img_b[:, :, np.newaxis]
    img_rgb = np.concatenate((img_r, img_g, img_b), 2)
    return img_rgb
'''

def modcrop(img_in, scale):
    # img_in: Numpy, HWC or HW
    img = np.copy(img_in)
    if img.ndim == 2:
        H, W = img.shape
        H_r, W_r = H % scale, W % scale
        img = img[:H - H_r, :W - W_r]
    elif img.ndim == 3:
        H, W, C = img.shape
        H_r, W_r = H % scale, W % scale
        img = img[:H - H_r, :W - W_r, :]
    else:
        raise ValueError('Wrong img ndim: [{:d}].'.format(img.ndim))
    return img

#TODO: this should probably be elsewhere (augmentations.py)
def augment(img_list, hflip=True, rot=True):
    # horizontal flip OR rotate
    hflip = hflip and random.random() < 0.5
    vflip = rot and random.random() < 0.5
    rot90 = rot and random.random() < 0.5
    #rot90n = rot and random.random() < 0.5

    def _augment(img):
        if hflip: img = np.flip(img, axis=1) #img[:, ::-1, :]
        if vflip: img = np.flip(img, axis=0) #img[::-1, :, :]
        #if rot90: img = img.transpose(1, 0, 2)
        if rot90: img = np.rot90(img, 1) #90 degrees # In PIL: img.transpose(Image.ROTATE_90)
        #if rot90n: img = np.rot90(img, -1) #-90 degrees
        return img

    return [_augment(img) for img in img_list]



####################
# Normalization functions
####################


#TODO: Could also automatically detect the possible range with min and max, like in def ssim()
def denorm(x, min_max=(-1.0, 1.0)):
    '''
        Denormalize from [-1,1] range to [0,1]
        formula: xi' = (xi - mu)/sigma
        Example: "out = (x + 1.0) / 2.0" for denorm 
            range (-1,1) to (0,1)
        for use with proper act in Generator output (ie. tanh)
    '''
    out = (x - min_max[0]) / (min_max[1] - min_max[0])
    if isinstance(x, torch.Tensor):
        return out.clamp(0, 1)
    elif isinstance(x, np.ndarray):
        return np.clip(out, 0, 1)
    else:
        raise TypeError("Got unexpected object type, expected torch.Tensor or \
        np.ndarray")

def norm(x): 
    #Normalize (z-norm) from [0,1] range to [-1,1]
    out = (x - 0.5) * 2.0
    if isinstance(x, torch.Tensor):
        return out.clamp(-1, 1)
    elif isinstance(x, np.ndarray):
        return np.clip(out, -1, 1)
    else:
        raise TypeError("Got unexpected object type, expected torch.Tensor or \
        np.ndarray")


####################
# np and tensor conversions
####################


#2tensor
def np2tensor(img, bgr2rgb=True, data_range=1., normalize=False, change_range=True, add_batch=True):
    """
    Converts a numpy image array into a Tensor array.
    Parameters:
        img (numpy array): the input image numpy array
        add_batch (bool): choose if new tensor needs batch dimension added 
    """
    if not isinstance(img, np.ndarray): #images expected to be uint8 -> 255
        raise TypeError("Got unexpected object type, expected np.ndarray")
    #check how many channels the image has, then condition, like in my BasicSR. ie. RGB, RGBA, Gray
    #if bgr2rgb:
        #img = img[:, :, [2, 1, 0]] #BGR to RGB -> in numpy, if using OpenCV, else not needed. Only if image has colors.
    if change_range:
        if np.issubdtype(img.dtype, np.integer):
            info = np.iinfo
        elif np.issubdtype(img.dtype, np.floating):
            info = np.finfo
        img = img*data_range/info(img.dtype).max #uint8 = /255
    img = torch.from_numpy(np.ascontiguousarray(np.transpose(img, (2, 0, 1)))).float() #"HWC to CHW" and "numpy to tensor"
    if bgr2rgb:
        if img.shape[0] == 3: #RGB
            #BGR to RGB -> in tensor, if using OpenCV, else not needed. Only if image has colors.
            img = bgr_to_rgb(img)
        elif img.shape[0] == 4: #RGBA
            #BGR to RGB -> in tensor, if using OpenCV, else not needed. Only if image has colors.)
            img = bgra_to_rgba(img)
    if add_batch:
        img.unsqueeze_(0) # Add fake batch dimension = 1 . squeeze() will remove the dimensions of size 1
    if normalize:
        img = norm(img)
    return img

#2np
def tensor2np(img, rgb2bgr=True, remove_batch=True, data_range=255, 
              denormalize=False, change_range=True, imtype=np.uint8):
    """
    Converts a Tensor array into a numpy image array.
    Parameters:
        img (tensor): the input image tensor array
            4D(B,(3/1),H,W), 3D(C,H,W), or 2D(H,W), any range, RGB channel order
        remove_batch (bool): choose if tensor of shape BCHW needs to be squeezed 
        denormalize (bool): Used to denormalize from [-1,1] range back to [0,1]
        imtype (type): the desired type of the converted numpy array (np.uint8 
            default)
    Output: 
        img (np array): 3D(H,W,C) or 2D(H,W), [0,255], np.uint8 (default)
    """
    if not isinstance(img, torch.Tensor):
        raise TypeError("Got unexpected object type, expected torch.Tensor")
    n_dim = img.dim()

    #TODO: Check: could denormalize here in tensor form instead, but end result is the same
    
    img = img.float().cpu()  
    
    if n_dim == 4 or n_dim == 3:
        #if n_dim == 4, has to convert to 3 dimensions, either removing batch or by creating a grid
        if n_dim == 4 and remove_batch:
            if img.shape[0] > 1:
                # leave only the first image in the batch
                img = img[0,...] 
            else:
                # remove a fake batch dimension
                img = img.squeeze()
                # squeeze removes batch and channel of grayscale images (dimensions = 1)
                if len(img.shape) < 3: 
                    #add back the lost channel dimension
                    img = img.unsqueeze(dim=0)
        # convert images in batch (BCHW) to a grid of all images (C B*H B*W)
        else:
            n_img = len(img)
            img = make_grid(img, nrow=int(math.sqrt(n_img)), normalize=False)
        
        if img.shape[0] == 3 and rgb2bgr: #RGB
            #RGB to BGR -> in tensor, if using OpenCV, else not needed. Only if image has colors.
            img_np = rgb_to_bgr(img).numpy()
        elif img.shape[0] == 4 and rgb2bgr: #RGBA
            #RGBA to BGRA -> in tensor, if using OpenCV, else not needed. Only if image has colors.
            img_np = rgba_to_bgra(img).numpy()
        else:
            img_np = img.numpy()
        img_np = np.transpose(img_np, (1, 2, 0))  # "CHW to HWC" -> # HWC, BGR
    elif n_dim == 2:
        img_np = img.numpy()
    else:
        raise TypeError(
            'Only support 4D, 3D and 2D tensor. But received with dimension: {:d}'.format(n_dim))

    #if rgb2bgr:
        #img_np = img_np[[2, 1, 0], :, :] #RGB to BGR -> in numpy, if using OpenCV, else not needed. Only if image has colors.
    #TODO: Check: could denormalize in the begining in tensor form instead
    if denormalize:
        img_np = denorm(img_np) #denormalize if needed
    if change_range:
        img_np = np.clip(data_range*img_np,0,data_range).round() #clip to the data_range
        # Important. Unlike matlab, numpy.unit8() WILL NOT round by default.
    #has to be in range (0,255) before changing to np.uint8, else np.float32
    return img_np.astype(imtype)




####################
# Prepare Images
####################
# https://github.com/sunreef/BlindSR/blob/master/src/image_utils.py
def patchify_tensor(features, patch_size, overlap=10):
    batch_size, channels, height, width = features.size()

    effective_patch_size = patch_size - overlap
    n_patches_height = (height // effective_patch_size)
    n_patches_width = (width // effective_patch_size)

    if n_patches_height * effective_patch_size < height:
        n_patches_height += 1
    if n_patches_width * effective_patch_size < width:
        n_patches_width += 1

    patches = []
    for b in range(batch_size):
        for h in range(n_patches_height):
            for w in range(n_patches_width):
                patch_start_height = min(h * effective_patch_size, height - patch_size)
                patch_start_width = min(w * effective_patch_size, width - patch_size)
                patches.append(features[b:b+1, :,
                               patch_start_height: patch_start_height + patch_size,
                               patch_start_width: patch_start_width + patch_size])
    return torch.cat(patches, 0)

def recompose_tensor(patches, full_height, full_width, overlap=10):

    batch_size, channels, patch_size, _ = patches.size()
    effective_patch_size = patch_size - overlap
    n_patches_height = (full_height // effective_patch_size)
    n_patches_width = (full_width // effective_patch_size)

    if n_patches_height * effective_patch_size < full_height:
        n_patches_height += 1
    if n_patches_width * effective_patch_size < full_width:
        n_patches_width += 1

    n_patches = n_patches_height * n_patches_width
    if batch_size % n_patches != 0:
        print("Error: The number of patches provided to the recompose function does not match the number of patches in each image.")
    final_batch_size = batch_size // n_patches

    blending_in = torch.linspace(0.1, 1.0, overlap)
    blending_out = torch.linspace(1.0, 0.1, overlap)
    middle_part = torch.ones(patch_size - 2 * overlap)
    blending_profile = torch.cat([blending_in, middle_part, blending_out], 0)

    horizontal_blending = blending_profile[None].repeat(patch_size, 1)
    vertical_blending = blending_profile[:, None].repeat(1, patch_size)
    blending_patch = horizontal_blending * vertical_blending

    blending_image = torch.zeros(1, channels, full_height, full_width)
    for h in range(n_patches_height):
        for w in range(n_patches_width):
            patch_start_height = min(h * effective_patch_size, full_height - patch_size)
            patch_start_width = min(w * effective_patch_size, full_width - patch_size)
            blending_image[0, :, patch_start_height: patch_start_height + patch_size, patch_start_width: patch_start_width + patch_size] += blending_patch[None]

    recomposed_tensor = torch.zeros(final_batch_size, channels, full_height, full_width)
    if patches.is_cuda:
        blending_patch = blending_patch.cuda()
        blending_image = blending_image.cuda()
        recomposed_tensor = recomposed_tensor.cuda()
    patch_index = 0
    for b in range(final_batch_size):
        for h in range(n_patches_height):
            for w in range(n_patches_width):
                patch_start_height = min(h * effective_patch_size, full_height - patch_size)
                patch_start_width = min(w * effective_patch_size, full_width - patch_size)
                recomposed_tensor[b, :, patch_start_height: patch_start_height + patch_size, patch_start_width: patch_start_width + patch_size] += patches[patch_index] * blending_patch
                patch_index += 1
    recomposed_tensor /= blending_image

    return recomposed_tensor


















#TODO: imresize could be an independent file (imresize.py)
####################
# Matlab imresize
####################


# These next functions are all interpolation methods. x is the distance from the left pixel center
def cubic(x):
    absx = torch.abs(x)
    absx2 = absx**2
    absx3 = absx**3
    return (1.5*absx3 - 2.5*absx2 + 1) * ((absx <= 1).type_as(absx)) + \
        (-0.5*absx3 + 2.5*absx2 - 4*absx + 2) * (((absx > 1)*(absx <= 2)).type_as(absx))

def box(x):
    return ((-0.5 <= x) & (x < 0.5)) * 1.0

def linear(x):
    return (x + 1) * ((-1 <= x) & (x < 0)) + (1 - x) * ((0 <= x) & (x <= 1))

def lanczos2(x):
    return (((torch.sin(math.pi*x) * torch.sin(math.pi*x/2) + torch.finfo(torch.float32).eps) /
             ((math.pi**2 * x**2 / 2) + torch.finfo(torch.float32).eps))
            * (torch.abs(x) < 2))

def lanczos3(x):
    return (((torch.sin(math.pi*x) * torch.sin(math.pi*x/3) + torch.finfo(torch.float32).eps) /
            ((math.pi**2 * x**2 / 3) + torch.finfo(torch.float32).eps))
            * (torch.abs(x) < 3))


def calculate_weights_indices(in_length, out_length, scale, kernel, kernel_width, antialiasing):
    if (scale < 1) and (antialiasing):
        # Use a modified kernel to simultaneously interpolate and antialias- larger kernel width
        kernel_width = kernel_width / scale

    # Output-space coordinates
    x = torch.linspace(1, out_length, out_length)

    # Input-space coordinates. Calculate the inverse mapping such that 0.5
    # in output space maps to 0.5 in input space, and 0.5+scale in output
    # space maps to 1.5 in input space.
    u = x / scale + 0.5 * (1 - 1 / scale)

    # What is the left-most pixel that can be involved in the computation?
    left = torch.floor(u - kernel_width / 2)

    # What is the maximum number of pixels that can be involved in the
    # computation?  Note: it's OK to use an extra pixel here; if the
    # corresponding weights are all zero, it will be eliminated at the end
    # of this function.
    P = math.ceil(kernel_width) + 2

    # The indices of the input pixels involved in computing the k-th output
    # pixel are in row k of the indices matrix.
    indices = left.view(out_length, 1).expand(out_length, P) + torch.linspace(0, P - 1, P).view(
        1, P).expand(out_length, P)

    # The weights used to compute the k-th output pixel are in row k of the
    # weights matrix.
    distance_to_center = u.view(out_length, 1).expand(out_length, P) - indices
    # apply kernel
    if (scale < 1) and (antialiasing):
        weights = scale * kernel(distance_to_center * scale)
    else:
        weights = kernel(distance_to_center)
    # Normalize the weights matrix so that each row sums to 1.
    weights_sum = torch.sum(weights, 1).view(out_length, 1)
    weights = weights / weights_sum.expand(out_length, P)

    # If a column in weights is all zero, get rid of it. only consider the first and last column.
    weights_zero_tmp = torch.sum((weights == 0), 0)
    if not math.isclose(weights_zero_tmp[0], 0, rel_tol=1e-6):
        indices = indices.narrow(1, 1, P - 2)
        weights = weights.narrow(1, 1, P - 2)
    if not math.isclose(weights_zero_tmp[-1], 0, rel_tol=1e-6):
        indices = indices.narrow(1, 0, P - 2)
        weights = weights.narrow(1, 0, P - 2)
    weights = weights.contiguous()
    indices = indices.contiguous()
    sym_len_s = -indices.min() + 1
    sym_len_e = indices.max() - in_length
    indices = indices + sym_len_s - 1
    return weights, indices, int(sym_len_s), int(sym_len_e)


def imresize(img, scale, antialiasing=True, interpolation=None):
    # The scale should be the same for H and W
    # input: img: CHW RGB [0,1]
    # output: CHW RGB [0,1] w/o round

    in_C, in_H, in_W = img.size()
    out_C, out_H, out_W = in_C, math.ceil(in_H * scale), math.ceil(in_W * scale)

    # Choose interpolation method, each method has the matching kernel size
    kernel, kernel_width = {
        "cubic": (cubic, 4.0),
        "lanczos2": (lanczos2, 4.0),
        "lanczos3": (lanczos3, 6.0),
        "box": (box, 1.0),
        "linear": (linear, 2.0),
        None: (cubic, 4.0)  # set default interpolation method as cubic
    }.get(interpolation)

    # Return the desired dimension order for performing the resize.  The
    # strategy is to perform the resize first along the dimension with the
    # smallest scale factor.
    # Now we do not support this.

    # get weights and indices
    weights_H, indices_H, sym_len_Hs, sym_len_He = calculate_weights_indices(
        in_H, out_H, scale, kernel, kernel_width, antialiasing)
    weights_W, indices_W, sym_len_Ws, sym_len_We = calculate_weights_indices(
        in_W, out_W, scale, kernel, kernel_width, antialiasing)
    # process H dimension
    # symmetric copying
    img_aug = torch.FloatTensor(in_C, in_H + sym_len_Hs + sym_len_He, in_W)
    img_aug.narrow(1, sym_len_Hs, in_H).copy_(img)

    sym_patch = img[:, :sym_len_Hs, :]
    inv_idx = torch.arange(sym_patch.size(1) - 1, -1, -1).long()
    sym_patch_inv = sym_patch.index_select(1, inv_idx)
    img_aug.narrow(1, 0, sym_len_Hs).copy_(sym_patch_inv)

    sym_patch = img[:, -sym_len_He:, :]
    inv_idx = torch.arange(sym_patch.size(1) - 1, -1, -1).long()
    sym_patch_inv = sym_patch.index_select(1, inv_idx)
    img_aug.narrow(1, sym_len_Hs + in_H, sym_len_He).copy_(sym_patch_inv)

    out_1 = torch.FloatTensor(in_C, out_H, in_W)
    kernel_width = weights_H.size(1)
    for i in range(out_H):
        idx = int(indices_H[i][0])
        out_1[0, i, :] = img_aug[0, idx:idx + kernel_width, :].transpose(0, 1).mv(weights_H[i])
        out_1[1, i, :] = img_aug[1, idx:idx + kernel_width, :].transpose(0, 1).mv(weights_H[i])
        out_1[2, i, :] = img_aug[2, idx:idx + kernel_width, :].transpose(0, 1).mv(weights_H[i])

    # process W dimension
    # symmetric copying
    out_1_aug = torch.FloatTensor(in_C, out_H, in_W + sym_len_Ws + sym_len_We)
    out_1_aug.narrow(2, sym_len_Ws, in_W).copy_(out_1)

    sym_patch = out_1[:, :, :sym_len_Ws]
    inv_idx = torch.arange(sym_patch.size(2) - 1, -1, -1).long()
    sym_patch_inv = sym_patch.index_select(2, inv_idx)
    out_1_aug.narrow(2, 0, sym_len_Ws).copy_(sym_patch_inv)

    sym_patch = out_1[:, :, -sym_len_We:]
    inv_idx = torch.arange(sym_patch.size(2) - 1, -1, -1).long()
    sym_patch_inv = sym_patch.index_select(2, inv_idx)
    out_1_aug.narrow(2, sym_len_Ws + in_W, sym_len_We).copy_(sym_patch_inv)

    out_2 = torch.FloatTensor(in_C, out_H, out_W)
    kernel_width = weights_W.size(1)
    for i in range(out_W):
        idx = int(indices_W[i][0])
        out_2[0, :, i] = out_1_aug[0, :, idx:idx + kernel_width].mv(weights_W[i])
        out_2[1, :, i] = out_1_aug[1, :, idx:idx + kernel_width].mv(weights_W[i])
        out_2[2, :, i] = out_1_aug[2, :, idx:idx + kernel_width].mv(weights_W[i])

    return out_2


def imresize_np(img, scale, antialiasing=True, interpolation=None):
    # Now the scale should be the same for H and W
    # input: img: Numpy, HWC BGR [0,1]
    # output: HWC BGR [0,1] w/o round
    
    change_range = False
    if img.max() > 1:
        img_type = img.dtype
        if np.issubdtype(img_type, np.integer):
            info = np.iinfo
        elif np.issubdtype(img_type, np.floating):
            info = np.finfo
        img = img/info(img_type).max
        change_range = True

    img = torch.from_numpy(img)

    in_H, in_W, in_C = img.size()
    out_C, out_H, out_W = in_C, math.ceil(in_H * scale), math.ceil(in_W * scale)

    # Choose interpolation method, each method has the matching kernel size
    kernel, kernel_width = {
        "cubic": (cubic, 4.0),
        "lanczos2": (lanczos2, 4.0),
        "lanczos3": (lanczos3, 6.0),
        "box": (box, 1.0),
        "linear": (linear, 2.0),
        None: (cubic, 4.0)  # set default interpolation method as cubic
    }.get(interpolation)

    # Return the desired dimension order for performing the resize.  The
    # strategy is to perform the resize first along the dimension with the
    # smallest scale factor.
    # Now we do not support this.

    # get weights and indices
    weights_H, indices_H, sym_len_Hs, sym_len_He = calculate_weights_indices(
        in_H, out_H, scale, kernel, kernel_width, antialiasing)
    weights_W, indices_W, sym_len_Ws, sym_len_We = calculate_weights_indices(
        in_W, out_W, scale, kernel, kernel_width, antialiasing)
    # process H dimension
    # symmetric copying
    img_aug = torch.FloatTensor(in_H + sym_len_Hs + sym_len_He, in_W, in_C)
    img_aug.narrow(0, sym_len_Hs, in_H).copy_(img)

    sym_patch = img[:sym_len_Hs, :, :]
    inv_idx = torch.arange(sym_patch.size(0) - 1, -1, -1).long()
    sym_patch_inv = sym_patch.index_select(0, inv_idx)
    img_aug.narrow(0, 0, sym_len_Hs).copy_(sym_patch_inv)

    sym_patch = img[-sym_len_He:, :, :]
    inv_idx = torch.arange(sym_patch.size(0) - 1, -1, -1).long()
    sym_patch_inv = sym_patch.index_select(0, inv_idx)
    img_aug.narrow(0, sym_len_Hs + in_H, sym_len_He).copy_(sym_patch_inv)

    out_1 = torch.FloatTensor(out_H, in_W, in_C)
    kernel_width = weights_H.size(1)
    for i in range(out_H):
        idx = int(indices_H[i][0])
        out_1[i, :, 0] = img_aug[idx:idx + kernel_width, :, 0].transpose(0, 1).mv(weights_H[i])
        out_1[i, :, 1] = img_aug[idx:idx + kernel_width, :, 1].transpose(0, 1).mv(weights_H[i])
        out_1[i, :, 2] = img_aug[idx:idx + kernel_width, :, 2].transpose(0, 1).mv(weights_H[i])

    # process W dimension
    # symmetric copying
    out_1_aug = torch.FloatTensor(out_H, in_W + sym_len_Ws + sym_len_We, in_C)
    out_1_aug.narrow(1, sym_len_Ws, in_W).copy_(out_1)

    sym_patch = out_1[:, :sym_len_Ws, :]
    inv_idx = torch.arange(sym_patch.size(1) - 1, -1, -1).long()
    sym_patch_inv = sym_patch.index_select(1, inv_idx)
    out_1_aug.narrow(1, 0, sym_len_Ws).copy_(sym_patch_inv)

    sym_patch = out_1[:, -sym_len_We:, :]
    inv_idx = torch.arange(sym_patch.size(1) - 1, -1, -1).long()
    sym_patch_inv = sym_patch.index_select(1, inv_idx)
    out_1_aug.narrow(1, sym_len_Ws + in_W, sym_len_We).copy_(sym_patch_inv)

    out_2 = torch.FloatTensor(out_H, out_W, in_C)
    kernel_width = weights_W.size(1)
    for i in range(out_W):
        idx = int(indices_W[i][0])
        out_2[:, i, 0] = out_1_aug[:, idx:idx + kernel_width, 0].mv(weights_W[i])
        out_2[:, i, 1] = out_1_aug[:, idx:idx + kernel_width, 1].mv(weights_W[i])
        out_2[:, i, 2] = out_1_aug[:, idx:idx + kernel_width, 2].mv(weights_W[i])

    out_2 = out_2.numpy().clip(0,1)
    
    if change_range:
        out_2 = out_2*info(img_type).max #uint8 = 255
        out_2 = out_2.astype(img_type)

    return out_2


if __name__ == '__main__':
    # test imresize function
    # read images
    img = cv2.imread('test.png')
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    # imresize
    scale = 1 / 4
    import time
    total_time = 0
    for i in range(10):
        start_time = time.time()
        rlt = imresize(img, scale, antialiasing=True)
        use_time = time.time() - start_time
        total_time += use_time
    print('average time: {}'.format(total_time / 10))

    import torchvision.utils
    torchvision.utils.save_image(
        (rlt * 255).round() / 255, 'rlt.png', nrow=1, padding=0, normalize=False)


In [ ]:
#@title perceptual.py
%%writefile /content/Colab-edge-connect/src/vic/perceptual.py
"""
BasicSR/codes/models/modules/architectures/perceptual.py (8-Nov-20)
https://github.com/victorca25/BasicSR/blob/dev2/codes/models/modules/architectures/perceptual.py
"""
import math
import torch
import torch.nn as nn
#import torchvision
import torchvision.models.vgg as vgg
import torchvision.models.resnet as resnet

from collections import OrderedDict


####################
# Perceptual Network
####################


# Assume input range is [0, 1]
class VGGFeatureExtractor(nn.Module):
    r"""
    Perceptual loss, VGG-based
    https://arxiv.org/abs/1603.08155
    https://github.com/dxyang/StyleTransfer/blob/master/utils.py
    """
    def __init__(self,
                 feature_layer=34,
                 use_bn=False,
                 use_input_norm=True,
                 device=torch.device('cpu'),
                 z_norm=False): #Note: PPON uses cuda instead of CPU
        super(VGGFeatureExtractor, self).__init__()
        if use_bn:
            model = vgg.vgg19_bn(pretrained=True)
        else:
            model = vgg.vgg19(pretrained=True)
        self.use_input_norm = use_input_norm
        if self.use_input_norm:
            if z_norm: # if input in range [-1,1]
                mean = torch.Tensor([0.485-1, 0.456-1, 0.406-1]).view(1, 3, 1, 1).to(device) 
                std = torch.Tensor([0.229*2, 0.224*2, 0.225*2]).view(1, 3, 1, 1).to(device)
            else: # input in range [0,1]
                mean = torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(device)                 
                std = torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(device)
            self.register_buffer('mean', mean)
            self.register_buffer('std', std)
        self.features = nn.Sequential(*list(model.features.children())[:(feature_layer + 1)])
        # No need to BP to variable
        for k, v in self.features.named_parameters():
            v.requires_grad = False

    def forward(self, x):
        if self.use_input_norm:
            x = (x - self.mean) / self.std
        output = self.features(x)
        return output


# VGG 19 layers to listen to
vgg_layer19 = {
    'conv_1_1': 0, 'conv_1_2': 2, 'pool_1': 4, 'conv_2_1': 5, 'conv_2_2': 7, 'pool_2': 9, 'conv_3_1': 10, 'conv_3_2': 12, 'conv_3_3': 14, 'conv_3_4': 16, 'pool_3': 18, 'conv_4_1': 19, 'conv_4_2': 21, 'conv_4_3': 23, 'conv_4_4': 25, 'pool_4': 27, 'conv_5_1': 28, 'conv_5_2': 30, 'conv_5_3': 32, 'conv_5_4': 34, 'pool_5': 36
}
vgg_layer_inv19 = {
    0: 'conv_1_1', 2: 'conv_1_2', 4: 'pool_1', 5: 'conv_2_1', 7: 'conv_2_2', 9: 'pool_2', 10: 'conv_3_1', 12: 'conv_3_2', 14: 'conv_3_3', 16: 'conv_3_4', 18: 'pool_3', 19: 'conv_4_1', 21: 'conv_4_2', 23: 'conv_4_3', 25: 'conv_4_4', 27: 'pool_4', 28: 'conv_5_1', 30: 'conv_5_2', 32: 'conv_5_3', 34: 'conv_5_4', 36: 'pool_5'
}
# VGG 16 layers to listen to
vgg_layer16 = {
    'conv_1_1': 0, 'conv_1_2': 2, 'pool_1': 4, 'conv_2_1': 5, 'conv_2_2': 7, 'pool_2': 9, 'conv_3_1': 10, 'conv_3_2': 12, 'conv_3_3': 14, 'pool_3': 16, 'conv_4_1': 17, 'conv_4_2': 19, 'conv_4_3': 21, 'pool_4': 23, 'conv_5_1': 24, 'conv_5_2': 26, 'conv_5_3': 28, 'pool_5': 30
}
vgg_layer_inv16 = {
    0: 'conv_1_1', 2: 'conv_1_2', 4: 'pool_1', 5: 'conv_2_1', 7: 'conv_2_2', 9: 'pool_2', 10: 'conv_3_1', 12: 'conv_3_2', 14: 'conv_3_3', 16: 'pool_3', 17: 'conv_4_1', 19: 'conv_4_2', 21: 'conv_4_3', 23: 'pool_4', 24: 'conv_5_1', 26: 'conv_5_2', 28: 'conv_5_3', 30: 'pool_5'
}

class VGG_Model(nn.Module):
    """
        A VGG model with listerners in the layers. 
        Will return a dictionary of outputs that correspond to the 
        layers set in "listen_list".
    """
    def __init__(self, listen_list=None, net='vgg19', use_input_norm=True, z_norm=False):
        super(VGG_Model, self).__init__()
        #vgg = vgg16(pretrained=True)
        if net == 'vgg19':
            vgg_net = vgg.vgg19(pretrained=True)
            vgg_layer = vgg_layer19
            self.vgg_layer_inv = vgg_layer_inv19
        elif net == 'vgg16':
            vgg_net = vgg.vgg16(pretrained=True)
            vgg_layer = vgg_layer16
            self.vgg_layer_inv = vgg_layer_inv16
        self.vgg_model = vgg_net.features
        self.use_input_norm = use_input_norm
        # image normalization
        if self.use_input_norm:
            if z_norm: # if input in range [-1,1]
                mean = torch.tensor(
                    [[[0.485-1]], [[0.456-1]], [[0.406-1]]], requires_grad=False)
                std = torch.tensor(
                    [[[0.229*2]], [[0.224*2]], [[0.225*2]]], requires_grad=False)
            else: # input in range [0,1]
                mean = torch.tensor(
                    [[[0.485]], [[0.456]], [[0.406]]], requires_grad=False)
                std = torch.tensor(
                    [[[0.229]], [[0.224]], [[0.225]]], requires_grad=False)
            self.register_buffer('mean', mean)
            self.register_buffer('std', std)

        vgg_dict = vgg_net.state_dict()
        vgg_f_dict = self.vgg_model.state_dict()
        vgg_dict = {k: v for k, v in vgg_dict.items() if k in vgg_f_dict}
        vgg_f_dict.update(vgg_dict)
        # no grad
        for p in self.vgg_model.parameters():
            p.requires_grad = False
        if listen_list == []:
            self.listen = []
        else:
            self.listen = set()
            for layer in listen_list:
                self.listen.add(vgg_layer[layer])
        self.features = OrderedDict()

    def forward(self, x):
        if self.use_input_norm:
            x = (x - self.mean.detach()) / self.std.detach()

        for index, layer in enumerate(self.vgg_model):
            x = layer(x)
            if index in self.listen:
                self.features[self.vgg_layer_inv[index]] = x
        return self.features




# Assume input range is [0, 1]
class ResNet101FeatureExtractor(nn.Module):
    def __init__(self, use_input_norm=True, device=torch.device('cpu'), z_norm=False):
        super(ResNet101FeatureExtractor, self).__init__()
        model = resnet.resnet101(pretrained=True)
        self.use_input_norm = use_input_norm
        if self.use_input_norm:
            if z_norm: # if input in range [-1,1]
                mean = torch.Tensor([0.485-1, 0.456-1, 0.406-1]).view(1, 3, 1, 1).to(device)
                std = torch.Tensor([0.229*2, 0.224*2, 0.225*2]).view(1, 3, 1, 1).to(device)
            else: # input in range [0,1]
                mean = torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(device)
                std = torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(device)
            self.register_buffer('mean', mean)
            self.register_buffer('std', std)
        self.features = nn.Sequential(*list(model.children())[:8])
        # No need to BP to variable
        for k, v in self.features.named_parameters():
            v.requires_grad = False

    def forward(self, x):
        if self.use_input_norm:
            x = (x - self.mean) / self.std
        output = self.features(x)
        return output


class MINCNet(nn.Module):
    def __init__(self):
        super(MINCNet, self).__init__()
        self.ReLU = nn.ReLU(True)
        self.conv11 = nn.Conv2d(3, 64, 3, 1, 1)
        self.conv12 = nn.Conv2d(64, 64, 3, 1, 1)
        self.maxpool1 = nn.MaxPool2d(2, stride=2, padding=0, ceil_mode=True)
        self.conv21 = nn.Conv2d(64, 128, 3, 1, 1)
        self.conv22 = nn.Conv2d(128, 128, 3, 1, 1)
        self.maxpool2 = nn.MaxPool2d(2, stride=2, padding=0, ceil_mode=True)
        self.conv31 = nn.Conv2d(128, 256, 3, 1, 1)
        self.conv32 = nn.Conv2d(256, 256, 3, 1, 1)
        self.conv33 = nn.Conv2d(256, 256, 3, 1, 1)
        self.maxpool3 = nn.MaxPool2d(2, stride=2, padding=0, ceil_mode=True)
        self.conv41 = nn.Conv2d(256, 512, 3, 1, 1)
        self.conv42 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv43 = nn.Conv2d(512, 512, 3, 1, 1)
        self.maxpool4 = nn.MaxPool2d(2, stride=2, padding=0, ceil_mode=True)
        self.conv51 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv52 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv53 = nn.Conv2d(512, 512, 3, 1, 1)

    def forward(self, x):
        out = self.ReLU(self.conv11(x))
        out = self.ReLU(self.conv12(out))
        out = self.maxpool1(out)
        out = self.ReLU(self.conv21(out))
        out = self.ReLU(self.conv22(out))
        out = self.maxpool2(out)
        out = self.ReLU(self.conv31(out))
        out = self.ReLU(self.conv32(out))
        out = self.ReLU(self.conv33(out))
        out = self.maxpool3(out)
        out = self.ReLU(self.conv41(out))
        out = self.ReLU(self.conv42(out))
        out = self.ReLU(self.conv43(out))
        out = self.maxpool4(out)
        out = self.ReLU(self.conv51(out))
        out = self.ReLU(self.conv52(out))
        out = self.conv53(out)
        return out


# Assume input range is [0, 1]
class MINCFeatureExtractor(nn.Module):
    def __init__(self, feature_layer=34, use_bn=False, use_input_norm=True, \
                device=torch.device('cpu')):
        super(MINCFeatureExtractor, self).__init__()

        self.features = MINCNet()
        self.features.load_state_dict(
            torch.load('../experiments/pretrained_models/VGG16minc_53.pth'), strict=True)
        self.features.eval()
        # No need to BP to variable
        for k, v in self.features.named_parameters():
            v.requires_grad = False

    def forward(self, x):
        output = self.features(x)
        return output




In [ ]:
#@title vic/filters.py
%%writefile /content/Colab-edge-connect/src/vic/filters.py
"""
BasicSR/codes/dataops/filters.py (8-Nov-20)
https://github.com/victorca25/BasicSR/blob/dev2/codes/dataops/filters.py
"""

'''
    Multiple image filters used by different functions. Can also be used as augmentations.
'''

import numbers
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

#from dataops.common import denorm


def get_kernel_size(sigma = 6):
    '''
        Get optimal gaussian kernel size according to sigma * 6 criterion 
        (must return an int)
        Alternative from Matlab: kernel_size=2*np.ceil(3*sigma)+1
        https://stackoverflow.com/questions/3149279/optimal-sigma-for-gaussian-filtering-of-an-image
    '''
    kernel_size = np.ceil(sigma*6)
    return kernel_size

def get_kernel_sigma(kernel_size = 5):
    '''
        Get optimal gaussian kernel sigma (variance) according to kernel_size/6 
        Alternative from Matlab: sigma = (kernel_size-1)/6
    '''
    return kernel_size/6.0

def get_kernel_mean(kernel_size = 5):
    '''
        Get gaussian kernel mean
    '''
    return (kernel_size - 1) / 2.0

def kernel_conv_w(kernel, channels: int =3):
    '''
        Reshape a H*W kernel to 2d depthwise convolutional 
            weight (for loading in a Conv2D)
    '''

    # Dynamic window expansion. expand() does not copy memory, needs contiguous()
    kernel = kernel.expand(channels, 1, *kernel.size()).contiguous()
    return kernel

#@torch.jit.script
def get_gaussian_kernel1d(kernel_size: int,
                sigma: float = 1.5, 
                #channel: int = None,
                force_even: bool = False) -> torch.Tensor:
    r"""Function that returns 1-D Gaussian filter kernel coefficients.

    Args:
        kernel_size (int): filter/window size. It should be odd and positive.
        sigma (float): gaussian standard deviation, sigma of normal distribution
        force_even (bool): overrides requirement for odd kernel size.

    Returns:
        torch.Tensor: 1D tensor with 1D gaussian filter coefficients.

    Shape:
        - Output: :math:`(\text{kernel_size})`

    Examples::

        >>> get_gaussian_kernel1d(3, 2.5)
        tensor([0.3243, 0.3513, 0.3243])

        >>> get_gaussian_kernel1d(5, 1.5)
        tensor([0.1201, 0.2339, 0.2921, 0.2339, 0.1201])
    """
        
    if (not isinstance(kernel_size, int) or (
            (kernel_size % 2 == 0) and not force_even) or (
            kernel_size <= 0)):
        raise TypeError(
            "kernel_size must be an odd positive integer. "
            "Got {}".format(kernel_size)
        )

    if kernel_size % 2 == 0:
        x = torch.arange(kernel_size).float() - kernel_size // 2    
        x = x + 0.5
        gauss = torch.exp((-x.pow(2.0) / float(2 * sigma ** 2)))
    else: #much faster
        gauss = torch.Tensor([np.exp(-(x - kernel_size//2)**2/float(2*sigma**2)) for x in range(kernel_size)])

    gauss /= gauss.sum()
    
    return gauss

#To get the kernel coefficients
def get_gaussian_kernel2d(
        #kernel_size: Tuple[int, int],
        kernel_size,
        #sigma: Tuple[float, float],
        sigma,
        force_even: bool = False) -> torch.Tensor:
    r"""Function that returns Gaussian filter matrix coefficients.
         Modified with a faster kernel creation if the kernel size
         is odd. 
    Args:
        kernel_size (Tuple[int, int]): filter (window) sizes in the x and y 
         direction. Sizes should be odd and positive, unless force_even is
         used.
        sigma (Tuple[int, int]): gaussian standard deviation in the x and y
         direction.
        force_even (bool): overrides requirement for odd kernel size.

    Returns:
        Tensor: 2D tensor with gaussian filter matrix coefficients.

    Shape:
        - Output: :math:`(\text{kernel_size}_x, \text{kernel_size}_y)`

    Examples::

        >>> get_gaussian_kernel2d((3, 3), (1.5, 1.5))
        tensor([[0.0947, 0.1183, 0.0947],
                [0.1183, 0.1478, 0.1183],
                [0.0947, 0.1183, 0.0947]])

        >>> get_gaussian_kernel2d((3, 5), (1.5, 1.5))
        tensor([[0.0370, 0.0720, 0.0899, 0.0720, 0.0370],
                [0.0462, 0.0899, 0.1123, 0.0899, 0.0462],
                [0.0370, 0.0720, 0.0899, 0.0720, 0.0370]])
    """

    if isinstance(kernel_size, (int, float)): 
        kernel_size = (kernel_size, kernel_size)

    if isinstance(sigma, (int, float)): 
        sigma = (sigma, sigma)

    if not isinstance(kernel_size, tuple) or len(kernel_size) != 2:
        raise TypeError(
            "kernel_size must be a tuple of length two. Got {}".format(
                kernel_size
            )
        )
    if not isinstance(sigma, tuple) or len(sigma) != 2:
        raise TypeError(
            "sigma must be a tuple of length two. Got {}".format(sigma)
        )
    ksize_x, ksize_y = kernel_size
    sigma_x, sigma_y = sigma
    kernel_x: torch.Tensor = get_gaussian_kernel1d(ksize_x, sigma_x, force_even)
    kernel_y: torch.Tensor = get_gaussian_kernel1d(ksize_y, sigma_y, force_even)
    
    kernel_2d: torch.Tensor = torch.matmul(
        kernel_x.unsqueeze(-1), kernel_y.unsqueeze(-1).t()
    )
    
    return kernel_2d

def get_gaussian_kernel(kernel_size=5, sigma=3, dim=2):
    '''
        This function can generate gaussian kernels in any dimension,
            but its 3 times slower than get_gaussian_kernel2d()
    Arguments:
        kernel_size (Tuple[int, int]): filter sizes in the x and y direction.
            Sizes should be odd and positive.
        sigma (Tuple[int, int]): gaussian standard deviation in the x and y
            direction.
        dim: the image dimension (2D=2, 3D=3, etc)
    Returns:
        Tensor: tensor with gaussian filter matrix coefficients.
    '''

    if isinstance(kernel_size, numbers.Number):
        kernel_size = [kernel_size] * dim
    if isinstance(sigma, numbers.Number):
        sigma = [sigma] * dim

    kernel = 1
    meshgrids = torch.meshgrid(
        [
            torch.arange(size, dtype=torch.float32)
            for size in kernel_size
        ]
    )
    for size, std, mgrid in zip(kernel_size, sigma, meshgrids):
        mean = (size - 1) / 2
        kernel *= 1 / (std * math.sqrt(2 * math.pi)) * \
                  torch.exp(-((mgrid - mean) / std) ** 2 / 2)

    kernel = kernel / torch.sum(kernel)    
    return kernel

#TODO: could be modified to generate kernels in different dimensions
def get_box_kernel(kernel_size: int = 5, dim=2):
    if isinstance(kernel_size, numbers.Number):
        kernel_size = [kernel_size] * dim

    kx: float=  float(kernel_size[0])
    ky: float=  float(kernel_size[1])
    box_kernel = torch.Tensor(np.ones((kx, ky)) / (kx*ky))

    return box_kernel



#TODO: Can change HFEN to use either LoG, DoG or XDoG
def get_log_kernel_5x5():
    '''
    This is a precomputed LoG kernel that has already been convolved with
    Gaussian, for edge detection. 
    
    http://fourier.eng.hmc.edu/e161/lectures/gradient/node8.html
    http://homepages.inf.ed.ac.uk/rbf/HIPR2/log.htm
    https://academic.mu.edu/phys/matthysd/web226/Lab02.htm
    The 2-D LoG can be approximated by a 5 by 5 convolution kernel such as:
    weight_log = torch.Tensor([
                    [0, 0, 1, 0, 0],
                    [0, 1, 2, 1, 0],
                    [1, 2, -16, 2, 1],
                    [0, 1, 2, 1, 0],
                    [0, 0, 1, 0, 0]
                ])
    This is an approximate to the LoG kernel with kernel size 5 and optimal 
    sigma ~6 (0.590155...).
    '''
    return torch.Tensor([
                [0, 0, 1, 0, 0],
                [0, 1, 2, 1, 0],
                [1, 2, -16, 2, 1],
                [0, 1, 2, 1, 0],
                [0, 0, 1, 0, 0]
            ])

#dim is the image dimension (2D=2, 3D=3, etc), but for now the final_kernel is hardcoded to 2D images
#Not sure if it would make sense in higher dimensions
#Note: Kornia suggests their laplacian kernel can also be used to generate LoG kernel: 
# https://torchgeometry.readthedocs.io/en/latest/_modules/kornia/filters/laplacian.html
def get_log_kernel2d(kernel_size=5, sigma=None, dim=2): #sigma=0.6; kernel_size=5
    
    #either kernel_size or sigma are required:
    if not kernel_size and sigma:
        kernel_size = get_kernel_size(sigma)
        kernel_size = [kernel_size] * dim #note: should it be [kernel_size] or [kernel_size-1]? look below 
    elif kernel_size and not sigma:
        sigma = get_kernel_sigma(kernel_size)
        sigma = [sigma] * dim

    if isinstance(kernel_size, numbers.Number):
        kernel_size = [kernel_size-1] * dim
    if isinstance(sigma, numbers.Number):
        sigma = [sigma] * dim

    grids = torch.meshgrid([torch.arange(-size//2,size//2+1,1) for size in kernel_size])

    kernel = 1
    for size, std, mgrid in zip(kernel_size, sigma, grids):
        kernel *= torch.exp(-(mgrid**2/(2.*std**2)))
    
    #TODO: For now hardcoded to 2 dimensions, test to make it work in any dimension
    final_kernel = (kernel) * ((grids[0]**2 + grids[1]**2) - (2*sigma[0]*sigma[1])) * (1/((2*math.pi)*(sigma[0]**2)*(sigma[1]**2)))
    
    #TODO: Test if normalization has to be negative (the inverted peak should not make a difference)
    final_kernel = -final_kernel / torch.sum(final_kernel)
    
    return final_kernel

def get_log_kernel(kernel_size: int = 5, sigma: float = None, dim: int = 2):
    '''
        Returns a Laplacian of Gaussian (LoG) kernel. If the kernel is known, use it,
        else, generate a kernel with the parameters provided (slower)
    '''
    if kernel_size ==5 and not sigma and dim == 2: 
        return get_log_kernel_5x5()
    else:
        return get_log_kernel2d(kernel_size, sigma, dim)

#TODO: use
# Implementation of binarize operation (for edge detectors)
def binarize(bin_img, threshold):
  #bin_img = img > threshold
  bin_img[bin_img < threshold] = 0.
  return bin_img




def get_laplacian_kernel_3x3(alt=False) -> torch.Tensor:
    """
        Utility function that returns a laplacian kernel of 3x3
            https://academic.mu.edu/phys/matthysd/web226/Lab02.htm
            http://homepages.inf.ed.ac.uk/rbf/HIPR2/log.htm
        
        This is called a negative Laplacian because the central peak is negative. 
        It is just as appropriate to reverse the signs of the elements, using 
        -1s and a +4, to get a positive Laplacian. It doesn't matter:

        laplacian_kernel = torch.Tensor([
                                    [0,  -1, 0],
                                    [-1, 4, -1],
                                    [0,  -1, 0]
                                ])

        Alternative Laplacian kernel as produced by Kornia (this is positive Laplacian,
        like: https://kornia.readthedocs.io/en/latest/filters.html
        laplacian_kernel = torch.Tensor([
                                    [-1, -1, -1],
                                    [-1,  8, -1],
                                    [-1, -1, -1]
                                ])

    """
    if alt:
        return torch.tensor([
                    [-1, -1, -1],
                    [-1,  8, -1],
                    [-1, -1, -1]
                ])
    else:
        return torch.tensor([
                    [0, 1, 0],
                    [1,-4, 1],
                    [0, 1, 0],
                ])

def get_gradient_kernel_3x3() -> torch.Tensor:
    """
        Utility function that returns a gradient kernel of 3x3
            in x direction (transpose for y direction)
            kernel_gradient_v = [[0, -1, 0], 
                                 [0, 0, 0], 
                                 [0, 1, 0]]
            kernel_gradient_h = [[0, 0, 0], 
                                 [-1, 0, 1], 
                                 [0, 0, 0]]
    """
    return torch.tensor([
                   [0, 0, 0], 
                   [-1, 0, 1], 
                   [0, 0, 0],
            ])

def get_scharr_kernel_3x3() -> torch.Tensor:
    """
        Utility function that returns a scharr kernel of 3x3
            in x direction (transpose for y direction)
    """
    return torch.tensor([
                   [-3, 0, 3],
                   [-10,0,10],
                   [-3, 0, 3],
    ])

def get_prewitt_kernel_3x3() -> torch.Tensor:
    """
        Utility function that returns a prewitt kernel of 3x3
            in x direction (transpose for y direction).
        
        Prewitt in x direction: This mask is called the 
            (vertical) Prewitt Edge Detector
            prewitt_x= np.array([[-1, 0, 1],
                                [-1, 0, 1],
                                [-1, 0, 1]])
        
        Prewitt in y direction: This mask is called the 
            (horizontal) Prewitt Edge Detector
            prewitt_y= np.array([[-1,-1,-1],
                                 [0, 0, 0],
                                 [1, 1, 1]])

        Note that a Prewitt operator is a 1D box filter convolved with 
            a derivative operator 
            finite_diff = [-1, 0, 1]
            simple_box = [1, 1, 1]

    """
    return torch.tensor([
                   [-1, 0, 1],
                   [-1, 0, 1],
                   [-1, 0, 1],
    ])

#https://github.com/kornia/kornia/blob/master/kornia/filters/kernels.py
def get_sobel_kernel_3x3() -> torch.Tensor:
    """Utility function that returns a sobel kernel of 3x3
        sobel in x direction
            sobel_x= np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]])
        sobel in y direction
            sobel_y= np.array([[-1,-2,-1],
                               [0, 0, 0],
                               [1, 2, 1]])
        
        Note that a Sobel operator is a [1 2 1] filter convolved with 
            a derivative operator.
            finite_diff = [1, 2, 1]
            simple_box = [1, 1, 1]
    """
    return torch.tensor([
        [-1., 0., 1.],
        [-2., 0., 2.],
        [-1., 0., 1.],
    ])

#https://towardsdatascience.com/implement-canny-edge-detection-from-scratch-with-pytorch-a1cccfa58bed
def get_sobel_kernel_2d(kernel_size=3):
    # get range
    range = torch.linspace(-(kernel_size // 2), kernel_size // 2, kernel_size)
    # compute a grid the numerator and the axis-distances
    y, x = torch.meshgrid(range, range)
    #Note: x is edge detector in x, y is edge detector in y, if not dividing by den
    den = (x ** 2 + y ** 2)
    #den[:, kernel_size // 2] = 1  # avoid division by zero at the center of den
    den[kernel_size // 2, kernel_size // 2] = 1  # avoid division by zero at the center of den
    #sobel_2D = x / den #produces kernel in range (0,1)
    sobel_2D = 2*x / den #produces same kernel as kornia
    return sobel_2D

def get_sobel_kernel(kernel_size=3):
    '''
    Sobel kernel
        https://en.wikipedia.org/wiki/Sobel_operator
    Note: using the Sobel filters needs two kernels, one in X axis and one in Y 
        axis (which is the transpose of X), to get the gradients in both directions.
        The same kernel can be used in both cases.
    '''
    if kernel_size==3:
        return get_sobel_kernel_3x3()
    else:
        return get_sobel_kernel_2d(kernel_size)



#To apply the 1D filter in X and Y axis (For SSIM)
#@torch.jit.script
def apply_1Dfilter(input, win, use_padding: bool=False):  
    r""" Apply 1-D kernel to input in X and Y axes.
         Separable filters like the Gaussian blur can be applied to 
         a two-dimensional image as two independent one-dimensional 
         calculations, so a 2-dimensional convolution operation can 
         be separated into two 1-dimensional filters. This reduces 
         the cost of computing the operator.
           https://en.wikipedia.org/wiki/Separable_filter
    Args:
        input (torch.Tensor): a batch of tensors to be filtered
        window (torch.Tensor): 1-D gauss kernel
        use_padding: padding image before conv
    Returns:
        torch.Tensor: filtered tensors
    """
    #N, C, H, W = input.shape
    C = input.shape[1]
    
    padding = 0
    if use_padding:
        window_size = win.shape[3]
        padding = window_size // 2

    #same 1D filter for both axes    
    out = F.conv2d(input, win, stride=1, padding=(0, padding), groups=C)
    out = F.conv2d(out, win.transpose(2, 3), stride=1, padding=(padding, 0), groups=C)
    return out

#convenient alias
apply_gaussian_filter = apply_1Dfilter



#TODO: use this in the initialization of class FilterX, so it can be used on 
# forward with an image (LoG, Gaussian, etc)
def load_filter(kernel, kernel_size=3, in_channels=3, out_channels=3, 
                stride=1, padding=True, groups=3, dim: int =2, 
                requires_grad=False):
    '''
        Loads a kernel's coefficients into a Conv layer that 
            can be used to convolve an image with, by default, 
            for depthwise convolution
        Can use nn.Conv1d, nn.Conv2d or nn.Conv3d, depending on
            the dimension set in dim (1,2,3)
        #From Pytorch Conv2D:
            https://pytorch.org/docs/master/_modules/torch/nn/modules/conv.html#Conv2d
            When `groups == in_channels` and `out_channels == K * in_channels`,
            where `K` is a positive integer, this operation is also termed in
            literature as depthwise convolution.
             At groups= :attr:`in_channels`, each input channel is convolved with
             its own set of filters, of size:
             :math:`\left\lfloor\frac{out\_channels}{in\_channels}\right\rfloor`.
    '''

    '''#TODO: check if this is necessary, probably not
    if isinstance(kernel_size, numbers.Number):
        kernel_size = [kernel_size] * dim
    '''

    # Reshape to 2d depthwise convolutional weight
    kernel = kernel_conv_w(kernel, in_channels)
    assert(len(kernel.shape)==4 and kernel.shape[0]==in_channels)

    if padding:
        pad = compute_padding(kernel_size)
    else:
        pad = 0
    
    # create filter as convolutional layer
    if dim == 1:
        conv = nn.Conv1d
    elif dim == 2:
        conv = nn.Conv2d
    elif dim == 3:
        conv = nn.Conv3d
    else:
        raise RuntimeError(
            'Only 1, 2 and 3 dimensions are supported for convolution. \
            Received {}.'.format(dim)
        )

    filter = conv(in_channels=in_channels, out_channels=out_channels,
                        kernel_size=kernel_size, stride=stride, padding=padding, 
                        groups=groups, bias=False)
    filter.weight.data = kernel
    filter.weight.requires_grad = requires_grad
    return filter


def compute_padding(kernel_size):
    '''
        Computes padding tuple. For square kernels, pad can be an
         int, else, a tuple with an element for each dimension
    '''
    # 4 or 6 ints:  (padding_left, padding_right, padding_top, padding_bottom)
    if isinstance(kernel_size, int):
        return kernel_size//2
    elif isinstance(kernel_size, list):
        computed = [k // 2 for k in kernel_size]

        out_padding = []

        for i in range(len(kernel_size)):
            computed_tmp = computed[-(i + 1)]
            # for even kernels we need to do asymetric padding
            if kernel_size[i] % 2 == 0:
                padding = computed_tmp - 1
            else:
                padding = computed_tmp
            out_padding.append(padding)
            out_padding.append(computed_tmp)
        return out_padding

def normalize_kernel2d(input: torch.Tensor) -> torch.Tensor:
    r"""Normalizes kernel.
    """
    if len(input.size()) < 2:
        raise TypeError("input should be at least 2D tensor. Got {}"
                        .format(input.size()))
    norm: torch.Tensor = input.abs().sum(dim=-1).sum(dim=-1)
    return input / (norm.unsqueeze(-1).unsqueeze(-1))

def filter2D(input: torch.Tensor, kernel: torch.Tensor,
             border_type: str = 'reflect', 
             dim: int =2,
             normalized: bool = False) -> torch.Tensor:
    r"""Function that convolves a tensor with a kernel.

    The function applies a given kernel to a tensor. The kernel is applied
    independently at each depth channel of the tensor. Before applying the
    kernel, the function applies padding according to the specified mode so
    that the output remains in the same shape.
    Args:
        input (torch.Tensor): the input tensor with shape of
          :math:`(B, C, H, W)`.
        kernel (torch.Tensor): the kernel to be convolved with the input
          tensor. The kernel shape must be :math:`(1, kH, kW)`.
        border_type (str): the padding mode to be applied before convolving.
          The expected modes are: ``'constant'``, ``'reflect'``,
          ``'replicate'`` or ``'circular'``. Default: ``'reflect'``.
        normalized (bool): If True, kernel will be L1 normalized.
    Return:
        torch.Tensor: the convolved tensor of same size and numbers of channels
        as the input.
    """
    if not isinstance(input, torch.Tensor):
        raise TypeError("Input type is not a torch.Tensor. Got {}"
                        .format(type(input)))

    if not isinstance(kernel, torch.Tensor):
        raise TypeError("Input kernel type is not a torch.Tensor. Got {}"
                        .format(type(kernel)))

    if not isinstance(border_type, str):
        raise TypeError("Input border_type is not string. Got {}"
                        .format(type(kernel)))

    #if not len(input.shape) == 4:
        #raise ValueError("Invalid input shape, we expect BxCxHxW. Got: {}"
                         #.format(input.shape))

    #if not len(kernel.shape) == 3:
        #raise ValueError("Invalid kernel shape, we expect 1xHxW. Got: {}"
                         #.format(kernel.shape))

    borders_list: List[str] = ['constant', 'reflect', 'replicate', 'circular']
    if border_type not in borders_list:
        raise ValueError("Invalid border_type, we expect the following: {0}."
                         "Got: {1}".format(borders_list, border_type))

    # prepare kernel
    b, c, h, w = input.shape
    tmp_kernel: torch.Tensor = kernel.unsqueeze(0).to(input.device).to(input.dtype)
    if normalized:
        tmp_kernel = normalize_kernel2d(tmp_kernel) 
    # pad the input tensor
    height, width = tmp_kernel.shape[-2:]
    padding_shape: List[int] = compute_padding((height, width))
    input_pad: torch.Tensor = F.pad(input, padding_shape, mode=border_type)
    b, c, hp, wp = input_pad.shape

    tmp_kernel = tmp_kernel.expand(c, -1, -1, -1)

    # convolve the tensor with the kernel.
    if dim == 1:
        conv = F.conv1d
    elif dim == 2:
        conv = F.conv2d
        #TODO: this needs a review, the final sizes don't match with .view(b, c, h, w), (they are larger).
            # using .view(b, c, -1, w) results in an output, but it's 3 times larger than it should be
        '''
        # if kernel_numel > 81 this is a faster algo
        kernel_numel: int = height * width #kernel_numel = torch.numel(tmp_kernel[-1:])
        if kernel_numel > 81:
            return conv(input_pad.reshape(b * c, 1, hp, wp), tmp_kernel, padding=0, stride=1).view(b, c, h, w)
        '''
    elif dim == 3:
        conv = F.conv3d
    else:
        raise RuntimeError(
            'Only 1, 2 and 3 dimensions are supported. Received {}.'.format(dim)
        )

    return conv(input_pad, tmp_kernel, groups=c, padding=0, stride=1)


#TODO: make one class to receive any arbitrary kernel and others that are specific (like gaussian, etc)
#class FilterX(nn.Module):
  #def __init__(self, ..., kernel_type, dim: int=2):
      #r"""
      #Args:
          #argument: ...
      #"""
      #super(filterXd, self).__init__()
      #Here receive an pre-made kernel of any type, load as tensor or as
      #convXd layer (class or functional)
      # self.filter = load_filter(kernel=kernel, kernel_size=kernel_size, 
                #in_channels=image_channels, out_channels=image_channels, stride=stride, 
                #padding=pad, groups=image_channels)
  #def forward:
      #This would apply the filter that was initialized
    


class FilterLow(nn.Module):
    def __init__(self, recursions=1, kernel_size=9, stride=1, padding=True, 
                image_channels=3, include_pad=True, filter_type=None):
        super(FilterLow, self).__init__()
        
        if padding:
            pad = compute_padding(kernel_size)
        else:
            pad = 0
        
        if filter_type == 'gaussian':
            sigma = get_kernel_sigma(kernel_size)
            kernel = get_gaussian_kernel2d(kernel_size=kernel_size, sigma=sigma)
            self.filter = load_filter(kernel=kernel, kernel_size=kernel_size, 
                    in_channels=image_channels, stride=stride, padding=pad)
        #elif filter_type == '': #TODO... box? (the same as average) What else?
        else:
            self.filter = nn.AvgPool2d(kernel_size=kernel_size, stride=stride, 
                    padding=pad, count_include_pad=include_pad)
        self.recursions = recursions

    def forward(self, img):
        for i in range(self.recursions):
            img = self.filter(img)
        return img


class FilterHigh(nn.Module):
    def __init__(self, recursions=1, kernel_size=9, stride=1, include_pad=True, 
            image_channels=3, normalize=True, filter_type=None, kernel=None):
        super(FilterHigh, self).__init__()
        
        # if is standard freq. separator, will use the same LPF to remove LF from image
        if filter_type=='gaussian' or filter_type=='average':
            self.type = 'separator'
            self.filter_low = FilterLow(recursions=1, kernel_size=kernel_size, stride=stride, 
                image_channels=image_channels, include_pad=include_pad, filter_type=filter_type)
        # otherwise, can use any independent filter
        else: #load any other filter for the high pass
            self.type = 'independent'
            #kernel and kernel_size should be provided. Options for edge detectors:
            # In both dimensions: get_log_kernel, get_laplacian_kernel_3x3 
            # and get_sobel_kernel
            # Single dimension: get_prewitt_kernel_3x3, get_scharr_kernel_3x3 
            # get_gradient_kernel_3x3 
            if include_pad:
                pad = compute_padding(kernel_size)
            else:
                pad = 0
            self.filter_low = load_filter(kernel=kernel, kernel_size=kernel_size, 
                in_channels=image_channels, out_channels=image_channels, stride=stride, 
                padding=pad, groups=image_channels)
        self.recursions = recursions
        self.normalize = normalize

    def forward(self, img):
        if self.type == 'separator':
            if self.recursions > 1:
                for i in range(self.recursions - 1):
                    img = self.filter_low(img)
            img = img - self.filter_low(img)
        elif self.type == 'independent':
            img = self.filter_low(img)
        if self.normalize:
            return denorm(img)
        else:
            return img

#TODO: check how similar getting the gradient with get_gradient_kernel_3x3 is from the alternative displacing the image
#ref from TF: https://github.com/tensorflow/tensorflow/blob/4386a6640c9fb65503750c37714971031f3dc1fd/tensorflow/python/ops/image_ops_impl.py#L3423
def get_image_gradients(image):
    """Returns image gradients (dy, dx) for each color channel.
    Both output tensors have the same shape as the input: [b, c, h, w]. 
    Places the gradient [I(x+1,y) - I(x,y)] on the base pixel (x, y). 
    That means that dy will always have zeros in the last row,
    and dx will always have zeros in the last column.

    This can be used to implement the anisotropic 2-D version of the 
    Total Variation formula:
        https://en.wikipedia.org/wiki/Total_variation_denoising
    (anisotropic is using l1, isotropic is using l2 norm)
    
    Arguments:
        image: Tensor with shape [b, c, h, w].
    Returns:
        Pair of tensors (dy, dx) holding the vertical and horizontal image
        gradients (1-step finite difference).  
    Raises:
      ValueError: If `image` is not a 3D image or 4D tensor.
    """
    
    image_shape = image.shape
      
    if len(image_shape) == 3:
        # The input is a single image with shape [height, width, channels].
        # Calculate the difference of neighboring pixel-values.
        # The images are shifted one pixel along the height and width by slicing.
        dx = image[:, 1:, :] - image[:, :-1, :] #pixel_dif2, f_v_1-f_v_2
        dy = image[1:, :, :] - image[:-1, :, :] #pixel_dif1, f_h_1-f_h_2

    elif len(image_shape) == 4:    
        # Return tensors with same size as original image
        #adds one pixel pad to the right and removes one pixel from the left
        right = F.pad(image, [0, 1, 0, 0])[..., :, 1:]
        #adds one pixel pad to the bottom and removes one pixel from the top
        bottom = F.pad(image, [0, 0, 0, 1])[..., 1:, :] 

        #right and bottom have the same dimensions as image
        dx, dy = right - image, bottom - image 
        
        #this is required because otherwise results in the last column and row having 
        # the original pixels from the image
        dx[:, :, :, -1] = 0 # dx will always have zeros in the last column, right-left
        dy[:, :, -1, :] = 0 # dy will always have zeros in the last row,    bottom-top
    else:
      raise ValueError(
          'image_gradients expects a 3D [h, w, c] or 4D tensor '
          '[batch_size, c, h, w], not %s.', image_shape)

    return dy, dx


def get_4dim_image_gradients(image):
    # Return tensors with same size as original image
    # Place the gradient [I(x+1,y) - I(x,y)] on the base pixel (x, y).
    right = F.pad(image, [0, 1, 0, 0])[..., :, 1:] #adds one pixel pad to the right and removes one pixel from the left
    bottom = F.pad(image, [0, 0, 0, 1])[..., 1:, :] #adds one pixel pad to the bottom and removes one pixel from the top
    botright = F.pad(image, [0, 1, 0, 1])[..., 1:, 1:] #displaces in diagonal direction

    dx, dy = right - image, bottom - image #right and bottom have the same dimensions as image
    dn, dp = botright - image, right - bottom
    #dp is positive diagonal (bottom left to top right)
    #dn is negative diagonal (top left to bottom right)
    
    #this is required because otherwise results in the last column and row having 
    # the original pixels from the image
    dx[:, :, :, -1] = 0 # dx will always have zeros in the last column, right-left
    dy[:, :, -1, :] = 0 # dy will always have zeros in the last row,    bottom-top
    dp[:, :, -1, :] = 0 # dp will always have zeros in the last row

    return dy, dx, dp, dn

#TODO: #https://towardsdatascience.com/implement-canny-edge-detection-from-scratch-with-pytorch-a1cccfa58bed
#TODO: https://link.springer.com/article/10.1007/s11220-020-00281-8
def grad_orientation(grad_y, grad_x):
    go = torch.atan(grad_y / grad_x)
    go = go * (360 / np.pi) + 180 # convert to degree
    go = torch.round(go / 45) * 45  # keep a split by 45
    return go


In [ ]:
#@title vic/colors.py
%%writefile /content/Colab-edge-connect/src/vic/colors.py
"""
BasicSR/codes/dataops/colors.py (8-Nov-20)
https://github.com/victorca25/BasicSR/blob/dev2/codes/dataops/colors.py
"""


'''
Functions for color operations on tensors.
If needed, there are more conversions that can be used:
https://github.com/kornia/kornia/tree/master/kornia/color
https://github.com/R08UST/Color_Conversion_pytorch/blob/master/differentiable_color_conversion/basic_op.py
'''


import torch
import math
import cv2

def bgr_to_rgb(image: torch.Tensor) -> torch.Tensor:
    # flip image channels
    out: torch.Tensor = image.flip(-3) #https://github.com/pytorch/pytorch/issues/229
    #out: torch.Tensor = image[[2, 1, 0], :, :] #RGB to BGR #may be faster
    return out

def rgb_to_bgr(image: torch.Tensor) -> torch.Tensor:
    #same operation as bgr_to_rgb(), flip image channels
    return bgr_to_rgb(image)

def bgra_to_rgba(image: torch.Tensor) -> torch.Tensor:
    out: torch.Tensor = image[[2, 1, 0, 3], :, :]
    return out

def rgba_to_bgra(image: torch.Tensor) -> torch.Tensor:
    #same operation as bgra_to_rgba(), flip image channels
    return bgra_to_rgba(image)

def rgb_to_grayscale(input: torch.Tensor) -> torch.Tensor:
    r, g, b = torch.chunk(input, chunks=3, dim=-3)
    gray: torch.Tensor = 0.299 * r + 0.587 * g + 0.114 * b
    #gray = rgb_to_yuv(input,consts='y')
    return gray

def bgr_to_grayscale(input: torch.Tensor) -> torch.Tensor:
    input_rgb = bgr_to_rgb(input)
    gray: torch.Tensor = rgb_to_grayscale(input_rgb)
    #gray = rgb_to_yuv(input_rgb,consts='y')
    return gray

def grayscale_to_rgb(input: torch.Tensor) -> torch.Tensor:
    #repeat the gray image to the three channels
    rgb: torch.Tensor = input.repeat(3, *[1] * (input.dim() - 1))
    return rgb

def grayscale_to_bgr(input: torch.Tensor) -> torch.Tensor:
    return grayscale_to_rgb(input)

def rgb_to_ycbcr(input: torch.Tensor, consts='yuv'):
    return rgb_to_yuv(input, consts == 'ycbcr')

def rgb_to_yuv(input: torch.Tensor, consts='yuv'):
    """Converts one or more images from RGB to YUV.
    Outputs a tensor of the same shape as the `input` image tensor, containing the YUV
    value of the pixels.
    The output is only well defined if the value in images are in [0,1].
    Y′CbCr is often confused with the YUV color space, and typically the terms YCbCr 
    and YUV are used interchangeably, leading to some confusion. The main difference 
    is that YUV is analog and YCbCr is digital: https://en.wikipedia.org/wiki/YCbCr
    Args:
      input: 2-D or higher rank. Image data to convert. Last dimension must be
        size 3. (Could add additional channels, ie, AlphaRGB = AlphaYUV)
      consts: YUV constant parameters to use. BT.601 or BT.709. Could add YCbCr
        https://en.wikipedia.org/wiki/YUV
    Returns:
      images: images tensor with the same shape as `input`.
    """
    
    #channels = input.shape[0]
    
    if consts == 'BT.709': # HDTV YUV
        Wr = 0.2126
        Wb = 0.0722
        Wg = 1 - Wr - Wb #0.7152
        Uc = 0.539
        Vc = 0.635
        delta: float = 0.5 #128 if image range in [0,255]
    elif consts == 'ycbcr': # Alt. BT.601 from Kornia YCbCr values, from JPEG conversion
        Wr = 0.299
        Wb = 0.114
        Wg = 1 - Wr - Wb #0.587
        Uc = 0.564 #(b-y) #cb
        Vc = 0.713 #(r-y) #cr
        delta: float = .5 #128 if image range in [0,255]
    elif consts == 'yuvK': # Alt. yuv from Kornia YUV values: https://github.com/kornia/kornia/blob/master/kornia/color/yuv.py
        Wr = 0.299
        Wb = 0.114
        Wg = 1 - Wr - Wb #0.587
        Ur = -0.147
        Ug = -0.289
        Ub = 0.436
        Vr = 0.615
        Vg = -0.515
        Vb = -0.100
        #delta: float = 0.0
    elif consts == 'y': #returns only Y channel, same as rgb_to_grayscale()
        #Note: torchvision uses ITU-R 601-2: Wr = 0.2989, Wg = 0.5870, Wb = 0.1140
        Wr = 0.299
        Wb = 0.114
        Wg = 1 - Wr - Wb #0.587
    else: # Default to 'BT.601', SDTV YUV
        Wr = 0.299
        Wb = 0.114
        Wg = 1 - Wr - Wb #0.587
        Uc = 0.493 #0.492
        Vc = 0.877
        delta: float = 0.5 #128 if image range in [0,255]

    r: torch.Tensor = input[..., 0, :, :]
    g: torch.Tensor = input[..., 1, :, :]
    b: torch.Tensor = input[..., 2, :, :]
    #TODO
    #r, g, b = torch.chunk(input, chunks=3, dim=-3) #Alt. Which one is faster? Appear to be the same. Differentiable? Kornia uses both in different places

    if consts == 'y':
        y: torch.Tensor = Wr * r + Wg * g + Wb * b
        #(0.2989 * input[0] + 0.5870 * input[1] + 0.1140 * input[2]).to(img.dtype)
        return y
    elif consts == 'yuvK':
        y: torch.Tensor = Wr * r + Wg * g + Wb * b
        u: torch.Tensor = Ur * r + Ug * g + Ub * b
        v: torch.Tensor = Vr * r + Vg * g + Vb * b
    else: #if consts == 'ycbcr' or consts == 'yuv' or consts == 'BT.709':
        y: torch.Tensor = Wr * r + Wg * g + Wb * b
        u: torch.Tensor = (b - y) * Uc + delta #cb
        v: torch.Tensor = (r - y) * Vc + delta #cr

    if consts == 'uv': #returns only UV channels
        return torch.stack((u, v), -3)
    else:
        return torch.stack((y, u, v), -3)

def ycbcr_to_rgb(input: torch.Tensor):
    return yuv_to_rgb(input, consts = 'ycbcr')

def yuv_to_rgb(input: torch.Tensor, consts='yuv') -> torch.Tensor:
    if consts == 'yuvK': # Alt. yuv from Kornia YUV values: https://github.com/kornia/kornia/blob/master/kornia/color/yuv.py
        Wr = 1.14 #1.402
        Wb = 2.029 #1.772
        Wgu = 0.396 #.344136
        Wgv = 0.581 #.714136
        delta: float = 0.0
    elif consts == 'yuv' or consts == 'ycbcr': # BT.601 from Kornia YCbCr values, from JPEG conversion
        Wr = 1.403 #1.402
        Wb = 1.773 #1.772
        Wgu = .344 #.344136
        Wgv = .714 #.714136
        delta: float = .5 #128 if image range in [0,255]
    
    #Note: https://github.com/R08UST/Color_Conversion_pytorch/blob/75150c5fbfb283ae3adb85c565aab729105bbb66/differentiable_color_conversion/basic_op.py#L65 has u and v flipped
    y: torch.Tensor = input[..., 0, :, :]
    u: torch.Tensor = input[..., 1, :, :] #cb
    v: torch.Tensor = input[..., 2, :, :] #cr
    #TODO
    #y, u, v = torch.chunk(input, chunks=3, dim=-3) #Alt. Which one is faster? Appear to be the same. Differentiable? Kornia uses both in different places

    u_shifted: torch.Tensor = u - delta #cb
    v_shifted: torch.Tensor = v - delta #cr

    r: torch.Tensor = y + Wr * v_shifted
    g: torch.Tensor = y - Wgv * v_shifted - Wgu * u_shifted
    b: torch.Tensor = y + Wb * u_shifted
    return torch.stack((r, g, b), -3) 

#Not tested:
def rgb2srgb(imgs):
    return torch.where(imgs<=0.04045,imgs/12.92,torch.pow((imgs+0.055)/1.055,2.4))

#Not tested:
def srgb2rgb(imgs):
    return torch.where(imgs<=0.0031308,imgs*12.92,1.055*torch.pow((imgs),1/2.4)-0.055)



In [ ]:
#@title vic/discriminators.py
%%writefile /content/Colab-edge-connect/src/vic/discriminators.py

"""
BasicSR/codes/models/modules/architectures/discriminators.py (8-Nov-20)
https://github.com/victorca25/BasicSR/blob/dev2/codes/models/modules/architectures/discriminators.py
"""

import math
import torch
import torch.nn as nn
import torchvision
from . import block as B
from . import spectral_norm as SN


####################
# Discriminator
####################


# VGG style Discriminator
class Discriminator_VGG(nn.Module):
    def __init__(self, size, in_nc, base_nf, norm_type='batch', act_type='leakyrelu', mode='CNA', convtype='Conv2D', arch='ESRGAN'):
        super(Discriminator_VGG, self).__init__()

        conv_blocks = []
        conv_blocks.append(B.conv_block(  in_nc, base_nf, kernel_size=3, stride=1, norm_type=None, \
            act_type=act_type, mode=mode))
        conv_blocks.append(B.conv_block(base_nf, base_nf, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode))

        cur_size = size // 2
        cur_nc = base_nf
        while cur_size > 4:
            out_nc = cur_nc * 2 if cur_nc < 512 else cur_nc
            conv_blocks.append(B.conv_block(cur_nc, out_nc, kernel_size=3, stride=1, norm_type=norm_type, \
                act_type=act_type, mode=mode))
            conv_blocks.append(B.conv_block(out_nc, out_nc, kernel_size=4, stride=2, norm_type=norm_type, \
                act_type=act_type, mode=mode))
            cur_nc = out_nc
            cur_size //= 2

        self.features = B.sequential(*conv_blocks)

        # classifier
        if arch=='PPON':
            self.classifier = nn.Sequential(
                nn.Linear(cur_nc * cur_size * cur_size, 128), nn.LeakyReLU(0.2, True), nn.Linear(128, 1))
        else: #arch='ESRGAN':
            self.classifier = nn.Sequential(
                nn.Linear(cur_nc * cur_size * cur_size, 100), nn.LeakyReLU(0.2, True), nn.Linear(100, 1))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# VGG style Discriminator with input size 96*96
class Discriminator_VGG_96(nn.Module):
    def __init__(self, in_nc, base_nf, norm_type='batch', act_type='leakyrelu', mode='CNA', convtype='Conv2D', arch='ESRGAN'):
        super(Discriminator_VGG_96, self).__init__()
        # features
        # hxw, c
        # 96, 64
        conv0 = B.conv_block(in_nc, base_nf, kernel_size=3, norm_type=None, act_type=act_type, \
            mode=mode)
        conv1 = B.conv_block(base_nf, base_nf, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 48, 64
        conv2 = B.conv_block(base_nf, base_nf*2, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv3 = B.conv_block(base_nf*2, base_nf*2, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 24, 128
        conv4 = B.conv_block(base_nf*2, base_nf*4, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv5 = B.conv_block(base_nf*4, base_nf*4, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 12, 256
        conv6 = B.conv_block(base_nf*4, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv7 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 6, 512
        conv8 = B.conv_block(base_nf*8, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv9 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 3, 512
        self.features = B.sequential(conv0, conv1, conv2, conv3, conv4, conv5, conv6, conv7, conv8,\
            conv9)

        # classifier
        if arch=='PPON':
            self.classifier = nn.Sequential(
                nn.Linear(512 * 3 * 3, 128), nn.LeakyReLU(0.2, True), nn.Linear(128, 1))
        else: #arch='ESRGAN':
            self.classifier = nn.Sequential(
                nn.Linear(512 * 3 * 3, 100), nn.LeakyReLU(0.2, True), nn.Linear(100, 1))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


# VGG style Discriminator with input size 128*128, Spectral Normalization
class Discriminator_VGG_128_SN(nn.Module):
    def __init__(self):
        super(Discriminator_VGG_128_SN, self).__init__()
        # features
        # hxw, c
        # 128, 64
        self.lrelu = nn.LeakyReLU(0.2, True)

        self.conv0 = SN.spectral_norm(nn.Conv2d(3, 64, 3, 1, 1))
        self.conv1 = SN.spectral_norm(nn.Conv2d(64, 64, 4, 2, 1))
        # 64, 64
        self.conv2 = SN.spectral_norm(nn.Conv2d(64, 128, 3, 1, 1))
        self.conv3 = SN.spectral_norm(nn.Conv2d(128, 128, 4, 2, 1))
        # 32, 128
        self.conv4 = SN.spectral_norm(nn.Conv2d(128, 256, 3, 1, 1))
        self.conv5 = SN.spectral_norm(nn.Conv2d(256, 256, 4, 2, 1))
        # 16, 256
        self.conv6 = SN.spectral_norm(nn.Conv2d(256, 512, 3, 1, 1))
        self.conv7 = SN.spectral_norm(nn.Conv2d(512, 512, 4, 2, 1))
        # 8, 512
        self.conv8 = SN.spectral_norm(nn.Conv2d(512, 512, 3, 1, 1))
        self.conv9 = SN.spectral_norm(nn.Conv2d(512, 512, 4, 2, 1))
        # 4, 512

        # classifier
        self.linear0 = SN.spectral_norm(nn.Linear(512 * 4 * 4, 100))
        self.linear1 = SN.spectral_norm(nn.Linear(100, 1))

    def forward(self, x):
        x = self.lrelu(self.conv0(x))
        x = self.lrelu(self.conv1(x))
        x = self.lrelu(self.conv2(x))
        x = self.lrelu(self.conv3(x))
        x = self.lrelu(self.conv4(x))
        x = self.lrelu(self.conv5(x))
        x = self.lrelu(self.conv6(x))
        x = self.lrelu(self.conv7(x))
        x = self.lrelu(self.conv8(x))
        x = self.lrelu(self.conv9(x))
        x = x.view(x.size(0), -1)
        x = self.lrelu(self.linear0(x))
        x = self.linear1(x)
        return x

# VGG style Discriminator with input size 128*128
class Discriminator_VGG_128(nn.Module):
    def __init__(self, in_nc, base_nf, norm_type='batch', act_type='leakyrelu', mode='CNA', convtype='Conv2D', arch='ESRGAN'):
        super(Discriminator_VGG_128, self).__init__()
        # features
        # hxw, c
        # 128, 64
        conv0 = B.conv_block(in_nc, base_nf, kernel_size=3, norm_type=None, act_type=act_type, \
            mode=mode)
        conv1 = B.conv_block(base_nf, base_nf, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 64, 64
        conv2 = B.conv_block(base_nf, base_nf*2, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv3 = B.conv_block(base_nf*2, base_nf*2, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 32, 128
        conv4 = B.conv_block(base_nf*2, base_nf*4, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv5 = B.conv_block(base_nf*4, base_nf*4, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 16, 256
        conv6 = B.conv_block(base_nf*4, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv7 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 8, 512
        conv8 = B.conv_block(base_nf*8, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv9 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 4, 512
        self.features = B.sequential(conv0, conv1, conv2, conv3, conv4, conv5, conv6, conv7, conv8,\
            conv9)

        # classifier
        if arch=='PPON':
            self.classifier = nn.Sequential(
                nn.Linear(512 * 4 * 4, 128), nn.LeakyReLU(0.2, True), nn.Linear(128, 1))
        else: #arch='ESRGAN':
            self.classifier = nn.Sequential(
                nn.Linear(512 * 4 * 4, 100), nn.LeakyReLU(0.2, True), nn.Linear(100, 1))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


# VGG style Discriminator with input size 192*192
class Discriminator_VGG_192(nn.Module): #vic in PPON is called Discriminator_192 
    def __init__(self, in_nc, base_nf, norm_type='batch', act_type='leakyrelu', mode='CNA', convtype='Conv2D', arch='ESRGAN'):
        super(Discriminator_VGG_192, self).__init__()
        # features
        # hxw, c
        # 192, 64
        conv0 = B.conv_block(in_nc, base_nf, kernel_size=3, norm_type=None, act_type=act_type, \
            mode=mode) # 3-->64
        conv1 = B.conv_block(base_nf, base_nf, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 64-->64, 96*96
        # 96, 64
        conv2 = B.conv_block(base_nf, base_nf*2, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 64-->128
        conv3 = B.conv_block(base_nf*2, base_nf*2, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 128-->128, 48*48
        # 48, 128
        conv4 = B.conv_block(base_nf*2, base_nf*4, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 128-->256
        conv5 = B.conv_block(base_nf*4, base_nf*4, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 256-->256, 24*24
        # 24, 256
        conv6 = B.conv_block(base_nf*4, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 256-->512
        conv7 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 512-->512 12*12
        # 12, 512
        conv8 = B.conv_block(base_nf*8, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 512-->512
        conv9 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 512-->512 6*6
        # 6, 512
        conv10 = B.conv_block(base_nf*8, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv11 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 3*3
        # 3, 512
        self.features = B.sequential(conv0, conv1, conv2, conv3, conv4, conv5, conv6, conv7, conv8,\
            conv9, conv10, conv11)

        # classifier
        if arch=='PPON':
            self.classifier = nn.Sequential(
                nn.Linear(512 * 3 * 3, 128), nn.LeakyReLU(0.2, True), nn.Linear(128, 1)) #vic PPON uses 128 and 128 instead of 100
        else: #arch='ESRGAN':
            self.classifier = nn.Sequential(
                nn.Linear(512 * 3 * 3, 100), nn.LeakyReLU(0.2, True), nn.Linear(100, 1))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# VGG style Discriminator with input size 256*256
class Discriminator_VGG_256(nn.Module):
    def __init__(self, in_nc, base_nf, norm_type='batch', act_type='leakyrelu', mode='CNA', convtype='Conv2D', arch='ESRGAN'):
        super(Discriminator_VGG_256, self).__init__()
        # features
        # hxw, c
        # 256, 64
        conv0 = B.conv_block(in_nc, base_nf, kernel_size=3, norm_type=None, act_type=act_type, \
            mode=mode)
        conv1 = B.conv_block(base_nf, base_nf, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 128, 64
        conv2 = B.conv_block(base_nf, base_nf*2, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv3 = B.conv_block(base_nf*2, base_nf*2, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 64, 128
        conv4 = B.conv_block(base_nf*2, base_nf*4, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv5 = B.conv_block(base_nf*4, base_nf*4, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 32, 256
        conv6 = B.conv_block(base_nf*4, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv7 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 16, 512
        conv8 = B.conv_block(base_nf*8, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv9 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 8, 512
        conv10 = B.conv_block(base_nf*8, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv11 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode) # 3*3
        # 4, 512
        self.features = B.sequential(conv0, conv1, conv2, conv3, conv4, conv5, conv6, conv7, conv8,\
            conv9, conv10, conv11)

        # classifier
        if arch=='PPON':
            self.classifier = nn.Sequential(
                nn.Linear(512 * 4 * 4, 128), nn.LeakyReLU(0.2, True), nn.Linear(128, 1))
        else: #arch='ESRGAN':
            self.classifier = nn.Sequential(
                nn.Linear(512 * 4 * 4, 100), nn.LeakyReLU(0.2, True), nn.Linear(100, 1))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


####################
# Perceptual Network
####################


# Assume input range is [0, 1]
class VGGFeatureExtractor(nn.Module):
    def __init__(self,
                 feature_layer=34,
                 use_bn=False,
                 use_input_norm=True,
                 device=torch.device('cpu'),
                 z_norm=False): #Note: PPON uses cuda instead of CPU
        super(VGGFeatureExtractor, self).__init__()
        if use_bn:
            model = torchvision.models.vgg19_bn(pretrained=True)
        else:
            model = torchvision.models.vgg19(pretrained=True)
        self.use_input_norm = use_input_norm
        if self.use_input_norm:
            if z_norm: # if input in range [-1,1]
                mean = torch.Tensor([0.485-1, 0.456-1, 0.406-1]).view(1, 3, 1, 1).to(device) 
                std = torch.Tensor([0.229*2, 0.224*2, 0.225*2]).view(1, 3, 1, 1).to(device)
            else: # input in range [0,1]
                mean = torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(device)                 
                std = torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(device)
            self.register_buffer('mean', mean)
            self.register_buffer('std', std)
        self.features = nn.Sequential(*list(model.features.children())[:(feature_layer + 1)])
        # No need to BP to variable
        for k, v in self.features.named_parameters():
            v.requires_grad = False

    def forward(self, x):
        if self.use_input_norm:
            x = (x - self.mean) / self.std
        output = self.features(x)
        return output

# Assume input range is [0, 1]
class ResNet101FeatureExtractor(nn.Module):
    def __init__(self, use_input_norm=True, device=torch.device('cpu'), z_norm=False):
        super(ResNet101FeatureExtractor, self).__init__()
        model = torchvision.models.resnet101(pretrained=True)
        self.use_input_norm = use_input_norm
        if self.use_input_norm:
            if z_norm: # if input in range [-1,1]
                mean = torch.Tensor([0.485-1, 0.456-1, 0.406-1]).view(1, 3, 1, 1).to(device)
                std = torch.Tensor([0.229*2, 0.224*2, 0.225*2]).view(1, 3, 1, 1).to(device)
            else: # input in range [0,1]
                mean = torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(device)
                std = torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(device)
            self.register_buffer('mean', mean)
            self.register_buffer('std', std)
        self.features = nn.Sequential(*list(model.children())[:8])
        # No need to BP to variable
        for k, v in self.features.named_parameters():
            v.requires_grad = False

    def forward(self, x):
        if self.use_input_norm:
            x = (x - self.mean) / self.std
        output = self.features(x)
        return output


class MINCNet(nn.Module):
    def __init__(self):
        super(MINCNet, self).__init__()
        self.ReLU = nn.ReLU(True)
        self.conv11 = nn.Conv2d(3, 64, 3, 1, 1)
        self.conv12 = nn.Conv2d(64, 64, 3, 1, 1)
        self.maxpool1 = nn.MaxPool2d(2, stride=2, padding=0, ceil_mode=True)
        self.conv21 = nn.Conv2d(64, 128, 3, 1, 1)
        self.conv22 = nn.Conv2d(128, 128, 3, 1, 1)
        self.maxpool2 = nn.MaxPool2d(2, stride=2, padding=0, ceil_mode=True)
        self.conv31 = nn.Conv2d(128, 256, 3, 1, 1)
        self.conv32 = nn.Conv2d(256, 256, 3, 1, 1)
        self.conv33 = nn.Conv2d(256, 256, 3, 1, 1)
        self.maxpool3 = nn.MaxPool2d(2, stride=2, padding=0, ceil_mode=True)
        self.conv41 = nn.Conv2d(256, 512, 3, 1, 1)
        self.conv42 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv43 = nn.Conv2d(512, 512, 3, 1, 1)
        self.maxpool4 = nn.MaxPool2d(2, stride=2, padding=0, ceil_mode=True)
        self.conv51 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv52 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv53 = nn.Conv2d(512, 512, 3, 1, 1)

    def forward(self, x):
        out = self.ReLU(self.conv11(x))
        out = self.ReLU(self.conv12(out))
        out = self.maxpool1(out)
        out = self.ReLU(self.conv21(out))
        out = self.ReLU(self.conv22(out))
        out = self.maxpool2(out)
        out = self.ReLU(self.conv31(out))
        out = self.ReLU(self.conv32(out))
        out = self.ReLU(self.conv33(out))
        out = self.maxpool3(out)
        out = self.ReLU(self.conv41(out))
        out = self.ReLU(self.conv42(out))
        out = self.ReLU(self.conv43(out))
        out = self.maxpool4(out)
        out = self.ReLU(self.conv51(out))
        out = self.ReLU(self.conv52(out))
        out = self.conv53(out)
        return out


# Assume input range is [0, 1]
class MINCFeatureExtractor(nn.Module):
    def __init__(self, feature_layer=34, use_bn=False, use_input_norm=True, \
                device=torch.device('cpu')):
        super(MINCFeatureExtractor, self).__init__()

        self.features = MINCNet()
        self.features.load_state_dict(
            torch.load('../experiments/pretrained_models/VGG16minc_53.pth'), strict=True)
        self.features.eval()
        # No need to BP to variable
        for k, v in self.features.named_parameters():
            v.requires_grad = False

    def forward(self, x):
        output = self.features(x)
        return output


#TODO
# moved from models.modules.architectures.ASRResNet_arch, did not bring the self-attention layer
# VGG style Discriminator with input size 128*128, with feature_maps extraction and self-attention
class Discriminator_VGG_128_fea(nn.Module):
    def __init__(self, in_nc, base_nf, norm_type='batch', act_type='leakyrelu', mode='CNA', convtype='Conv2D', \
         arch='ESRGAN', spectral_norm=False, self_attention = False, max_pool=False, poolsize = 4):
        super(Discriminator_VGG_128_fea, self).__init__()
        # features
        # hxw, c
        # 128, 64
        
        # Self-Attention configuration
        '''#TODO
        self.self_attention = self_attention
        self.max_pool = max_pool
        self.poolsize = poolsize
        '''
        
        # Remove BatchNorm2d if using spectral_norm
        if spectral_norm:
            norm_type = None
        
        self.conv0 = B.conv_block(in_nc, base_nf, kernel_size=3, norm_type=None, act_type=act_type, \
            mode=mode)
        self.conv1 = B.conv_block(base_nf, base_nf, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode, spectral_norm=spectral_norm)
        # 64, 64
        self.conv2 = B.conv_block(base_nf, base_nf*2, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode, spectral_norm=spectral_norm)
        self.conv3 = B.conv_block(base_nf*2, base_nf*2, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode, spectral_norm=spectral_norm)
        # 32, 128
        self.conv4 = B.conv_block(base_nf*2, base_nf*4, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode, spectral_norm=spectral_norm)
        self.conv5 = B.conv_block(base_nf*4, base_nf*4, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode, spectral_norm=spectral_norm)
        # 16, 256
        
        '''#TODO
        if self.self_attention:
            self.FSA = SelfAttentionBlock(in_dim = base_nf*4, max_pool=self.max_pool, poolsize = self.poolsize, spectral_norm=spectral_norm)
        '''

        self.conv6 = B.conv_block(base_nf*4, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode, spectral_norm=spectral_norm)
        self.conv7 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode, spectral_norm=spectral_norm)
        # 8, 512
        self.conv8 = B.conv_block(base_nf*8, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode, spectral_norm=spectral_norm)
        self.conv9 = B.conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode, spectral_norm=spectral_norm)
        # 4, 512
        # self.features = B.sequential(conv0, conv1, conv2, conv3, conv4, conv5, conv6, conv7, conv8,\
            # conv9)

        # classifier
        if arch=='PPON':
            self.classifier = nn.Sequential(
                nn.Linear(512 * 4 * 4, 128), nn.LeakyReLU(0.2, True), nn.Linear(128, 1))
        else: #arch='ESRGAN':
            self.classifier = nn.Sequential(
                nn.Linear(512 * 4 * 4, 100), nn.LeakyReLU(0.2, True), nn.Linear(100, 1))

    #TODO: modify to a listening dictionary like VGG_Model(), can select what maps to use
    def forward(self, x, return_maps=False):
        feature_maps = []
        # x = self.features(x)
        x = self.conv0(x)
        feature_maps.append(x)
        x = self.conv1(x)
        feature_maps.append(x)
        x = self.conv2(x)
        feature_maps.append(x)
        x = self.conv3(x)
        feature_maps.append(x)
        x = self.conv4(x)
        feature_maps.append(x)
        x = self.conv5(x)
        feature_maps.append(x)
        x = self.conv6(x)
        feature_maps.append(x)
        x = self.conv7(x)
        feature_maps.append(x)
        x = self.conv8(x)
        feature_maps.append(x)
        x = self.conv9(x)
        feature_maps.append(x)
        
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        if return_maps:
            return [x, feature_maps]
        return x





class NLayerDiscriminator(nn.Module):
    r"""
    PatchGAN discriminator
    https://arxiv.org/pdf/1611.07004v3.pdf
    https://arxiv.org/pdf/1803.07422.pdf

    """

    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d, use_sigmoid=False, getIntermFeat=False):
        """Construct a PatchGAN discriminator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
        """
        super(NLayerDiscriminator, self).__init__()
        '''
        if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d
        '''
        #self.getIntermFeat = getIntermFeat # not used for now
        #use_sigmoid not used for now
        #TODO: test if there are benefits by incorporating the use of intermediate features from pix2pixHD

        use_bias = False
        kw = 4
        padw = 1 # int(np.ceil((kw-1.0)/2))

        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):  # gradually increase the number of filters
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)

    def forward(self, x):
        """Standard forward."""
        return self.model(x)


class MultiscaleDiscriminator(nn.Module):
    r"""
    Multiscale PatchGAN discriminator
    https://arxiv.org/pdf/1711.11585.pdf

    """
    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d, 
                 use_sigmoid=False, num_D=3, getIntermFeat=False):
        """Construct a pyramid of PatchGAN discriminators
        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
            use_sigmoid     -- boolean to use sigmoid in patchGAN discriminators
            num_D (int)     -- number of discriminators/downscales in the pyramid
            getIntermFeat   -- boolean to get intermediate features (unused for now)
        """
        super(MultiscaleDiscriminator, self).__init__()
        self.num_D = num_D
        self.n_layers = n_layers
        self.getIntermFeat = getIntermFeat
     
        for i in range(num_D):
            netD = NLayerDiscriminator(input_nc, ndf, n_layers, norm_layer, use_sigmoid, getIntermFeat)
            if getIntermFeat:                                
                for j in range(n_layers+2):
                    setattr(self, 'scale'+str(i)+'_layer'+str(j), getattr(netD, 'model'+str(j)))                                   
            else:
                setattr(self, 'layer'+str(i), netD.model)

        self.downsample = nn.AvgPool2d(3, stride=2, padding=[1, 1], count_include_pad=False)

    def singleD_forward(self, model, input):
        if self.getIntermFeat:
            result = [input]
            for i in range(len(model)):
                result.append(model[i](result[-1]))
            return result[1:]
        else:
            return [model(input)]

    def forward(self, input):        
        num_D = self.num_D
        result = []
        input_downsampled = input
        for i in range(num_D):
            if self.getIntermFeat:
                model = [getattr(self, 'scale'+str(num_D-1-i)+'_layer'+str(j)) for j in range(self.n_layers+2)]
            else:
                model = getattr(self, 'layer'+str(num_D-1-i))
            result.append(self.singleD_forward(model, input_downsampled))
            if i != (num_D-1):
                input_downsampled = self.downsample(input_downsampled)
        return result


class PixelDiscriminator(nn.Module):
    """Defines a 1x1 PatchGAN discriminator (pixelGAN)"""

    def __init__(self, input_nc, ndf=64, norm_layer=nn.BatchNorm2d):
        """Construct a 1x1 PatchGAN discriminator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            norm_layer      -- normalization layer
        """
        super(PixelDiscriminator, self).__init__()
        '''
        if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d
        '''
        use_bias = False

        self.net = [
            nn.Conv2d(input_nc, ndf, kernel_size=1, stride=1, padding=0),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(ndf, ndf * 2, kernel_size=1, stride=1, padding=0, bias=use_bias),
            norm_layer(ndf * 2),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(ndf * 2, 1, kernel_size=1, stride=1, padding=0, bias=use_bias)]

        self.net = nn.Sequential(*self.net)

    def forward(self, input):
        """Standard forward."""
        return self.net(input)




In [ ]:
#@title vic/block.py
%%writefile /content/Colab-edge-connect/src/vic/block.py

"""
BasicSR/codes/models/modules/architectures/block.py (8-Nov-20)
https://github.com/victorca25/BasicSR/blob/dev2/codes/models/modules/architectures/block.py
"""
from collections import OrderedDict
import torch
import torch.nn as nn
#from models.modules.architectures.convolutions.partialconv2d import PartialConv2d #TODO
from .partialconv2d import PartialConv2d

####################
# Basic blocks
####################

# Swish activation funtion
def swish_func(x, beta=1.0):
    '''
    "Swish: a Self-Gated Activation Function"
    Searching for Activation Functions (https://arxiv.org/abs/1710.05941)
    
    If beta=1 applies the Sigmoid Linear Unit (SiLU) function element-wise
    If beta=0, Swish becomes the scaled linear function (identity 
      activation) f(x) = x/2
    As beta -> ∞, the sigmoid component converges to approach a 0-1 function
      (unit step), and multiplying that by x gives us f(x)=2max(0,x), which 
      is the ReLU multiplied by a constant factor of 2, so Swish becomes like 
      the ReLU function.
        
    Including beta, Swish can be loosely viewed as a smooth function that 
      nonlinearly interpolate between identity (linear) and ReLU function.
      The degree of interpolation can be controlled by the model if beta is 
      set as a trainable parameter.
      
    Alt: 1.78718727865 * (x * sigmoid(x) - 0.20662096414)
    '''
    
    # In-place implementation, may consume less GPU memory: 
    """ 
    result = x.clone()
    torch.sigmoid_(beta*x)
    x *= result
    return x
    #"""
    
    # Normal out-of-place implementation:
    #"""
    return x * torch.sigmoid(beta*x)
    #"""
    
# Swish module
class Swish(nn.Module):
    
    __constants__ = ['beta', 'slope', 'inplace']
    
    def __init__(self, beta = 1.0, slope = 1.67653251702, inplace=False):
        '''
        Shape:
        - Input: (N, *) where * means, any number of additional
          dimensions
        - Output: (N, *), same shape as the input
        '''
        super().__init__()
        self.inplace = inplace
        #self.beta = beta # user-defined beta parameter, non-trainable
        #self.beta = beta * torch.nn.Parameter(torch.ones(1)) # learnable beta parameter, create a tensor out of beta
        self.beta = torch.nn.Parameter(torch.tensor(beta)) # learnable beta parameter, create a tensor out of beta
        self.beta.requiresGrad = True # set requiresGrad to true to make it trainable

        self.slope = slope/2 # user-defined "slope", non-trainable
        #self.slope = slope * torch.nn.Parameter(torch.ones(1)) # learnable slope parameter, create a tensor out of slope
        #self.slope = torch.nn.Parameter(torch.tensor(slope)) # learnable slope parameter, create a tensor out of slope
        #self.slope.requiresGrad = True # set requiresGrad to true to true to make it trainable
    
    def forward(self, input):
        """
        # Disabled, using inplace causes:
        # "RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation"
        if self.inplace:
            input.mul_(torch.sigmoid(self.beta*input))
            return 2 * self.slope * input
        else:
            return 2 * self.slope * swish_func(input, self.beta)
        """
        return 2 * self.slope * swish_func(input, self.beta)
        

def act(act_type, inplace=True, neg_slope=0.2, n_prelu=1, beta=1.0):
    # helper selecting activation
    # neg_slope: for leakyrelu and init of prelu
    # n_prelu: for p_relu num_parameters
    # beta: for swish
    act_type = act_type.lower()
    if act_type == 'relu':
        layer = nn.ReLU(inplace)
    elif act_type == 'leakyrelu' or act_type == 'lrelu':
        layer = nn.LeakyReLU(neg_slope, inplace)
    elif act_type == 'prelu':
        layer = nn.PReLU(num_parameters=n_prelu, init=neg_slope)
    elif act_type == 'Tanh' or act_type == 'tanh' : # [-1, 1] range output
        layer = nn.Tanh()
    elif act_type == 'sigmoid': # [0, 1] range output
        layer = nn.Sigmoid()
    elif act_type == 'swish':
        layer = Swish(beta=beta,inplace=inplace)
    else:
        raise NotImplementedError('activation layer [{:s}] is not found'.format(act_type))
    return layer


def norm(norm_type, nc):
    # helper selecting normalization layer
    norm_type = norm_type.lower()
    if norm_type == 'batch':
        layer = nn.BatchNorm2d(nc, affine=True)
    elif norm_type == 'instance':
        layer = nn.InstanceNorm2d(nc, affine=False)
    else:
        raise NotImplementedError('normalization layer [{:s}] is not found'.format(norm_type))
    return layer


def pad(pad_type, padding):
    # helper selecting padding layer
    # if padding is 'zero', do by conv layers
    pad_type = pad_type.lower()
    if padding == 0:
        return None
    if pad_type == 'reflect':
        layer = nn.ReflectionPad2d(padding)
    elif pad_type == 'replicate':
        layer = nn.ReplicationPad2d(padding)
    else:
        raise NotImplementedError('padding layer [{:s}] is not implemented'.format(pad_type))
    return layer


def get_valid_padding(kernel_size, dilation):
    kernel_size = kernel_size + (kernel_size - 1) * (dilation - 1)
    padding = (kernel_size - 1) // 2
    return padding


class ConcatBlock(nn.Module):
    # Concat the output of a submodule to its input
    def __init__(self, submodule):
        super(ConcatBlock, self).__init__()
        self.sub = submodule

    def forward(self, x):
        output = torch.cat((x, self.sub(x)), dim=1)
        return output

    def __repr__(self):
        tmpstr = 'Identity .. \n|'
        modstr = self.sub.__repr__().replace('\n', '\n|')
        tmpstr = tmpstr + modstr
        return tmpstr


class ShortcutBlock(nn.Module):
    #Elementwise sum the output of a submodule to its input
    def __init__(self, submodule):
        super(ShortcutBlock, self).__init__()
        self.sub = submodule

    def forward(self, x):
        output = x + self.sub(x)
        return output

    def __repr__(self):
        tmpstr = 'Identity + \n|'
        modstr = self.sub.__repr__().replace('\n', '\n|')
        tmpstr = tmpstr + modstr
        return tmpstr


def sequential(*args):
    # Flatten Sequential. It unwraps nn.Sequential.
    if len(args) == 1:
        if isinstance(args[0], OrderedDict):
            raise NotImplementedError('sequential does not support OrderedDict input.')
        return args[0]  # No sequential is needed.
    modules = []
    for module in args:
        if isinstance(module, nn.Sequential):
            for submodule in module.children():
                modules.append(submodule)
        elif isinstance(module, nn.Module):
            modules.append(module)
    return nn.Sequential(*modules)


def conv_block(in_nc, out_nc, kernel_size, stride=1, dilation=1, groups=1, bias=True, \
               pad_type='zero', norm_type=None, act_type='relu', mode='CNA', convtype='Conv2D', \
               spectral_norm=False):
    '''
    Conv layer with padding, normalization, activation
    mode: CNA --> Conv -> Norm -> Act
        NAC --> Norm -> Act --> Conv (Identity Mappings in Deep Residual Networks, ECCV16)
    '''
    assert mode in ['CNA', 'NAC', 'CNAC'], 'Wrong conv mode [{:s}]'.format(mode)
    padding = get_valid_padding(kernel_size, dilation)
    p = pad(pad_type, padding) if pad_type and pad_type != 'zero' else None
    padding = padding if pad_type == 'zero' else 0
    
    if convtype=='PartialConv2D':
        c = PartialConv2d(in_nc, out_nc, kernel_size=kernel_size, stride=stride, padding=padding, \
               dilation=dilation, bias=bias, groups=groups)
    else: #default case is standard 'Conv2D':
        c = nn.Conv2d(in_nc, out_nc, kernel_size=kernel_size, stride=stride, padding=padding, \
                dilation=dilation, bias=bias, groups=groups) #normal conv2d
            
    if spectral_norm:
        c = nn.utils.spectral_norm(c)
    
    a = act(act_type) if act_type else None
    if 'CNA' in mode:
        n = norm(norm_type, out_nc) if norm_type else None
        return sequential(p, c, n, a)
    elif mode == 'NAC':
        if norm_type is None and act_type is not None:
            a = act(act_type, inplace=False)
            # Important!
            # input----ReLU(inplace)----Conv--+----output
            #        |________________________|
            # inplace ReLU will modify the input, therefore wrong output
        n = norm(norm_type, in_nc) if norm_type else None
        return sequential(n, a, p, c)


####################
# Useful blocks
####################


class ResNetBlock(nn.Module):
    '''
    ResNet Block, 3-3 style
    with extra residual scaling used in EDSR
    (Enhanced Deep Residual Networks for Single Image Super-Resolution, CVPRW 17)
    '''

    def __init__(self, in_nc, mid_nc, out_nc, kernel_size=3, stride=1, dilation=1, groups=1, \
            bias=True, pad_type='zero', norm_type=None, act_type='relu', mode='CNA', res_scale=1, convtype='Conv2D'):
        super(ResNetBlock, self).__init__()
        conv0 = conv_block(in_nc, mid_nc, kernel_size, stride, dilation, groups, bias, pad_type, \
            norm_type, act_type, mode, convtype)
        if mode == 'CNA':
            act_type = None
        if mode == 'CNAC':  # Residual path: |-CNAC-|
            act_type = None
            norm_type = None
        conv1 = conv_block(mid_nc, out_nc, kernel_size, stride, dilation, groups, bias, pad_type, \
            norm_type, act_type, mode, convtype)
        # if in_nc != out_nc:
        #     self.project = conv_block(in_nc, out_nc, 1, stride, dilation, 1, bias, pad_type, \
        #         None, None)
        #     print('Need a projecter in ResNetBlock.')
        # else:
        #     self.project = lambda x:x
        self.res = sequential(conv0, conv1)
        self.res_scale = res_scale

    def forward(self, x):
        res = self.res(x).mul(self.res_scale)
        return x + res


class ResidualDenseBlock_5C(nn.Module):
    '''
    Residual Dense Block
    style: 5 convs
    The core module of paper: (Residual Dense Network for Image Super-Resolution, CVPR 18)
    Modified options that can be used:
        - "Partial Convolution based Padding" arXiv:1811.11718
        - "Spectral normalization" arXiv:1802.05957
        - "ICASSP 2020 - ESRGAN+ : Further Improving ESRGAN" N. C. {Rakotonirina} and A. {Rasoanaivo}
    '''

    def __init__(self, nc, kernel_size=3, gc=32, stride=1, bias=True, pad_type='zero', \
            norm_type=None, act_type='leakyrelu', mode='CNA', convtype='Conv2D', \
            spectral_norm=False, gaussian_noise=False, plus=False):
        super(ResidualDenseBlock_5C, self).__init__()
        # gc: growth channel, i.e. intermediate channels
        
        ## +
        self.noise = GaussianNoise() if gaussian_noise else None
        self.conv1x1 = conv1x1(nc, gc) if plus else None
        ## +

        self.conv1 = conv_block(nc, gc, kernel_size, stride, bias=bias, pad_type=pad_type, \
            norm_type=norm_type, act_type=act_type, mode=mode, convtype=convtype, \
            spectral_norm=spectral_norm)
        self.conv2 = conv_block(nc+gc, gc, kernel_size, stride, bias=bias, pad_type=pad_type, \
            norm_type=norm_type, act_type=act_type, mode=mode, convtype=convtype, \
            spectral_norm=spectral_norm)
        self.conv3 = conv_block(nc+2*gc, gc, kernel_size, stride, bias=bias, pad_type=pad_type, \
            norm_type=norm_type, act_type=act_type, mode=mode, convtype=convtype, \
            spectral_norm=spectral_norm)
        self.conv4 = conv_block(nc+3*gc, gc, kernel_size, stride, bias=bias, pad_type=pad_type, \
            norm_type=norm_type, act_type=act_type, mode=mode, convtype=convtype, \
            spectral_norm=spectral_norm)
        if mode == 'CNA':
            last_act = None
        else:
            last_act = act_type
        self.conv5 = conv_block(nc+4*gc, nc, 3, stride, bias=bias, pad_type=pad_type, \
            norm_type=norm_type, act_type=last_act, mode=mode, convtype=convtype, \
            spectral_norm=spectral_norm)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(torch.cat((x, x1), 1))
        if self.conv1x1:
            x2 = x2 + self.conv1x1(x) #+
        x3 = self.conv3(torch.cat((x, x1, x2), 1))
        x4 = self.conv4(torch.cat((x, x1, x2, x3), 1))
        if self.conv1x1:
            x4 = x4 + x2 #+
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        if self.noise:
            return self.noise(x5.mul(0.2) + x)
        else:
            return x5.mul(0.2) + x

class RRDB(nn.Module):
    '''
    Residual in Residual Dense Block
    (ESRGAN: Enhanced Super-Resolution Generative Adversarial Networks)
    '''

    def __init__(self, nc, kernel_size=3, gc=32, stride=1, bias=True, pad_type='zero', \
            norm_type=None, act_type='leakyrelu', mode='CNA', convtype='Conv2D', \
            spectral_norm=False, gaussian_noise=False, plus=False):
        super(RRDB, self).__init__()
        self.RDB1 = ResidualDenseBlock_5C(nc, kernel_size, gc, stride, bias, pad_type, \
                norm_type, act_type, mode, convtype, spectral_norm=spectral_norm, \
                gaussian_noise=gaussian_noise, plus=plus)
        self.RDB2 = ResidualDenseBlock_5C(nc, kernel_size, gc, stride, bias, pad_type, \
                norm_type, act_type, mode, convtype, spectral_norm=spectral_norm, \
                gaussian_noise=gaussian_noise, plus=plus)
        self.RDB3 = ResidualDenseBlock_5C(nc, kernel_size, gc, stride, bias, pad_type, \
                norm_type, act_type, mode, convtype, spectral_norm=spectral_norm, \
                gaussian_noise=gaussian_noise, plus=plus)

    def forward(self, x):
        out = self.RDB1(x)
        out = self.RDB2(out)
        out = self.RDB3(out)
        return out.mul(0.2) + x
            


#PPON
class _ResBlock_32(nn.Module):
    def __init__(self, nc=64):
        super(_ResBlock_32, self).__init__()
        self.c1 = conv_layer(nc, nc, 3, 1, 1)
        self.d1 = conv_layer(nc, nc//2, 3, 1, 1)  # rate=1
        self.d2 = conv_layer(nc, nc//2, 3, 1, 2)  # rate=2
        self.d3 = conv_layer(nc, nc//2, 3, 1, 3)  # rate=3
        self.d4 = conv_layer(nc, nc//2, 3, 1, 4)  # rate=4
        self.d5 = conv_layer(nc, nc//2, 3, 1, 5)  # rate=5
        self.d6 = conv_layer(nc, nc//2, 3, 1, 6)  # rate=6
        self.d7 = conv_layer(nc, nc//2, 3, 1, 7)  # rate=7
        self.d8 = conv_layer(nc, nc//2, 3, 1, 8)  # rate=8
        self.act = act('lrelu')
        self.c2 = conv_layer(nc * 4, nc, 1, 1, 1)  # 256-->64

    def forward(self, input):
        output1 = self.act(self.c1(input))
        d1 = self.d1(output1)
        d2 = self.d2(output1)
        d3 = self.d3(output1)
        d4 = self.d4(output1)
        d5 = self.d5(output1)
        d6 = self.d6(output1)
        d7 = self.d7(output1)
        d8 = self.d8(output1)

        add1 = d1 + d2
        add2 = add1 + d3
        add3 = add2 + d4
        add4 = add3 + d5
        add5 = add4 + d6
        add6 = add5 + d7
        add7 = add6 + d8

        combine = torch.cat([d1, add1, add2, add3, add4, add5, add6, add7], 1)
        output2 = self.c2(self.act(combine))
        output = input + output2.mul(0.2)

        return output

class RRBlock_32(nn.Module):
    def __init__(self):
        super(RRBlock_32, self).__init__()
        self.RB1 = _ResBlock_32()
        self.RB2 = _ResBlock_32()
        self.RB3 = _ResBlock_32()

    def forward(self, input):
        out = self.RB1(input)
        out = self.RB2(out)
        out = self.RB3(out)
        return out.mul(0.2) + input


####################
# Upsampler
####################

class Upsample(nn.Module):
    #To prevent warning: nn.Upsample is deprecated
    #https://discuss.pytorch.org/t/which-function-is-better-for-upsampling-upsampling-or-interpolate/21811/8
    #From: https://pytorch.org/docs/stable/_modules/torch/nn/modules/upsampling.html#Upsample
    #Alternative: https://discuss.pytorch.org/t/using-nn-function-interpolate-inside-nn-sequential/23588/2?u=ptrblck
    
    def __init__(self, size=None, scale_factor=None, mode="nearest", align_corners=None):
        super(Upsample, self).__init__()
        if isinstance(scale_factor, tuple):
            self.scale_factor = tuple(float(factor) for factor in scale_factor)
        else:
            self.scale_factor = float(scale_factor) if scale_factor else None
        self.mode = mode
        self.size = size
        self.align_corners = align_corners
        #self.interp = nn.functional.interpolate
    
    def forward(self, x):
        return nn.functional.interpolate(x, size=self.size, scale_factor=self.scale_factor, mode=self.mode, align_corners=self.align_corners)
        #return self.interp(x, size=self.size, scale_factor=self.scale_factor, mode=self.mode, align_corners=self.align_corners)
    
    def extra_repr(self):
        if self.scale_factor is not None:
            info = 'scale_factor=' + str(self.scale_factor)
        else:
            info = 'size=' + str(self.size)
        info += ', mode=' + self.mode
        return info

def pixelshuffle_block(in_nc, out_nc, upscale_factor=2, kernel_size=3, stride=1, bias=True, \
                        pad_type='zero', norm_type=None, act_type='relu', convtype='Conv2D'):
    '''
    Pixel shuffle layer
    (Real-Time Single Image and Video Super-Resolution Using an Efficient Sub-Pixel Convolutional
    Neural Network, CVPR17)
    '''
    conv = conv_block(in_nc, out_nc * (upscale_factor ** 2), kernel_size, stride, bias=bias, \
                        pad_type=pad_type, norm_type=None, act_type=None, convtype=convtype)
    pixel_shuffle = nn.PixelShuffle(upscale_factor)

    n = norm(norm_type, out_nc) if norm_type else None
    a = act(act_type) if act_type else None
    return sequential(conv, pixel_shuffle, n, a)

def upconv_blcok(in_nc, out_nc, upscale_factor=2, kernel_size=3, stride=1, bias=True, \
                pad_type='zero', norm_type=None, act_type='relu', mode='nearest', convtype='Conv2D'):
    # Up conv
    # described in https://distill.pub/2016/deconv-checkerboard/
    #upsample = nn.Upsample(scale_factor=upscale_factor, mode=mode)
    upsample = Upsample(scale_factor=upscale_factor, mode=mode) #Updated to prevent the "nn.Upsample is deprecated" Warning
    conv = conv_block(in_nc, out_nc, kernel_size, stride, bias=bias, \
                        pad_type=pad_type, norm_type=norm_type, act_type=act_type, convtype=convtype)
    return sequential(upsample, conv)

#PPON
def conv_layer(in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1):
    padding = int((kernel_size - 1) / 2) * dilation
    return nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=padding, bias=True, dilation=dilation, groups=groups)




####################
#ESRGANplus
####################

class GaussianNoise(nn.Module):
    def __init__(self, sigma=0.1, is_relative_detach=False):
        super().__init__()
        self.sigma = sigma
        self.is_relative_detach = is_relative_detach
        self.noise = torch.tensor(0, dtype=torch.float).to(torch.device('cuda'))

    def forward(self, x):
        if self.training and self.sigma != 0:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            sampled_noise = self.noise.repeat(*x.size()).normal_() * scale
            x = x + sampled_noise
        return x 

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


#TODO: Not used:
# https://github.com/github-pengge/PyTorch-progressive_growing_of_gans/blob/master/models/base_model.py
class minibatch_std_concat_layer(nn.Module):
    def __init__(self, averaging='all'):
        super(minibatch_std_concat_layer, self).__init__()
        self.averaging = averaging.lower()
        if 'group' in self.averaging:
            self.n = int(self.averaging[5:])
        else:
            assert self.averaging in ['all', 'flat', 'spatial', 'none', 'gpool'], 'Invalid averaging mode'%self.averaging
        self.adjusted_std = lambda x, **kwargs: torch.sqrt(torch.mean((x - torch.mean(x, **kwargs)) ** 2, **kwargs) + 1e-8)

    def forward(self, x):
        shape = list(x.size())
        target_shape = copy.deepcopy(shape)
        vals = self.adjusted_std(x, dim=0, keepdim=True)
        if self.averaging == 'all':
            target_shape[1] = 1
            vals = torch.mean(vals, dim=1, keepdim=True)
        elif self.averaging == 'spatial':
            if len(shape) == 4:
                vals = mean(vals, axis=[2,3], keepdim=True)             # torch.mean(torch.mean(vals, 2, keepdim=True), 3, keepdim=True)
        elif self.averaging == 'none':
            target_shape = [target_shape[0]] + [s for s in target_shape[1:]]
        elif self.averaging == 'gpool':
            if len(shape) == 4:
                vals = mean(x, [0,2,3], keepdim=True)                   # torch.mean(torch.mean(torch.mean(x, 2, keepdim=True), 3, keepdim=True), 0, keepdim=True)
        elif self.averaging == 'flat':
            target_shape[1] = 1
            vals = torch.FloatTensor([self.adjusted_std(x)])
        else:                                                           # self.averaging == 'group'
            target_shape[1] = self.n
            vals = vals.view(self.n, self.shape[1]/self.n, self.shape[2], self.shape[3])
            vals = mean(vals, axis=0, keepdim=True).view(1, self.n, 1, 1)
        vals = vals.expand(*target_shape)
        return torch.cat([x, vals], 1)



In [ ]:
#@title vic/partialconv2d.py
%%writefile /content/Colab-edge-connect/src/vic/partialconv2d.py
"""
BasicSR/codes/models/modules/architectures/convolutions/partialconv2d.py (8-Nov-20)
https://github.com/victorca25/BasicSR/blob/dev2/codes/models/modules/architectures/convolutions/partialconv2d.py
"""

###############################################################################
# BSD 3-Clause License
#
# Copyright (c) 2018, NVIDIA CORPORATION. All rights reserved.
#
# Author & Contact: Guilin Liu (guilinl@nvidia.com)
#
# Source: https://github.com/NVIDIA/partialconv/blob/master/models/partialconv2d.py
###############################################################################

import torch
import torch.nn.functional as F
from torch import nn, cuda
from torch.autograd import Variable

class PartialConv2d(nn.Conv2d):
    def __init__(self, *args, **kwargs):

        # whether the mask is multi-channel or not
        if 'multi_channel' in kwargs:
            self.multi_channel = kwargs['multi_channel']
            kwargs.pop('multi_channel')
        else:
            self.multi_channel = False  

        if 'return_mask' in kwargs:
            self.return_mask = kwargs['return_mask']
            kwargs.pop('return_mask')
        else:
            self.return_mask = False

        super(PartialConv2d, self).__init__(*args, **kwargs)

        if self.multi_channel:
            self.weight_maskUpdater = torch.ones(self.out_channels, self.in_channels, self.kernel_size[0], self.kernel_size[1])
        else:
            self.weight_maskUpdater = torch.ones(1, 1, self.kernel_size[0], self.kernel_size[1])
            
        self.slide_winsize = self.weight_maskUpdater.shape[1] * self.weight_maskUpdater.shape[2] * self.weight_maskUpdater.shape[3]

        self.last_size = (None, None, None, None)
        self.update_mask = None
        self.mask_ratio = None

    def forward(self, input, mask_in=None):
        assert len(input.shape) == 4
        if mask_in is not None or self.last_size != tuple(input.shape):
            self.last_size = tuple(input.shape)

            with torch.no_grad():
                if self.weight_maskUpdater.type() != input.type():
                    self.weight_maskUpdater = self.weight_maskUpdater.to(input)

                if mask_in is None:
                    # if mask is not provided, create a mask
                    if self.multi_channel:
                        mask = torch.ones(input.data.shape[0], input.data.shape[1], input.data.shape[2], input.data.shape[3]).to(input)
                    else:
                        mask = torch.ones(1, 1, input.data.shape[2], input.data.shape[3]).to(input)
                else:
                    mask = mask_in
                        
                self.update_mask = F.conv2d(mask, self.weight_maskUpdater, bias=None, stride=self.stride, padding=self.padding, dilation=self.dilation, groups=1)

                self.mask_ratio = self.slide_winsize/(self.update_mask + 1e-8)
                #make sure the value of self.mask_ratio for the entries in the interior (no need for padding) have value 1. If not, you replace with the line below.
                # self.mask_ratio = torch.max(self.update_mask)/(self.update_mask + 1e-8)
                self.update_mask = torch.clamp(self.update_mask, 0, 1)
                self.mask_ratio = torch.mul(self.mask_ratio, self.update_mask)

        # if self.update_mask.type() != input.type() or self.mask_ratio.type() != input.type():
        #     self.update_mask.to(input)
        #     self.mask_ratio.to(input)

        raw_out = super(PartialConv2d, self).forward(torch.mul(input, mask) if mask_in is not None else input)

        if self.bias is not None:
            bias_view = self.bias.view(1, self.out_channels, 1, 1)
            output = torch.mul(raw_out - bias_view, self.mask_ratio) + bias_view
            output = torch.mul(output, self.update_mask)
        else:
            output = torch.mul(raw_out, self.mask_ratio)


        if self.return_mask:
            return output, self.update_mask
        else:
            return output


In [ ]:
#@title vic/spectral_norm.py
%%writefile /content/Colab-edge-connect/src/vic/spectral_norm.py
"""
BasicSR/codes/models/modules/architectures/spectral_norm.py
https://github.com/victorca25/BasicSR/blob/dev2/codes/models/modules/architectures/spectral_norm.py
"""

'''
Copy from pytorch github repo
Spectral Normalization from https://arxiv.org/abs/1802.05957
'''
import torch
from torch.nn.functional import normalize
from torch.nn.parameter import Parameter


class SpectralNorm(object):
    def __init__(self, name='weight', n_power_iterations=1, dim=0, eps=1e-12):
        self.name = name
        self.dim = dim
        if n_power_iterations <= 0:
            raise ValueError('Expected n_power_iterations to be positive, but '
                             'got n_power_iterations={}'.format(n_power_iterations))
        self.n_power_iterations = n_power_iterations
        self.eps = eps

    def compute_weight(self, module):
        weight = getattr(module, self.name + '_orig')
        u = getattr(module, self.name + '_u')
        weight_mat = weight
        if self.dim != 0:
            # permute dim to front
            weight_mat = weight_mat.permute(self.dim,
                                            *[d for d in range(weight_mat.dim()) if d != self.dim])
        height = weight_mat.size(0)
        weight_mat = weight_mat.reshape(height, -1)
        with torch.no_grad():
            for _ in range(self.n_power_iterations):
                # Spectral norm of weight equals to `u^T W v`, where `u` and `v`
                # are the first left and right singular vectors.
                # This power iteration produces approximations of `u` and `v`.
                v = normalize(torch.matmul(weight_mat.t(), u), dim=0, eps=self.eps)
                u = normalize(torch.matmul(weight_mat, v), dim=0, eps=self.eps)

        sigma = torch.dot(u, torch.matmul(weight_mat, v))
        weight = weight / sigma
        return weight, u

    def remove(self, module):
        weight = getattr(module, self.name)
        delattr(module, self.name)
        delattr(module, self.name + '_u')
        delattr(module, self.name + '_orig')
        module.register_parameter(self.name, torch.nn.Parameter(weight))

    def __call__(self, module, inputs):
        if module.training:
            weight, u = self.compute_weight(module)
            setattr(module, self.name, weight)
            setattr(module, self.name + '_u', u)
        else:
            r_g = getattr(module, self.name + '_orig').requires_grad
            getattr(module, self.name).detach_().requires_grad_(r_g)

    @staticmethod
    def apply(module, name, n_power_iterations, dim, eps):
        fn = SpectralNorm(name, n_power_iterations, dim, eps)
        weight = module._parameters[name]
        height = weight.size(dim)

        u = normalize(weight.new_empty(height).normal_(0, 1), dim=0, eps=fn.eps)
        delattr(module, fn.name)
        module.register_parameter(fn.name + "_orig", weight)
        # We still need to assign weight back as fn.name because all sorts of
        # things may assume that it exists, e.g., when initializing weights.
        # However, we can't directly assign as it could be an nn.Parameter and
        # gets added as a parameter. Instead, we register weight.data as a
        # buffer, which will cause weight to be included in the state dict
        # and also supports nn.init due to shared storage.
        module.register_buffer(fn.name, weight.data)
        module.register_buffer(fn.name + "_u", u)

        module.register_forward_pre_hook(fn)
        return fn


def spectral_norm(module, name='weight', n_power_iterations=1, eps=1e-12, dim=None):
    r"""Applies spectral normalization to a parameter in the given module.

    .. math::
         \mathbf{W} &= \dfrac{\mathbf{W}}{\sigma(\mathbf{W})} \\
         \sigma(\mathbf{W}) &= \max_{\mathbf{h}: \mathbf{h} \ne 0} \dfrac{\|\mathbf{W} \mathbf{h}\|_2}{\|\mathbf{h}\|_2}

    Spectral normalization stabilizes the training of discriminators (critics)
    in Generaive Adversarial Networks (GANs) by rescaling the weight tensor
    with spectral norm :math:`\sigma` of the weight matrix calculated using
    power iteration method. If the dimension of the weight tensor is greater
    than 2, it is reshaped to 2D in power iteration method to get spectral
    norm. This is implemented via a hook that calculates spectral norm and
    rescales weight before every :meth:`~Module.forward` call.

    See `Spectral Normalization for Generative Adversarial Networks`_ .

    .. _`Spectral Normalization for Generative Adversarial Networks`: https://arxiv.org/abs/1802.05957

    Args:
        module (nn.Module): containing module
        name (str, optional): name of weight parameter
        n_power_iterations (int, optional): number of power iterations to
            calculate spectal norm
        eps (float, optional): epsilon for numerical stability in
            calculating norms
        dim (int, optional): dimension corresponding to number of outputs,
            the default is 0, except for modules that are instances of
            ConvTranspose1/2/3d, when it is 1

    Returns:
        The original module with the spectal norm hook

    Example::

        >>> m = spectral_norm(nn.Linear(20, 40))
        Linear (20 -> 40)
        >>> m.weight_u.size()
        torch.Size([20])

    """
    if dim is None:
        if isinstance(
                module,
            (torch.nn.ConvTranspose1d, torch.nn.ConvTranspose2d, torch.nn.ConvTranspose3d)):
            dim = 1
        else:
            dim = 0
    SpectralNorm.apply(module, name, n_power_iterations, dim, eps)
    return module


def remove_spectral_norm(module, name='weight'):
    r"""Removes the spectral normalization reparameterization from a module.

    Args:
        module (nn.Module): containing module
        name (str, optional): name of weight parameter

    Example:
        >>> m = spectral_norm(nn.Linear(40, 10))
        >>> remove_spectral_norm(m)
    """
    for k, hook in module._forward_pre_hooks.items():
        if isinstance(hook, SpectralNorm) and hook.name == name:
            hook.remove(module)
            del module._forward_pre_hooks[k]
            return module

    raise ValueError("spectral_norm of '{}' not found in {}".format(name, module))


In [ ]:
#@title diffaug.py
%%writefile /content/Colab-edge-connect/src/diffaug.py

# Differentiable Augmentation for Data-Efficient GAN Training
# Shengyu Zhao, Zhijian Liu, Ji Lin, Jun-Yan Zhu, and Song Han
# https://arxiv.org/pdf/2006.10738
import torch
import torch.nn.functional as F 

policy = 'color,translation,cutout' 

import torch.nn.functional as nnf
import random

#torch.autograd.set_detect_anomaly(True)
scaler = torch.cuda.amp.GradScaler() 

def DiffAugment(x, policy='', channels_first=True):
    if policy:
        if not channels_first:
            x = x.permute(0, 3, 1, 2)
        for p in policy.split(','):
            for f in AUGMENT_FNS[p]:
                x = f(x)
        if not channels_first:
            x = x.permute(0, 2, 3, 1)
        x = x.contiguous()
    return x


def rand_brightness(x):
    x = x + (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) - 0.5)
    return x


def rand_saturation(x):
    x_mean = x.mean(dim=1, keepdim=True)
    x = (x - x_mean) * (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) * 2) + x_mean
    return x


def rand_contrast(x):
    x_mean = x.mean(dim=[1, 2, 3], keepdim=True)
    x = (x - x_mean) * (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) + 0.5) + x_mean
    return x


def rand_translation(x, ratio=0.125):
    shift_x, shift_y = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
    translation_x = torch.randint(-shift_x, shift_x + 1, size=[x.size(0), 1, 1], device=x.device)
    translation_y = torch.randint(-shift_y, shift_y + 1, size=[x.size(0), 1, 1], device=x.device)
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype=torch.long, device=x.device),
        torch.arange(x.size(2), dtype=torch.long, device=x.device),
        torch.arange(x.size(3), dtype=torch.long, device=x.device),
    )
    grid_x = torch.clamp(grid_x + translation_x + 1, 0, x.size(2) + 1)
    grid_y = torch.clamp(grid_y + translation_y + 1, 0, x.size(3) + 1)
    x_pad = F.pad(x, [1, 1, 1, 1, 0, 0, 0, 0])
    x = x_pad.permute(0, 2, 3, 1).contiguous()[grid_batch, grid_x, grid_y].permute(0, 3, 1, 2)
    return x


def rand_cutout(x, ratio=0.5):
    cutout_size = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
    offset_x = torch.randint(0, x.size(2) + (1 - cutout_size[0] % 2), size=[x.size(0), 1, 1], device=x.device)
    offset_y = torch.randint(0, x.size(3) + (1 - cutout_size[1] % 2), size=[x.size(0), 1, 1], device=x.device)
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype=torch.long, device=x.device),
        torch.arange(cutout_size[0], dtype=torch.long, device=x.device),
        torch.arange(cutout_size[1], dtype=torch.long, device=x.device),
    )
    grid_x = torch.clamp(grid_x + offset_x - cutout_size[0] // 2, min=0, max=x.size(2) - 1)
    grid_y = torch.clamp(grid_y + offset_y - cutout_size[1] // 2, min=0, max=x.size(3) - 1)
    mask = torch.ones(x.size(0), x.size(2), x.size(3), dtype=x.dtype, device=x.device)
    mask[grid_batch, grid_x, grid_y] = 0
    x = x * mask.unsqueeze(1)
    return x


AUGMENT_FNS = {
    'color': [rand_brightness, rand_saturation, rand_contrast],
    'translation': [rand_translation],
    'cutout': [rand_cutout],
}


In [ ]:
#@title models.py
%%writefile /content/Colab-edge-connect/src/models.py

from .vic.loss import CharbonnierLoss, GANLoss, GradientPenaltyLoss, HFENLoss, TVLoss, GradientLoss, ElasticLoss, RelativeL1, L1CosineSim, ClipL1, MaskedL1Loss, MultiscalePixelLoss, FFTloss, OFLoss, L1_regularization, ColorLoss, AverageLoss, GPLoss, CPLoss, SPL_ComputeWithTrace, SPLoss, Contextual_Loss
from .vic.filters import *
from .vic.colors import *
from .vic.discriminators import *
from .diffaug import *

import os
import torch
import torch.nn as nn
import torch.optim as optim
from .networks import InpaintGenerator, EdgeGenerator, Discriminator
from .loss import AdversarialLoss, PerceptualLoss, StyleLoss

class BaseModel(nn.Module):
    def __init__(self, name, config):
        super(BaseModel, self).__init__()

        self.name = name
        self.config = config
        self.iteration = 0
        self.mosaic_test = config.MOSAIC_TEST

		# loading previous weights
        self.gen_weights_path = os.path.join(config.PATH, name + '_gen.pth')
        self.dis_weights_path = os.path.join(config.PATH, name + '_dis.pth')

    def load(self):
        if os.path.exists(self.gen_weights_path):
            print('Loading %s generator...' % self.name)

            if torch.cuda.is_available():
                data = torch.load(self.gen_weights_path)
            else:
                data = torch.load(self.gen_weights_path, map_location=lambda storage, loc: storage)

            self.generator.load_state_dict(data['generator'])
            self.iteration = data['iteration']

        # load discriminator only when training
        if self.config.MODE == 1 and os.path.exists(self.dis_weights_path):
            print('Loading %s discriminator...' % self.name)

            if torch.cuda.is_available():
                data = torch.load(self.dis_weights_path)
            else:
                data = torch.load(self.dis_weights_path, map_location=lambda storage, loc: storage)

            if self.config.DISCRIMINATOR == 'default':
              self.discriminator.load_state_dict(data['discriminator'])
            if self.config.DISCRIMINATOR == 'pixel':
              self.PixelDiscriminator.load_state_dict(data['discriminator'])
            if self.config.DISCRIMINATOR == 'patch':
              self.NLayerDiscriminator.load_state_dict(data['discriminator'])

    def save(self):
        print('\nsaving %s...\n' % self.name)
        torch.save({
            'iteration': self.iteration,
            'generator': self.generator.state_dict()
        }, os.path.join(self.config.PATH, self.name + "_" + str(self.iteration) + "_gen.pth"))

        if self.config.DISCRIMINATOR == 'default':
          torch.save({
              #'discriminator': self.discriminator.state_dict()
              'discriminator': self.PixelDiscriminator.state_dict()
          }, os.path.join(self.config.PATH, self.name + "_" + str(self.iteration) + "_dis.pth"))
        if self.config.DISCRIMINATOR == 'pixel':
          torch.save({
              'discriminator': self.PixelDiscriminator.state_dict()
          }, os.path.join(self.config.PATH, self.name + "_" + str(self.iteration) + "_dis.pth"))
        if self.config.DISCRIMINATOR == 'patch':
          torch.save({
              'discriminator': self.NLayerDiscriminator.state_dict()
          }, os.path.join(self.config.PATH, self.name + "_" + str(self.iteration) + "_dis.pth"))

class EdgeModel(BaseModel):
    def __init__(self, config):
        super(EdgeModel, self).__init__('EdgeModel', config)

        # generator input: [grayscale(1) + edge(1) + mask(1)]
        # discriminator input: (grayscale(1) + edge(1))
        generator = EdgeGenerator(use_spectral_norm=True)
        discriminator = Discriminator(in_channels=2, use_sigmoid=config.GAN_LOSS != 'hinge')
        if len(config.GPU) > 1:
            generator = nn.DataParallel(generator, config.GPU)
            discriminator = nn.DataParallel(discriminator, config.GPU)
        l1_loss = nn.L1Loss()
        adversarial_loss = AdversarialLoss(type=config.GAN_LOSS)

        self.add_module('generator', generator)
        self.add_module('discriminator', discriminator)

        self.add_module('l1_loss', l1_loss)
        self.add_module('adversarial_loss', adversarial_loss)

        self.gen_optimizer = optim.Adam(
            params=generator.parameters(),
            lr=float(config.LR),
            betas=(config.BETA1, config.BETA2)
        )

        self.dis_optimizer = optim.Adam(
            params=discriminator.parameters(),
            lr=float(config.LR) * float(config.D2G_LR),
            betas=(config.BETA1, config.BETA2)
        )
        

    def process(self, images, edges, masks):
        self.iteration += 1


        # zero optimizers
        self.gen_optimizer.zero_grad()
        self.dis_optimizer.zero_grad()


        # process outputs
        outputs = self(images, edges, masks)
        gen_loss = 0
        dis_loss = 0


        # discriminator loss
        dis_input_real = torch.cat((images, edges), dim=1)
        dis_input_fake = torch.cat((images, outputs.detach()), dim=1)
        #real_scores = Discriminator(DiffAugment(reals, policy=policy))
        dis_real, dis_real_feat = self.discriminator(DiffAugment(dis_input_real, policy=policy))        # in: (grayscale(1) + edge(1))
        dis_fake, dis_fake_feat = self.discriminator(DiffAugment(dis_input_fake, policy=policy))        # in: (grayscale(1) + edge(1))
        dis_real_loss = self.adversarial_loss(dis_real, True, True)
        dis_fake_loss = self.adversarial_loss(dis_fake, False, True)
        dis_loss += (dis_real_loss + dis_fake_loss) / 2


        # generator adversarial loss
        gen_input_fake = torch.cat((images, outputs), dim=1)
        gen_fake, gen_fake_feat = self.discriminator(DiffAugment(gen_input_fake, policy=policy))         # in: (grayscale(1) + edge(1))
        gen_gan_loss = self.adversarial_loss(gen_fake, True, False)
        gen_loss += gen_gan_loss


        # generator feature matching loss
        gen_fm_loss = 0
        for i in range(len(dis_real_feat)):
            gen_fm_loss += self.l1_loss(gen_fake_feat[i], dis_real_feat[i].detach())
        gen_fm_loss = gen_fm_loss * self.config.FM_LOSS_WEIGHT
        gen_loss += gen_fm_loss


        # create logs
        logs = [
            ("l_d1", dis_loss.item()),
            ("l_g1", gen_gan_loss.item()),
            ("l_fm", gen_fm_loss.item()),
        ]

        return outputs, gen_loss, dis_loss, logs

    def forward(self, images, edges, masks):
        edges_masked = (edges * (1 - masks))
        images_masked = (images * (1 - masks)) + masks
        inputs = torch.cat((images_masked, edges_masked, masks), dim=1)
        outputs = self.generator(inputs)                                    # in: [grayscale(1) + edge(1) + mask(1)]
        return outputs

    def backward(self, gen_loss=None, dis_loss=None):
        if dis_loss is not None:
            dis_loss.backward()
        self.dis_optimizer.step()

        if gen_loss is not None:
            gen_loss.backward()
        self.gen_optimizer.step()


class InpaintingModel(BaseModel):
    def __init__(self, config):
        super(InpaintingModel, self).__init__('InpaintingModel', config)

        # generator input: [rgb(3) + edge(1)]
        # discriminator input: [rgb(3)]
        generator = InpaintGenerator()
        """
        if len(config.GPU) > 1:
            generator = nn.DataParallel(generator, config.GPU)
            discriminator = nn.DataParallel(discriminator , config.GPU)
        """
        # original loss
        l1_loss = nn.L1Loss()
        perceptual_loss = PerceptualLoss()
        style_loss = StyleLoss()
        adversarial_loss = AdversarialLoss(type=config.GAN_LOSS)

        self.generator_loss = config.GENERATOR_LOSS

        self.add_module('generator', generator)

        if self.config.DISCRIMINATOR == 'default':
          discriminator = Discriminator(in_channels=3, use_sigmoid=config.GAN_LOSS != 'hinge')
          self.add_module('discriminator', discriminator)

        if self.config.DISCRIMINATOR == 'pixel':
          _PixelDiscriminator = PixelDiscriminator(input_nc=3, ndf=64, norm_layer=nn.BatchNorm2d)
          self.add_module('PixelDiscriminator', _PixelDiscriminator)
        
        if self.config.DISCRIMINATOR == 'patch':
          _NLayerDiscriminator = NLayerDiscriminator(input_nc=3, ndf=64, norm_layer=nn.BatchNorm2d)
          self.add_module('NLayerDiscriminator', _NLayerDiscriminator)

        self.add_module('l1_loss', l1_loss)
        self.add_module('perceptual_loss', perceptual_loss)
        self.add_module('style_loss', style_loss)
        self.add_module('adversarial_loss', adversarial_loss)

        # new added loss
        # CharbonnierLoss (L1) (already implemented?)
        _CharbonnierLoss = CharbonnierLoss()
        self.add_module('_CharbonnierLoss', _CharbonnierLoss)
        # GANLoss (vanilla, lsgan, srpgan, nsgan, hinge, wgan-gp)
        _GANLoss = GANLoss('vanilla', real_label_val=1.0, fake_label_val=0.0)
        self.add_module('_GANLoss', _GANLoss)
        # GradientPenaltyLoss
        _GradientPenaltyLoss = GradientPenaltyLoss()
        self.add_module('_GradientPenaltyLoss', _GradientPenaltyLoss)
        # HFENLoss
        #l_hfen_type = CharbonnierLoss() # nn.L1Loss(), nn.MSELoss(), CharbonnierLoss(), ElasticLoss(), RelativeL1(), L1CosineSim()
        if self.config.HFEN_TYPE == 'L1':
          l_hfen_type = nn.L1Loss()
        if self.config.HFEN_TYPE == 'MSE': 
          l_hfen_type = nn.MSELoss()
        if self.config.HFEN_TYPE == 'Charbonnier':
          l_hfen_type = CharbonnierLoss()
        if self.config.HFEN_TYPE == 'ElasticLoss':
          l_hfen_type = ElasticLoss()
        if self.config.HFEN_TYPE == 'RelativeL1':
          l_hfen_type = RelativeL1()        
        if self.config.HFEN_TYPE == 'L1CosineSim':
          l_hfen_type = L1CosineSim()

        _HFENLoss = HFENLoss(loss_f=l_hfen_type, kernel='log', kernel_size=15, sigma = 2.5, norm = False)
        self.add_module('_HFENLoss', _HFENLoss)
        # TVLoss
        _TVLoss = TVLoss(tv_type='tv', p = 1)
        self.add_module('_TVLoss', _TVLoss)
        # GradientLoss
        _GradientLoss = GradientLoss(loss_f = None, reduction='mean', gradientdir='2d')
        self.add_module('_GradientLoss', _GradientLoss)
        # ElasticLoss
        _ElasticLoss = ElasticLoss(a=0.2, reduction='mean')
        self.add_module('_ElasticLoss', _ElasticLoss)
        # RelativeL1 (todo?)
        _RelativeL1 = RelativeL1(eps=.01, reduction='mean')
        self.add_module('_RelativeL1', _RelativeL1)
        # L1CosineSim
        _L1CosineSim = L1CosineSim(loss_lambda=5, reduction='mean')
        self.add_module('_L1CosineSim', _L1CosineSim)
        # ClipL1
        _ClipL1 = ClipL1(clip_min=0.0, clip_max=10.0)
        self.add_module('_ClipL1', _ClipL1)
        # FFTloss
        _FFTloss = FFTloss(loss_f = torch.nn.L1Loss, reduction='mean')
        self.add_module('_FFTloss', _FFTloss)
        # OFLoss
        _OFLoss = OFLoss()
        self.add_module('_OFLoss', _OFLoss)
        # ColorLoss (untested)
        ds_f = torch.nn.AvgPool2d=((3, 3)) # kernel_size=5
        _ColorLoss = ColorLoss(loss_f = torch.nn.L1Loss, reduction='mean', ds_f=ds_f)
        self.add_module('_ColorLoss', _ColorLoss)
        # GPLoss
        _GPLoss = GPLoss(trace=False, spl_denorm=False)
        self.add_module('_GPLoss', _GPLoss)
        # CPLoss (SPL_ComputeWithTrace, SPLoss)
        _CPLoss = CPLoss(rgb=True, yuv=True, yuvgrad=True, trace=False, spl_denorm=False, yuv_denorm=False)
        self.add_module('_CPLoss', _CPLoss)
        # Contextual_Loss
        layers_weights = {'conv_1_1': 1.0, 'conv_3_2': 1.0}
        _Contextual_Loss = Contextual_Loss(layers_weights, crop_quarter=False, max_1d_size=100, 
            distance_type = 'cosine', b=1.0, band_width=0.5, 
            use_vgg = True, net = 'vgg19', calc_type = 'regular')
        self.add_module('_Contextual_Loss', _Contextual_Loss)

        """
        if self.config.DISCRIMINATOR == 'pixel':
          pixel_criterion = torch.nn.BCEWithLogitsLoss()
          self.add_module('pixel_criterion', pixel_criterion)

        if self.config.DISCRIMINATOR == 'patch':
          patch_criterion = torch.nn.BCEWithLogitsLoss()
          self.add_module('patch_criterion', patch_criterion)
        """

        if self.config.DISCRIMINATOR_CALC == 'BCEWithLogitsLoss':
          bce_criterion = nn.BCEWithLogitsLoss()
          self.add_module('bce_criterion', bce_criterion)

        if self.config.DISCRIMINATOR_CALC == 'MSELoss':
          mse_criterion = nn.MSELoss()
          self.add_module('mse_criterion', mse_criterion)

        self.gen_optimizer = optim.Adam(
            params=generator.parameters(),
            lr=float(config.LR),
            betas=(config.BETA1, config.BETA2)
        )

        if self.config.DISCRIMINATOR == 'default':
          self.dis_optimizer = optim.Adam(
              params=discriminator.parameters(),
              #params=self.PixelDiscriminator.parameters(),
              lr=float(config.LR) * float(config.D2G_LR),
              betas=(config.BETA1, config.BETA2)
          )
        if self.config.DISCRIMINATOR == 'pixel':
          self.dis_optimizer = optim.Adam(
              #params=discriminator.parameters(),
              params=self.PixelDiscriminator.parameters(),
              lr=float(config.LR) * float(config.D2G_LR),
              betas=(config.BETA1, config.BETA2)
          )

      
        if self.config.DISCRIMINATOR == 'patch':
          self.dis_optimizer = optim.Adam(
              #params=discriminator.parameters(),
              params=self.NLayerDiscriminator.parameters(),
              lr=float(config.LR) * float(config.D2G_LR),
              betas=(config.BETA1, config.BETA2)
          )




        self.use_amp = config.USE_AMP

    def process(self, images, edges, masks, mosaic_size=None):
        self.iteration += 1

        # zero optimizers
        self.gen_optimizer.zero_grad()
        self.dis_optimizer.zero_grad()

        if(mosaic_size != None):
          # resize image with random size. (256 currently hardcoded)
          #mosaic_size = int(random.triangular(int(min(256*0.01, 256*0.01)), int(min(256*0.2, 256*0.2)), int(min(256*0.0625, 256*0.0625))))
          images_mosaic = nnf.interpolate(images, size=(mosaic_size, mosaic_size), mode='nearest')
          images_mosaic = nnf.interpolate(images_mosaic, size=(256, 256), mode='nearest')
          images_mosaic = (images * (1 - masks).float()) + (images_mosaic * (masks).float())
          outputs = self(images_mosaic, edges, masks)
        else:
          outputs = self(images, edges, masks)

        if self.use_amp == 1:
          with torch.cuda.amp.autocast(): 
            # process outputs
            #outputs = self(images, edges, masks)
            gen_loss = 0
            dis_loss = 0


            # discriminator loss
            dis_input_real = images
            dis_input_fake = outputs.detach()
            #real_scores = Discriminator(DiffAugment(reals, policy=policy))

            if self.config.DISCRIMINATOR == 'default':
              dis_real, _ = self.discriminator(DiffAugment(dis_input_real, policy=policy))                    # in: [rgb(3)]
              dis_fake, _ = self.discriminator(DiffAugment(dis_input_fake, policy=policy))                    # in: [rgb(3)]

            if self.config.DISCRIMINATOR == 'pixel':
              dis_real = self.PixelDiscriminator(DiffAugment(dis_input_real, policy=policy))                    # in: [rgb(3)]
              dis_fake = self.PixelDiscriminator(DiffAugment(dis_input_fake, policy=policy))                    # in: [rgb(3)]

            if self.config.DISCRIMINATOR == 'patch':
              dis_real = self.NLayerDiscriminator(DiffAugment(dis_input_real, policy=policy))                    # in: [rgb(3)]
              dis_fake = self.NLayerDiscriminator(DiffAugment(dis_input_fake, policy=policy))                    # in: [rgb(3)]



            if self.config.DISCRIMINATOR_CALC == 'BCEWithLogitsLoss':
              dis_fake_loss = self.bce_criterion(dis_fake, torch.ones_like(dis_fake))
              dis_real_loss = self.bce_criterion(dis_real, torch.zeros_like(dis_real))

            if self.config.DISCRIMINATOR_CALC == 'MSELoss':
              dis_fake_loss = self.mse_criterion(dis_fake, torch.ones_like(dis_fake))
              dis_real_loss = self.mse_criterion(dis_real, torch.zeros_like(dis_real))


            dis_loss += ((dis_real_loss * self.config.DISCRIMINATOR_REAL_LOSS_WEIGHT) + (dis_fake_loss * self.config.DISCRIMINATOR_FAKE_LOSS_WEIGHT)) / 2


            # original generator loss
            # generator adversarial loss
            gen_input_fake = outputs
            
            if 'DEFAULT_GAN' in self.generator_loss:
              if self.config.DISCRIMINATOR == 'default':
                gen_fake, _ = self.discriminator(DiffAugment(gen_input_fake, policy=policy))                  # in: [rgb(3)]
              if self.config.DISCRIMINATOR == 'pixel':
                gen_fake = self.PixelDiscriminator(DiffAugment(gen_input_fake, policy=policy))                  # in: [rgb(3)]
              if self.config.DISCRIMINATOR == 'patch':
                gen_fake = self.NLayerDiscriminator(DiffAugment(gen_input_fake, policy=policy))                  # in: [rgb(3)]

              #gen_gan_loss = self.adversarial_loss(gen_fake, True, False) * self.config.INPAINT_ADV_LOSS_WEIGHT

              if self.config.GENERATOR_CALC == 'BCEWithLogitsLoss':
                gen_gan_loss = self.bce_criterion(gen_fake, torch.ones_like(gen_fake)) 

              if self.config.GENERATOR_CALC == 'MSELoss':
                gen_gan_loss = self.mse_criterion(gen_fake, torch.ones_like(gen_fake))

              gen_loss += gen_gan_loss * self.config.GENERATOR_CALC_WEIGHT

            # generator l1 loss
            if 'DEFAULT_L1' in self.generator_loss:
              gen_l1_loss = self.l1_loss(outputs, images) * self.config.L1_LOSS_WEIGHT / torch.mean(masks)
              gen_loss += gen_l1_loss

            # generator perceptual loss
            if 'Perceptual' in self.generator_loss:
              gen_content_loss = self.perceptual_loss(outputs, images)
              gen_content_loss = gen_content_loss * self.config.CONTENT_LOSS_WEIGHT
              gen_loss += gen_content_loss

            # generator style loss
            if 'Style' in self.generator_loss:
              gen_style_loss = self.style_loss(outputs * masks, images * masks)
              gen_style_loss = gen_style_loss * self.config.STYLE_LOSS_WEIGHT
              gen_loss += gen_style_loss


            # new loss 
            # CharbonnierLoss (L1) (already implemented?)
            if 'NEW_L1' in self.generator_loss:
              gen_loss += self.config.L1_LOSS_WEIGHT * self._CharbonnierLoss(outputs, images)
            
            # GANLoss (vanilla, lsgan, srpgan, nsgan, hinge, wgan-gp)
            if 'NEW_GAN' in self.generator_loss:
              gen_loss += self.config.NEW_GAN_WEIGHT * self._GANLoss(outputs, images)
            # GradientPenaltyLoss
            #gen_loss += self._GradientPenaltyLoss(outputs, images, interp_crit) # not sure what interp_crit is
            # HFENLoss
            if 'HFEN' in self.generator_loss:
              gen_loss += self.config.HFEN_WEIGHT * self._HFENLoss(outputs, images)
            # TVLoss
            if 'TV' in self.generator_loss:
              gen_loss += self.config.TV_WEIGHT * self._TVLoss(outputs)
            # GradientLoss
            #gen_loss += self._GradientLoss(outputs, images) # TypeError: 'NoneType' object is not callable
            # ElasticLoss
            if 'ElasticLoss' in self.generator_loss:
              gen_loss += self.config.ElasticLoss_WEIGHT * self._ElasticLoss(outputs, images)
            # RelativeL1 (todo?)
            if 'RelativeL1' in self.generator_loss:
              gen_loss += self.config.RelativeL1_WEIGHT * self._RelativeL1(outputs, images)
            # L1CosineSim
            if 'L1CosineSim' in self.generator_loss:
              gen_loss += self.config.L1CosineSim_WEIGHT * self._L1CosineSim(outputs, images)
            # ClipL1
            if 'ClipL1' in self.generator_loss:
              gen_loss += self.config.ClipL1_WEIGHT * self._ClipL1(outputs, images)
            # FFTloss
            if 'FFT' in self.generator_loss:
              gen_loss += self.config.FFT_WEIGHT * self._FFTloss(outputs, images)
            # OFLoss
            if 'OF' in self.generator_loss:
              gen_loss += self.config.OF_WEIGHT * self._OFLoss(outputs)
            # ColorLoss (untested)
            #gen_loss += self._ColorLoss(outputs, images) # TypeError: 'NoneType' object is not callable
            # GPLoss
            if 'GP' in self.generator_loss:
              gen_loss += self.config.GP_WEIGHT * self._GPLoss(outputs, images)
            # CPLoss (SPL_ComputeWithTrace, SPLoss)
            if 'CP' in self.generator_loss:
              gen_loss += self.config.CP_WEIGHT * self._CPLoss(outputs, images)
            # Contextual_Loss
            if 'Contextual' in self.generator_loss:
              gen_loss += self.config.Contextual_WEIGHT * self._Contextual_Loss(outputs, images)

        else:
          # process outputs
          #outputs = self(images, edges, masks)
          gen_loss = 0
          dis_loss = 0


          # discriminator loss
          dis_input_real = images
          dis_input_fake = outputs.detach()
          #real_scores = Discriminator(DiffAugment(reals, policy=policy))

          if self.config.DISCRIMINATOR == 'default':
            dis_real, _ = self.discriminator(DiffAugment(dis_input_real, policy=policy))                    # in: [rgb(3)]
            dis_fake, _ = self.discriminator(DiffAugment(dis_input_fake, policy=policy))                    # in: [rgb(3)]

          if self.config.DISCRIMINATOR == 'pixel':
            dis_real = self.PixelDiscriminator(DiffAugment(dis_input_real, policy=policy))                    # in: [rgb(3)]
            dis_fake = self.PixelDiscriminator(DiffAugment(dis_input_fake, policy=policy))                    # in: [rgb(3)]

          if self.config.DISCRIMINATOR == 'patch':
            dis_real = self.NLayerDiscriminator(DiffAugment(dis_input_real, policy=policy))                    # in: [rgb(3)]
            dis_fake = self.NLayerDiscriminator(DiffAugment(dis_input_fake, policy=policy))                    # in: [rgb(3)]



          if self.config.DISCRIMINATOR_CALC == 'BCEWithLogitsLoss':
            dis_fake_loss = self.bce_criterion(dis_fake, torch.ones_like(dis_fake))
            dis_real_loss = self.bce_criterion(dis_real, torch.zeros_like(dis_real))

          if self.config.DISCRIMINATOR_CALC == 'MSELoss':
            dis_fake_loss = self.mse_criterion(dis_fake, torch.ones_like(dis_fake))
            dis_real_loss = self.mse_criterion(dis_real, torch.zeros_like(dis_real))


          dis_loss += ((dis_real_loss * self.config.DISCRIMINATOR_REAL_LOSS_WEIGHT) + (dis_fake_loss * self.config.DISCRIMINATOR_FAKE_LOSS_WEIGHT)) / 2


          # original generator loss
          # generator adversarial loss
          gen_input_fake = outputs
          
          if 'DEFAULT_GAN' in self.generator_loss:
            if self.config.DISCRIMINATOR == 'default':
              gen_fake, _ = self.discriminator(DiffAugment(gen_input_fake, policy=policy))                  # in: [rgb(3)]
            if self.config.DISCRIMINATOR == 'pixel':
              gen_fake = self.PixelDiscriminator(DiffAugment(gen_input_fake, policy=policy))                  # in: [rgb(3)]
            if self.config.DISCRIMINATOR == 'patch':
              gen_fake = self.NLayerDiscriminator(DiffAugment(gen_input_fake, policy=policy))                  # in: [rgb(3)]

            #gen_gan_loss = self.adversarial_loss(gen_fake, True, False) * self.config.INPAINT_ADV_LOSS_WEIGHT

            if self.config.GENERATOR_CALC == 'BCEWithLogitsLoss':
              gen_gan_loss = self.bce_criterion(gen_fake, torch.ones_like(gen_fake)) 

            if self.config.GENERATOR_CALC == 'MSELoss':
              gen_gan_loss = self.mse_criterion(gen_fake, torch.ones_like(gen_fake))

            gen_loss += gen_gan_loss * self.config.GENERATOR_CALC_WEIGHT

          # generator l1 loss
          if 'DEFAULT_L1' in self.generator_loss:
            gen_l1_loss = self.l1_loss(outputs, images) * self.config.L1_LOSS_WEIGHT / torch.mean(masks)
            gen_loss += gen_l1_loss

          # generator perceptual loss
          if 'Perceptual' in self.generator_loss:
            gen_content_loss = self.perceptual_loss(outputs, images)
            gen_content_loss = gen_content_loss * self.config.CONTENT_LOSS_WEIGHT
            gen_loss += gen_content_loss

          # generator style loss
          if 'Style' in self.generator_loss:
            gen_style_loss = self.style_loss(outputs * masks, images * masks)
            gen_style_loss = gen_style_loss * self.config.STYLE_LOSS_WEIGHT
            gen_loss += gen_style_loss


          # new loss 
          # CharbonnierLoss (L1) (already implemented?)
          if 'NEW_L1' in self.generator_loss:
            gen_loss += self.config.L1_LOSS_WEIGHT * self._CharbonnierLoss(outputs, images)
          
          # GANLoss (vanilla, lsgan, srpgan, nsgan, hinge, wgan-gp)
          if 'NEW_GAN' in self.generator_loss:
            gen_loss += self.config.NEW_GAN_WEIGHT * self._GANLoss(outputs, images)
          # GradientPenaltyLoss
          #gen_loss += self._GradientPenaltyLoss(outputs, images, interp_crit) # not sure what interp_crit is
          # HFENLoss
          if 'HFEN' in self.generator_loss:
            gen_loss += self.config.HFEN_WEIGHT * self._HFENLoss(outputs, images)
          # TVLoss
          if 'TV' in self.generator_loss:
            gen_loss += self.config.TV_WEIGHT * self._TVLoss(outputs)
          # GradientLoss
          #gen_loss += self._GradientLoss(outputs, images) # TypeError: 'NoneType' object is not callable
          # ElasticLoss
          if 'ElasticLoss' in self.generator_loss:
            gen_loss += self.config.ElasticLoss_WEIGHT * self._ElasticLoss(outputs, images)
          # RelativeL1 (todo?)
          if 'RelativeL1' in self.generator_loss:
            gen_loss += self.config.RelativeL1_WEIGHT * self._RelativeL1(outputs, images)
          # L1CosineSim
          if 'L1CosineSim' in self.generator_loss:
            gen_loss += self.config.L1CosineSim_WEIGHT * self._L1CosineSim(outputs, images)
          # ClipL1
          if 'ClipL1' in self.generator_loss:
            gen_loss += self.config.ClipL1_WEIGHT * self._ClipL1(outputs, images)
          # FFTloss
          if 'FFT' in self.generator_loss:
            gen_loss += self.config.FFT_WEIGHT * self._FFTloss(outputs, images)
          # OFLoss
          if 'OF' in self.generator_loss:
            gen_loss += self.config.OF_WEIGHT * self._OFLoss(outputs)
          # ColorLoss (untested)
          #gen_loss += self._ColorLoss(outputs, images) # TypeError: 'NoneType' object is not callable
          # GPLoss
          if 'GP' in self.generator_loss:
            gen_loss += self.config.GP_WEIGHT * self._GPLoss(outputs, images)
          # CPLoss (SPL_ComputeWithTrace, SPLoss)
          if 'CP' in self.generator_loss:
            gen_loss += self.config.CP_WEIGHT * self._CPLoss(outputs, images)
          # Contextual_Loss
          if 'Contextual' in self.generator_loss:
            gen_loss += self.config.Contextual_WEIGHT * self._Contextual_Loss(outputs, images)


        """
        # create logs
        logs = [
            ("l_d2", dis_loss.item()),
            ("l_g2", gen_gan_loss.item()),
            ("l_l1", gen_l1_loss.item()),
            ("l_per", gen_content_loss.item()),
            ("l_sty", gen_style_loss.item()),
        ]
        """
        logs = [] # txt logs currently unsupported
        return outputs, gen_loss, dis_loss, logs

    def forward(self, images, edges, masks):
        if (self.mosaic_test == 1):
          # mosaic test
          images_masked = images
        else:
          images_masked = (images * (1 - masks).float()) + masks

        inputs = torch.cat((images_masked, edges), dim=1)
        outputs = self.generator(inputs)                                    # in: [rgb(3) + edge(1)]
        return outputs

    def backward(self, gen_loss=None, dis_loss=None):
        #dis_loss.backward(retain_graph = True)
        #gen_loss.backward()
        scaler.scale(dis_loss).backward(retain_graph = True) 
        scaler.scale(gen_loss).backward() 

        #self.gen_optimizer.step()
        #self.dis_optimizer.step()
        scaler.step(self.gen_optimizer) 
        scaler.step(self.dis_optimizer) 

        scaler.update() 